API

In [2]:
import pandas as pd

In [3]:
import requests as r
import json

In [4]:
with open("env.json", "r", encoding="utf-8") as file:
    config = json.load(file) 


In [5]:
#Get Daftar Data Dinamis BPS Bali
def getVarByDomain(domain='5100'):
    global config
    var_pages=[]
    var = r.get(f'https://webapi.bps.go.id/v1/api/list/model/var/domain/{domain}/key/{config['token']}/')
    pages=var.json()['data'][0]['pages']
    current_page = 1
    while current_page <= pages:
        url = f'https://webapi.bps.go.id/v1/api/list/model/var/domain/{domain}/page/{current_page}/key/{config['token']}/'
        var_page=r.get(url)
        var_pages.append(pd.json_normalize(var_page.json()['data'][1]))
        current_page=current_page+1
    var_pages_df = pd.concat(var_pages)
    return var_pages_df

In [6]:
def getPriodeByVar(domain='5100',varId='71'):
    global config
    current_page = 1
    url = f'https://webapi.bps.go.id/v1/api/list/model/th/domain/{domain}/var/{varId}/page/{current_page}/key/{config['token']}/'
    dt_json = r.get(url)
    pages = dt_json.json()['data'][0]['pages']
    dt_all = []
    while current_page <= pages:
        url = f'https://webapi.bps.go.id/v1/api/list/model/th/domain/{domain}/var/{varId}/page/{current_page}/key/{config['token']}/'
        dt = pd.json_normalize(dt_json.json()['data'][1])
        dt_all.append(dt)
        current_page = current_page+1
    dt_all = pd.concat(dt_all)
    dt_all['varId']=varId
    return dt_all

In [7]:
def crossJoin(left,right):
    if left.empty:
        left = right
    else:
        left = pd.merge(left,right,how='cross')
    return left
def getDataByVarId(varId='71',domain='5100'):
    global config
    dt_th = getPriodeByVar(domain=domain,varId=varId)
    dt_all = []
    for i in dt_th.index:
        th_id = dt_th.iloc[i]['th_id']
        url = f'https://webapi.bps.go.id/v1/api/list/model/data/domain/{domain}/var/{varId}/key/{config['token']}/th/{th_id}/'
        dt_json = r.get(url)
        attr = pd.DataFrame()
        if 'vervar' in dt_json.json().keys():
            vervar=pd.json_normalize(dt_json.json()['vervar'])
            vervar=vervar.rename(columns={'val': 'verval_val','label': 'vervar_label'})
            attr = crossJoin(attr,vervar)
        if 'var' in dt_json.json().keys():
            var=pd.json_normalize(dt_json.json()['var'])
            var=var.rename(columns={'val': 'var_val','label': 'var_label'})
            var=var[['var_val']]
            attr = crossJoin(attr,var)
        if 'turvar' in dt_json.json().keys():
            turvar=pd.json_normalize(dt_json.json()['turvar'])
            turvar=turvar.rename(columns={'val': 'turvar_val','label': 'turvar_label'})
            attr = crossJoin(attr,turvar)
        if 'tahun' in dt_json.json().keys():
            tahun=pd.json_normalize(dt_json.json()['tahun'])
            tahun=tahun.rename(columns={'val': 'tahun_val','label': 'tahun_label'})
            attr = crossJoin(attr,tahun)
        if 'turtahun' in dt_json.json().keys():
            turtahun=pd.json_normalize(dt_json.json()['turtahun'])
            turtahun=turtahun.rename(columns={'val': 'turtahun_val','label': 'turtahun_label'})
            attr = crossJoin(attr,turtahun)
        val_columns = [col for col in attr.columns if col.endswith('val')]
        attr['id']=attr[val_columns].astype('str').agg(''.join, axis=1)
        dt_content=pd.json_normalize(dt_json.json()['datacontent']).transpose().reset_index().rename(columns={'index':'content_val',0:'content_label'})
        attr = pd.merge(attr,dt_content,left_on='id',right_on='content_val',how='left')
        attr = attr[attr.content_label.notna()]
        dt_all.append(attr)
    dt_all=pd.concat(dt_all)
    return dt_all

(162)

In [199]:
# daftar_data=getVarByDomain(domain='5100')
# data=getDataByVarId(domain='5100',varId='162')
# data

In [ ]:
# proses cleaning
data = data[['verval_val','vervar_label','tahun_label','turtahun_label','content_label']]
data["vervar_label"] = data["vervar_label"].str.replace(r"<.*?>", "", regex=True)

# proses rename kolom
data_tb = data.rename(columns={
    'vervar_label'   : 'Lapangan_Usaha',
    'tahun_label'    : 'Tahun',
    'turtahun_label' : 'Triwulan',
    'content_label'  : 'Nilai'
})

# pivoting tabel (tambahkan Triwulan di columns)
# data_tb = data_tb.pivot_table(
    index=['verval_val','Lapangan_Usaha'],
    columns=['Tahun','Triwulan'],
    values='Nilai'
)

# multi index jadikan 1 kolom
data_tb.columns = ['_'.join([str(c) for c in col if c]).strip() if isinstance(col, tuple) else str(col) 
                   for col in data_tb.columns.values]
# 
# reset index
data_tb = data_tb.reset_index()


In [ ]:
data_tb

In [ ]:
data_tb.columns

In [ ]:
data_tb.to_excel("data_162.xlsx", sheet_name="PivotTriwulan", index=False)


Banyaknya Kecelakaan Lalu Lintas di Provinsi Bali (251)

In [109]:
daftar_data=getVarByDomain(domain='5100')
data=getDataByVarId(domain='5100',varId='251')
data

,verval_val,vervar_label,var_val,turvar_val,turvar_label,tahun_val,tahun_label,turtahun_val,turtahun_label,id,content_val,content_label
0,1,Kejadian,251,0,Tidak ada,124,2024,0,Tahun,125101240,125101240,8307
1,2,Meninggal Dunia,251,0,Tidak ada,124,2024,0,Tahun,225101240,225101240,269
2,3,Luka Berat,251,0,Tidak ada,124,2024,0,Tahun,325101240,325101240,76
3,4,Luka Ringan,251,0,Tidak ada,124,2024,0,Tahun,425101240,425101240,6783
0,1,Kejadian,251,0,Tidak ada,123,2023,0,Tahun,125101230,125101230,7467
...,...,...,...,...,...,...,...,...,...,...,...,...
3,4,Luka Ringan,251,0,Tidak ada,116,2016,0,Tahun,425101160,425101160,1938
0,1,Kejadian,251,0,Tidak ada,115,2015,0,Tahun,125101150,125101150,1441
1,2,Meninggal Dunia,251,0,Tidak ada,115,2015,0,Tahun,225101150,225101150,395
2,3,Luka Berat,251,0,Tidak ada,115,2015,0,Tahun,325101150,325101150,332


In [205]:
# data_tb = data[data['Tahun']]
# print(data)


In [206]:
# # proses cleaning
# data = data[['verval_val','vervar_label','tahun_label','turtahun_label','content_label']]
# data["vervar_label"] = data["vervar_label"].str.replace(r"<.*?>", "", regex=True)

# # proses rename kolom
# data_tb = data.rename(columns={
#     'vervar_label'   : 'Laka Lantas',
#     'tahun_label'    : 'Tahun',
#     'content_label'  : 'Nilai'
# })

# # pivoting tabel (tambahkan Triwulan di columns)
# data_tb = data_tb.pivot_table(
#     index=['verval_val','Laka Lantas'],
#     columns=['Tahun'],
#     values='Nilai'
# )

# # multi index jadikan 1 kolom
# data_tb.columns = ['_'.join([str(c) for c in col if c]).strip() if isinstance(col, tuple) else str(col) 
#                    for col in data_tb.columns.values]

# # reset index
# data_tb = data_tb.reset_index()


In [207]:
# data_tb

In [208]:
# from IPython.display import display

In [209]:
# cols = [col for col in data_tb.columns if col not in ['verval_val', 'Laka Lantas']]

# data_tb[cols] = data_tb[cols].fillna(0).astype(int)

# data_tb.rename(columns=lambda x: x.split('_')[-1] if '_' in x else x, inplace=True)

# # display(data_tb)

In [210]:
# data_tb.columns

In [211]:
# data_tb.to_excel("data_251.xlsx", sheet_name="Laka Lantas", index=False)


Jumlah Penumpang Tiba di Pelabuhan Provinsi Bali (253)

In [213]:
# daftar_data=getVarByDomain(domain='5100')
# data=getDataByVarId(domain='5100',varId='253')
# data


Jumlah Penumpang Berangkat pada Pelabuhan di Provinsi Bali (255)

In [214]:
# daftar_data=getVarByDomain(domain='5100')
# data=getDataByVarId(domain='5100',varId='255')
# data

Persentase Penduduk Miskin Provinsi Bali Menurut Kabupaten/Kota (125) belom di benerin kota urutan

In [215]:
# daftar_data=getVarByDomain(domain='5100')
# data=getDataByVarId(domain='5100',varId='125')
# data

In [216]:

# data_tb = data[['vervar_label', 'tahun_label', 'content_label']].copy()

# data_tb["vervar_label"] = data_tb["vervar_label"].str.replace(r"<.*?>", "", regex=True)

# data_tb = data_tb.rename(columns={
#     'vervar_label'  : 'Kabupaten/Kota',
#     'tahun_label'   : 'Tahun',
#     'content_label' : 'Nilai'
# })

# data_tb = data_tb.pivot_table(
#     index='Kabupaten/Kota',
#     columns='Tahun',
#     values='Nilai'
# )

# data_tb.columns = [str(col) for col in data_tb.columns]

# data_tb = data_tb.reset_index()

# tahun_cols = [col for col in data_tb.columns if col != 'Kabupaten/Kota']
# data_tb[tahun_cols] = data_tb[tahun_cols].astype(float)


In [217]:
# data_tb

In [218]:
# data_tb.to_excel("data_125 new.xlsx", sheet_name="Penduduk", index=False)


Garis Kemiskinan Provinsi Bali Menurut Kabupaten/Kota (197)

In [219]:
# daftar_data=getVarByDomain(domain='5100')
# data=getDataByVarId(domain='5100',varId='197')
# data

In [220]:
# import pandas as pd

# # --- ambil urutan kabupaten/kota sesuai API ---
# order = data['vervar_label'].astype(str).str.replace(r'<.*?>', '', regex=True).drop_duplicates().tolist()

# # --- ambil & bersihkan data ---
# data_tb = data[['vervar_label', 'tahun_label', 'content_label']].copy()
# data_tb["vervar_label"] = data_tb["vervar_label"].str.replace(r"<.*?>", "", regex=True)

# data_tb = data_tb.rename(columns={
#     'vervar_label'  : 'Kabupaten/Kota',
#     'tahun_label'   : 'Tahun',
#     'content_label' : 'Nilai'
# })

# # --- pivot table ---
# data_tb = data_tb.pivot_table(
#     index='Kabupaten/Kota',
#     columns='Tahun',
#     values='Nilai'
# )

# # --- gabungkan multi-index kolom jadi satu string ---
# data_tb.columns = [str(col) for col in data_tb.columns]

# # --- reset index supaya Kabupaten/Kota jadi kolom biasa ---
# data_tb = data_tb.reset_index()

# # --- ubah hanya kolom yang bisa jadi angka dan hapus .0 di belakang ---
# for col in data_tb.columns:
#     if col != 'Kabupaten/Kota':
#         data_tb[col] = pd.to_numeric(data_tb[col], errors='coerce')
#         # hapus .0 di belakang, ubah ke string
#         data_tb[col] = data_tb[col].apply(lambda x: str(int(x)) if pd.notnull(x) and x.is_integer() else str(x) if pd.notnull(x) else '')

# # --- ubah Kabupaten/Kota menjadi categorical sesuai urutan API ---
# data_tb['Kabupaten/Kota'] = pd.Categorical(data_tb['Kabupaten/Kota'], categories=order, ordered=True)

# # --- urutkan sesuai urutan API ---
# data_tb = data_tb.sort_values('Kabupaten/Kota').reset_index(drop=True)


In [221]:
# data_tb

In [222]:
# data_tb.to_excel("data_197.xlsx", sheet_name="Garis Kemiskinan", index=False)


Indeks Kedalaman Kemiskinan Provinsi Bali Menurut Kabupaten/Kota (199)

In [223]:
# daftar_data=getVarByDomain(domain='5100')
# data=getDataByVarId(domain='5100',varId='199')
# data

In [224]:
# # ambil urutan kabupaten/kota sesuai API
# order = data['vervar_label'].astype(str).str.replace(r'<.*?>','', regex=True).drop_duplicates().tolist()

# # pivot table seperti biasa
# data_tb = data[['vervar_label', 'tahun_label', 'content_label']].copy()
# data_tb["vervar_label"] = data_tb["vervar_label"].str.replace(r"<.*?>", "", regex=True)
# data_tb = data_tb.rename(columns={
#     'vervar_label'  : 'Kabupaten/Kota',
#     'tahun_label'   : 'Tahun',
#     'content_label' : 'Nilai'
# })
# data_tb = data_tb.pivot_table(
#     index='Kabupaten/Kota',
#     columns='Tahun',
#     values='Nilai'
# )

# # gabungkan multi-index kolom jadi string
# data_tb.columns = [str(col) for col in data_tb.columns]

# # reset index supaya Kabupaten/Kota jadi kolom biasa
# data_tb = data_tb.reset_index()

# # ubah hanya kolom yang bisa jadi angka
# for col in data_tb.columns:
#     if col != 'Kabupaten/Kota':
#         data_tb[col] = pd.to_numeric(data_tb[col], errors='coerce')

# # ubah Kabupaten/Kota menjadi categorical sesuai urutan API
# data_tb['Kabupaten/Kota'] = pd.Categorical(data_tb['Kabupaten/Kota'], categories=order, ordered=True)

# # urutkan sesuai urutan API
# data_tb = data_tb.sort_values('Kabupaten/Kota').reset_index(drop=True)


In [225]:
# data_tb

In [226]:
data_tb.to_excel("data_199.xlsx", sheet_name="Kedalaman Kemiskinan", index=False)

Persentase Penduduk Miskin Provinsi Bali Menurut Klasifikasi Daerah (70)

In [227]:
# daftar_data=getVarByDomain(domain='5100')
# data=getDataByVarId(domain='5100',varId='70')
# data

In [228]:
# # cek dulu struktur kolom
# print(data.columns.tolist())
# print(data.head())

# # ambil kolom utama (ubah kalau ternyata nama kolom berbeda)
# data_tb = data[['vervar_label', 'tahun_label', 'subround_label', 'content_label']].copy()

# # bersihkan teks HTML (kalau ada tag <span> dll.)
# data_tb["vervar_label"] = data_tb["vervar_label"].str.replace(r"<.*?>", "", regex=True)

# # ganti nama kolom biar lebih rapi
# data_tb = data_tb.rename(columns={
#     'vervar_label'    : 'Klasifikasi',
#     'tahun_label'     : 'Tahun',
#     'subround_label'  : 'Bulan',   # ini biasanya Maret/September
#     'content_label'   : 'Nilai'
# })

# # pivot tabel: baris = Klasifikasi, kolom = Tahun-Bulan, isi = Nilai
# data_tb = data_tb.pivot_table(
#     index='Klasifikasi',
#     columns=['Tahun', 'Bulan'],
#     values='Nilai'
# )

# # gabungkan multiindex kolom (Tahun, Bulan) jadi string "2020_Maret"
# data_tb.columns = [f"{thn}_{bln}" for thn, bln in data_tb.columns]

# # reset index supaya 'Klasifikasi' jadi kolom biasa
# data_tb = data_tb.reset_index()

# # tampilkan tabel rapi
# import pandas as pd
# pd.set_option("display.max_columns", None)  # tampilkan semua kolom
# pd.set_option("display.width", 150)         # lebar output
# print(data_tb.head())


Jumlah Penduduk Miskin Provinsi Bali Menurut Klasifikasi Daerah (54)

In [229]:
# daftar_data=getVarByDomain(domain='5100')
# data=getDataByVarId(domain='5100',varId='54')
# data

In [230]:
# data_tb = data[['vervar_label', 'tahun_label', 'turvar_label', 'content_label']].copy()

# # bersihkan html
# data_tb["vervar_label"] = data_tb["vervar_label"].str.replace(r"<.*?>", "", regex=True)

# # rename kolom agar rapi
# data_tb = data_tb.rename(columns={
#     'vervar_label'  : 'Klasifikasi Daerah',
#     'tahun_label'   : 'Tahun',
#     'turvar_label'  : 'Bulan',
#     'content_label' : 'Nilai'
# })

# # filter hanya bulan Maret & September
# data_tb = data_tb[data_tb['Bulan'].isin(['Maret', 'September'])]

# # pivot: index = daerah, kolom = Tahun + Bulan
# data_tb = data_tb.pivot_table(
#     index='Klasifikasi Daerah',
#     columns=['Tahun', 'Bulan'],
#     values='Nilai'
# )

# # gabungkan multi-index kolom jadi string "Tahun_Bulan"
# data_tb.columns = [f"{thn}_{bln}" for thn, bln in data_tb.columns]

# # reset index supaya daerah jadi kolom biasa
# data_tb = data_tb.reset_index()

# # ubah kolom numerik ke angka
# for col in data_tb.columns:
#     if col != 'Klasifikasi Daerah':
#         data_tb[col] = pd.to_numeric(data_tb[col], errors='coerce')

# data_tb


In [231]:
# data_tb.to_excel("data_54.xlsx", sheet_name="Penduduk Miskin", index=False)

Angka Melek Huruf Provinsi Bali Menurut Kabupaten/Kota dan Jenis Kelamin (71)

In [232]:
# daftar_data=getVarByDomain(domain='5100')
# data=getDataByVarId(domain='5100',varId='71')
# data

In [233]:
# # ambil data relevan
# data_tb = data[['vervar_label', 'tahun_label', 'turvar_label', 'content_label']].copy()

# # bersihkan nama
# data_tb["vervar_label"] = data_tb["vervar_label"].str.replace(r"<.*?>", "", regex=True)

# # ganti nama kolom
# data_tb = data_tb.rename(columns={
#     # 'vervar_label'  : 'Kabupaten/Kota',
#     'tahun_label'   : 'Tahun',
#     'turvar_label' : 'Jenis Kelamin',
#     'content_label' : 'Nilai'
# })

# # pivot supaya kolom ada Tahun + Jenis Kelamin
# data_tb = data_tb.pivot_table(
#     index='Kabupaten/Kota',
#     columns=['Jenis Kelamin', 'Tahun'],
#     values='Nilai'
# )

# # gabungkan multiindex kolom menjadi satu string: "Jenis Kelamin Tahun"
# data_tb.columns = [f"{col[0]} {col[1]}" for col in data_tb.columns]

# # reset index supaya Kabupaten/Kota tetap jadi kolom
# data_tb = data_tb.reset_index()

# # ubah ke numeric
# for col in data_tb.columns:
#     if col != 'Kabupaten/Kota':
#         data_tb[col] = pd.to_numeric(data_tb[col], errors='coerce')

# data_tb


In [234]:

# # --- ambil & bersihkan ---
# data_tb = data[['vervar_label','tahun_label','content_label','turvar_label']].copy()
# data_tb['kabupaten_kota'] = (data_tb['vervar_label'].astype(str)
#                              .str.replace(r'<.*?>','', regex=True)
#                              .str.replace(r'\s+',' ', regex=True)
#                              .str.strip())

# # --- filter tahun >= 2015 dan pastikan numeric ---
# df = data_tb.rename(columns={'tahun_label':'tahun','content_label':'nilai','turvar_label':'jenis'})
# df['tahun'] = pd.to_numeric(df['tahun'], errors='coerce')
# df['nilai'] = pd.to_numeric(df['nilai'], errors='coerce')
# df = df.dropna(subset=['tahun','nilai'])
# df['tahun'] = df['tahun'].astype(int)
# df = df[df['tahun'] >= 2015]

# # --- pivot langsung ambil dari API (Laki-laki, Perempuan, dan Laki-laki + Perempuan) ---
# pivot = df[df['jenis'].isin(['Laki-laki','Perempuan','Laki-laki + Perempuan'])]
# wide = pivot.pivot_table(
#     index='kabupaten_kota',
#     columns=['tahun','jenis'],
#     values='nilai',
#     aggfunc='first'
# )

# # --- urut kolom ---
# jenis_prioritas = ['Laki-laki','Perempuan','Laki-laki + Perempuan']
# sorted_cols = []
# for th in sorted(wide.columns.levels[0]):
#     for jk in jenis_prioritas:
#         if (th,jk) in wide.columns:
#             sorted_cols.append((th,jk))
# wide = wide[sorted_cols]

# # --- urut baris sesuai urutan awal ---
# order = df['kabupaten_kota'].drop_duplicates().tolist()
# wide.index = pd.CategoricalIndex(wide.index, categories=order, ordered=True)
# wide = wide.sort_index()


In [235]:
# wide

Angka Partisipasi Murni (APM) Provinsi Bali Menurut Jenjang Pendidikan dan Jenis Kelamin (79)


In [74]:

data_79=getDataByVarId(domain='5100',varId='79')
data_79

,verval_val,vervar_label,var_val,turvar_val,turvar_label,tahun_val,tahun_label,turtahun_val,turtahun_label,id,content_val,content_label
0,1,SD,79,41,Laki-laki,122,2022,0,Tahun,179411220,179411220,97.53
1,1,SD,79,42,Perempuan,122,2022,0,Tahun,179421220,179421220,97.39
2,1,SD,79,43,Laki-Laki + Perempuan,122,2022,0,Tahun,179431220,179431220,97.46
3,2,SLTP,79,41,Laki-laki,122,2022,0,Tahun,279411220,279411220,87.42
4,2,SLTP,79,42,Perempuan,122,2022,0,Tahun,279421220,279421220,86.30
...,...,...,...,...,...,...,...,...,...,...,...,...
7,3,SLTA,79,42,Perempuan,113,2013,0,Tahun,379421130,379421130,62.48
8,3,SLTA,79,43,Laki-Laki + Perempuan,113,2013,0,Tahun,379431130,379431130,67.10
9,4,Perguruan Tinggi,79,41,Laki-laki,113,2013,0,Tahun,479411130,479411130,19.81
10,4,Perguruan Tinggi,79,42,Perempuan,113,2013,0,Tahun,479421130,479421130,15.67


In [237]:
# data.columns.tolist()


In [76]:
# --- ambil & bersihkan untuk urutan asli ---
data_79 = data_79[['vervar_label','tahun_label','content_label']].copy()
data_79['Jenjang Pendidikan'] = (data_79['vervar_label'].astype(str)
                          .str.replace(r'<.*?>','', regex=True)
                          .str.replace(r'\s+',' ', regex=True)
                          .str.strip())
order = data_79['Jenjang Pendidikan'].drop_duplicates().tolist()

# --- siapkan untuk pivot ---
df = data_79.rename(columns={'tahun_label':'tahun','content_label':'nilai'})[['Jenjang Pendidikan','tahun','nilai']]
df['tahun'] = pd.to_numeric(df['tahun'], errors='coerce')
df = df.dropna(subset=['tahun']).copy()
df['tahun'] = df['tahun'].astype(int)

# --- PIVOT: kolom = tahun (MELEBAR) ---
wide = df.pivot_table(index='Jenjang Pendidikan', columns='tahun', values='nilai', aggfunc='first', sort=False)

# urut kolom (tahun naik) + baris sesuai urutan awal
wide = wide.reindex(sorted(wide.columns), axis=1)
wide.index = pd.CategoricalIndex(wide.index, categories=[k for k in order if k in wide.index], ordered=True)
wide = wide.sort_index()

In [77]:
wide

tahun,2013,2014,2015,2016,2017,2018,2019,2020,2021,2022
Jenjang Pendidikan,,,,,,,,,,
SD,95.10,96.23,96.61,97.10,95.66,96.60,97.04,96.71,97.38,97.53
SLTP,78.78,83.37,85.60,86.15,86.55,87.21,85.85,86.06,87.21,87.42
SLTA,71.15,74.64,77.58,74.80,74.44,72.38,73.55,74.18,74.01,74.91
Perguruan Tinggi,19.81,23.32,22.81,22.69,23.53,22.45,24.17,24.68,25.28,26.97


In [78]:
wide.reset_index().to_excel("data_79.xlsx", index=False, engine="openpyxl")

Angka Partisipasi Kasar (APK) Provinsi Bali Menurut Jenjang Pendidikan dan Jenis Kelamin (82)

In [240]:

# data_82=getDataByVarId(domain='5100',varId='82')
# data_82

In [241]:
# # pivot table langsung ke data_82
# data_82 = data_82.pivot_table(
#     index=['Jenjang Pendidikan'],
#     columns=['Tahun', 'Jenis Kelamin'],
#     values='Nilai'
# )

# # ubah multi-index kolom jadi single string
# data_82.columns = ['_'.join([str(c) for c in col if c]) for col in data_82.columns]

# # reset index supaya 'Jenjang Pendidikan' jadi kolom biasa
# data_82 = data_82.reset_index()


In [242]:
# data_82

Persentase Penduduk Usia 25 Tahun ke Atas dengan Pendidikan SMA ke Atas di Provinsi Bali Menurut Kabupaten/Kota dan Jenis Kelamin (398)

In [243]:
# data_398=getDataByVarId(domain='5100',varId='398')
# data_398

In [244]:
# # proses cleaning
# data_398 = data_398[['verval_val','vervar_label','turvar_label','tahun_label','content_label']]
# data_398["vervar_label"] = data_398["vervar_label"].str.replace(r"<.*?>", "", regex=True)

# # proses rename kolom
# data_tb = data_398.rename(columns={
#     'vervar_label'   : 'Kabupaten/Kota',
#     'turvar_label'   : 'Jenis Kelamin',
#     'tahun_label'    : 'Tahun',
#     'content_label'  : 'Nilai'
# })

# # pivoting tabel (tambahkan Triwulan di columns)
# data_tb = data_tb.pivot_table(
#     index=['Kabupaten/Kota'],
#     columns=['Jenis Kelamin','Tahun'],
#     values='Nilai'
# )

# # multi index jadikan 1 kolom
# data_tb.columns = ['_'.join([str(c) for c in col if c]).strip() if isinstance(col, tuple) else str(col) 
#                    for col in data_tb.columns.values]
# # 
# # reset index
# data_tb = data_tb.reset_index()


In [245]:
# # --- proses cleaning ---
# data_398 = data_398[['verval_val','vervar_label','turvar_label','tahun_label','content_label']].copy()
# data_398["vervar_label"] = data_398["vervar_label"].str.replace(r"<.*?>", "", regex=True)

# # --- rename kolom ---
# data_tb = data_398.rename(columns={
#     'vervar_label'   : 'Kabupaten/Kota',
#     'turvar_label'   : 'Jenis Kelamin',
#     'tahun_label'    : 'Tahun',
#     'content_label'  : 'Nilai'
# })

# # --- ambil urutan otomatis dari API ---
# api_order = data_398['vervar_label'].drop_duplicates().tolist()
# data_tb['Kabupaten/Kota'] = pd.Categorical(data_tb['Kabupaten/Kota'], categories=api_order, ordered=True)

# # --- pivoting tabel ---
# data_tb = data_tb.pivot_table(
#     index=['Kabupaten/Kota'],
#     columns=['Jenis Kelamin','Tahun'],
#     values='Nilai'
# )

# # --- gabungkan multi index jadi satu kolom ---
# data_tb.columns = ['_'.join([str(c) for c in col if c]).strip() if isinstance(col, tuple) else str(col) 
#                    for col in data_tb.columns.values]

# # --- reset index (urut sesuai API otomatis) ---
# data_tb = data_tb.reset_index()

# data_tb


In [246]:
# data_398.to_excel("data_398.xlsx", sheet_name="Penduduk_25_Tahun", index=False)

Proporsi Perempuan Pernah Kawin Usia 15-49 Tahun yang Pernah Melahirkan Anak Lahir Hidup dalam 2 Tahun Terakhir Tidak di Fasilitas Kesehatan (MTF) (397)

In [247]:
# data_397=getDataByVarId(domain='5100',varId='397')
# data_397

In [248]:
# # --- proses cleaning ---
# data_397 = data_397[['verval_val','vervar_label','tahun_label','content_label']].copy()
# data_397["vervar_label"] = data_397["vervar_label"].str.replace(r"<.*?>", "", regex=True)

# # --- rename kolom ---
# data_tb = data_397.rename(columns={
#     'vervar_label'   : 'Kabupaten/Kota',
#     'tahun_label'    : 'Tahun',
#     'content_label'  : 'Nilai'
# })

# # --- ambil urutan otomatis dari API ---
# api_order = data_397['vervar_label'].drop_duplicates().tolist()
# data_tb['Kabupaten/Kota'] = pd.Categorical(data_tb['Kabupaten/Kota'], categories=api_order, ordered=True)

# # --- pivoting tabel ---
# data_tb = data_tb.pivot_table(
#     index=['Kabupaten/Kota'],
#     columns=['Tahun'],
#     values='Nilai'
# )

# # --- gabungkan multi index jadi satu kolom ---
# data_tb.columns = ['_'.join([str(c) for c in col if c]).strip() if isinstance(col, tuple) else str(col) 
#                    for col in data_tb.columns.values]

# # --- reset index (urut sesuai API otomatis) ---
# data_tb = data_tb.reset_index()

# data_tb


In [249]:
# data_397.to_excel("data_397.xlsx", sheet_name="MTF", index=False)

Proporsi Perempuan Pernah Kawin Usia 15-49 Tahun yang Melahirkan Anak Lahir Hidup Yang Pertama Kali Berumur Kurang dari 20 Tahun (MHPK20) (396)

In [250]:
# data_396=getDataByVarId(domain='5100',varId='396')
# data_396

In [251]:
# # --- proses cleaning ---
# data_396 = data_396[['verval_val','vervar_label','tahun_label','content_label']].copy()
# data_396["vervar_label"] = data_396["vervar_label"].str.replace(r"<.*?>", "", regex=True)

# # --- rename kolom ---
# data_tb = data_396.rename(columns={
#     'vervar_label'   : 'Kabupaten/Kota',
#     'tahun_label'    : 'Tahun',
#     'content_label'  : 'Nilai'
# })

# # --- ambil urutan otomatis dari API ---
# api_order = data_396['vervar_label'].drop_duplicates().tolist()
# data_tb['Kabupaten/Kota'] = pd.Categorical(data_tb['Kabupaten/Kota'], categories=api_order, ordered=True)

# # --- pivoting tabel ---
# data_tb = data_tb.pivot_table(
#     index=['Kabupaten/Kota'],
#     columns=['Tahun'],
#     values='Nilai'
# )

# # --- gabungkan multi index jadi satu kolom ---
# data_tb.columns = ['_'.join([str(c) for c in col if c]).strip() if isinstance(col, tuple) else str(col) 
#                    for col in data_tb.columns.values]

# # --- reset index (urut sesuai API otomatis) ---
# data_tb = data_tb.reset_index()

# data_tb


In [252]:
# data_396.to_excel("data_396.xlsx", sheet_name="MHPK20", index=False)

Indeks Ketimpangan Gender (IKG) Provinsi Bali Menurut Kabupaten/Kota (395)

In [253]:
# data_395=getDataByVarId(domain='5100',varId='395')
# data_395

In [254]:
# # --- proses cleaning ---
# data_395 = data_395[['verval_val','vervar_label','tahun_label','content_label']].copy()
# data_395["vervar_label"] = data_395["vervar_label"].str.replace(r"<.*?>", "", regex=True)

# # --- rename kolom ---
# data_tb = data_395.rename(columns={
#     'vervar_label'   : 'Kabupaten/Kota',
#     'tahun_label'    : 'Tahun',
#     'content_label'  : 'Nilai'
# })

# # --- ambil urutan otomatis dari API ---
# api_order = data_395['vervar_label'].drop_duplicates().tolist()
# data_tb['Kabupaten/Kota'] = pd.Categorical(data_tb['Kabupaten/Kota'], categories=api_order, ordered=True)

# # --- pivoting tabel ---
# data_tb = data_tb.pivot_table(
#     index=['Kabupaten/Kota'],
#     columns=['Tahun'],
#     values='Nilai'
# )

# # --- gabungkan multi index jadi satu kolom ---
# data_tb.columns = ['_'.join([str(c) for c in col if c]).strip() if isinstance(col, tuple) else str(col) 
#                    for col in data_tb.columns.values]

# # --- reset index (urut sesuai API otomatis) ---
# data_tb = data_tb.reset_index()

# data_tb


In [255]:
# data_395.to_excel("data_395.xlsx", sheet_name="IKG", index=False)

IPM Provinsi Bali Menurut Kabupaten/Kota dan Jenis Kelamin (347)

In [ ]:
# daftar_var = getVarByDomain(domain='5100')
# print(daftar_var)  # lihat varId yang valid


    var_id                       title                         sub_id                    sub_name                    subcsa_id   subcsa_name   def  \
0       1   Inflasi Tahunan Kota Denpasar, Singaraja, dan ...      3                                        Inflasi     536        Harga-Harga       
1      47   Inflasi Tahunan Kota Singaraja Menurut 7 Kelom...      3                                        Inflasi     536        Harga-Harga       
2      65   Inflasi Tahunan Kota Denpasar Menurut 7 Kelomp...      3                                        Inflasi     536        Harga-Harga       
3     314   Inflasi Bulanan Kota Denpasar, Singaraja, dan ...      3                                        Inflasi     536        Harga-Harga       
4     362   Indeks Harga Konsumen (IHK) Bulanan Kota Denpa...      3                                        Inflasi     536        Harga-Harga       
..     ...                                                ...     ...                               

In [ ]:
# data_347=getDataByVarId(domain='5100',varId='347')
# data_347

AttributeError: 'NoneType' object has no attribute 'keys'

Persentase Keterlibatan Perempuan Sebagai Tenaga Profesional di Provinsi Bali Menurut Kabupaten/Kota (178)

In [ ]:
# data_178=getDataByVarId(domain='5100',varId='178')
# data_178

,verval_val,vervar_label,var_val,turvar_val,turvar_label,tahun_val,tahun_label,turtahun_val,turtahun_label,id,content_val,content_label
0,1,Kab. Jembrana,178,0,Tidak ada,124,2024,0,Tahun,117801240,117801240,46.67
1,2,Kab. Tabanan,178,0,Tidak ada,124,2024,0,Tahun,217801240,217801240,49.25
2,3,Kab. Badung,178,0,Tidak ada,124,2024,0,Tahun,317801240,317801240,42.01
3,4,Kab. Gianyar,178,0,Tidak ada,124,2024,0,Tahun,417801240,417801240,59.00
4,5,Kab. Klungkung,178,0,Tidak ada,124,2024,0,Tahun,517801240,517801240,47.95
...,...,...,...,...,...,...,...,...,...,...,...,...
5,6,Kab. Bangli,178,0,Tidak ada,115,2015,0,Tahun,617801150,617801150,47.33
6,7,Kab. Karangasem,178,0,Tidak ada,115,2015,0,Tahun,717801150,717801150,40.40
7,8,Kab. Buleleng,178,0,Tidak ada,115,2015,0,Tahun,817801150,817801150,34.43
8,9,Kota Denpasar,178,0,Tidak ada,115,2015,0,Tahun,917801150,917801150,50.43


In [ ]:
# # --- proses cleaning ---
# data_178 = data_178[['verval_val','vervar_label','tahun_label','content_label']].copy()
# data_178["vervar_label"] = data_178["vervar_label"].str.replace(r"<.*?>", "", regex=True)

# # --- rename kolom ---
# data_tb = data_178.rename(columns={
#     'vervar_label'   : 'Kabupaten/Kota',
#     'tahun_label'    : 'Tahun',
#     'content_label'  : 'Nilai'
# })

# # --- ambil urutan otomatis dari API ---
# api_order = data_178['vervar_label'].drop_duplicates().tolist()
# data_tb['Kabupaten/Kota'] = pd.Categorical(data_tb['Kabupaten/Kota'], categories=api_order, ordered=True)

# # --- pivoting tabel ---
# data_tb = data_tb.pivot_table(
#     index=['Kabupaten/Kota'],
#     columns=['Tahun'],
#     values='Nilai'
# )

# # --- gabungkan multi index jadi satu kolom ---
# data_tb.columns = ['_'.join([str(c) for c in col if c]).strip() if isinstance(col, tuple) else str(col) 
#                    for col in data_tb.columns.values]

# # --- reset index (urut sesuai API otomatis) ---
# data_tb = data_tb.reset_index()

# data_tb


C:\Users\user\AppData\Local\Temp\ipykernel_13044\4181447160.py:17: FutureWarning: The default value of observed=False is deprecated and will change to observed=True in a future version of pandas. Specify observed=False to silence this warning and retain the current behavior
  data_tb = data_tb.pivot_table(


,Kabupaten/Kota,2015,2016,2017,2018,2019,2020,2021,2022,2023,2024
0,Kab. Jembrana,42.36,NaN,49.33,46.06,51.91,52.45,51.76,47.68,55.08,46.67
1,Kab. Tabanan,41.65,NaN,46.46,46.61,51.52,46.67,49.15,52.61,45.14,49.25
2,Kab. Badung,47.23,NaN,41.73,45.24,50.16,49.65,51.57,53.77,54.80,42.01
3,Kab. Gianyar,43.23,NaN,44.87,45.33,51.80,51.45,54.94,53.29,55.15,59.00
4,Kab. Klungkung,48.89,NaN,36.14,43.77,46.58,47.52,49.04,52.69,48.59,47.95
5,Kab. Bangli,47.33,NaN,37.47,42.61,43.46,41.79,42.93,54.27,46.37,54.04
6,Kab. Karangasem,40.40,NaN,36.24,37.71,37.56,38.51,46.69,41.24,47.19,41.43
7,Kab. Buleleng,34.43,NaN,41.16,40.80,45.50,45.53,43.54,51.12,42.27,41.96
8,Kota Denpasar,50.43,NaN,47.38,48.59,48.02,50.41,52.43,50.70,51.28,47.98
9,Provinsi Bali,45.46,49.28,44.17,45.58,48.05,49.12,50.89,51.25,50.36,47.60


In [ ]:
# data_178.to_excel("data_178.xlsx", sheet_name="Keterlibatan_Perempuan_Tenaga_Prof", index=False)

C:\Users\user\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.12_qbz5n2kfra8p0\LocalCache\local-packages\Python312\site-packages\openpyxl\workbook\child.py:99: UserWarning: Title is more than 31 characters. Some applications may not be able to read the file
  warnings.warn("Title is more than 31 characters. Some applications may not be able to read the file")


Produksi Daging Ternak Provinsi Bali Menurut Kabupaten/Kota(188)

In [ ]:
# data_188=getDataByVarId(domain='5100',varId='188')
# data_188

,verval_val,vervar_label,var_val,turvar_val,turvar_label,tahun_val,tahun_label,turtahun_val,turtahun_label,id,content_val,content_label
0,1,Kab. Jembrana,188,88,Sapi,122,2022,0,Tahun,1188881220,1188881220,285.00
1,1,Kab. Jembrana,188,89,Kerbau,122,2022,0,Tahun,1188891220,1188891220,0.00
2,1,Kab. Jembrana,188,90,Babi,122,2022,0,Tahun,1188901220,1188901220,1357.00
3,1,Kab. Jembrana,188,91,Kambing,122,2022,0,Tahun,1188911220,1188911220,152.00
4,2,Kab. Tabanan,188,88,Sapi,122,2022,0,Tahun,2188881220,2188881220,246.00
...,...,...,...,...,...,...,...,...,...,...,...,...
35,9,Kota Denpasar,188,91,Kambing,113,2013,0,Tahun,9188911130,9188911130,172.25
36,10,<b>Provinsi Bali</b>,188,88,Sapi,113,2013,0,Tahun,10188881130,10188881130,7535.02
37,10,<b>Provinsi Bali</b>,188,89,Kerbau,113,2013,0,Tahun,10188891130,10188891130,9.15
38,10,<b>Provinsi Bali</b>,188,90,Babi,113,2013,0,Tahun,10188901130,10188901130,102204.74


In [ ]:
data_188 = data_188[['verval_val','vervar_label','turvar_label','tahun_label','content_label']].copy()
data_188["vervar_label"] = data_188["vervar_label"].str.replace(r"<.*?>", "", regex=True)

data_tb = data_188.rename(columns={
    'vervar_label'   : 'Kabupaten/Kota',
    'turvar_label'   : 'Hewan Ternak',
    'tahun_label'    : 'Tahun',
    'content_label'  : 'Nilai'
})

api_order_kab = data_188['vervar_label'].drop_duplicates().tolist()
api_order_hewan = data_188['turvar_label'].drop_duplicates().tolist()  

# set kategori supaya urut sesuai API
data_tb['Kabupaten/Kota'] = pd.Categorical(data_tb['Kabupaten/Kota'], categories=api_order_kab, ordered=True)
data_tb['Hewan Ternak'] = pd.Categorical(data_tb['Hewan Ternak'], categories=api_order_hewan, ordered=True)

data_tb = data_tb.pivot_table(
    index=['Kabupaten/Kota'],
    columns=['Tahun','Hewan Ternak'],
    values='Nilai'
)

# --- gabungkan multi index jadi satu kolom dengan format Tahun_HewanTernak ---
data_tb.columns = [f"{col[0]}_{col[1]}" for col in data_tb.columns]

# --- reset index (urut sesuai API otomatis) ---
data_tb = data_tb.reset_index()

data_tb


C:\Users\user\AppData\Local\Temp\ipykernel_13044\4208395115.py:22: FutureWarning: The default value of observed=False is deprecated and will change to observed=True in a future version of pandas. Specify observed=False to silence this warning and retain the current behavior
  data_tb = data_tb.pivot_table(


,Kabupaten/Kota,2013_Sapi,2013_Kerbau,2013_Babi,2013_Kambing,2014_Sapi,2014_Kerbau,2014_Babi,2014_Kambing,2015_Sapi,2015_Kerbau,2015_Babi,2015_Kambing,2016_Sapi,2016_Kerbau,2016_Babi,2016_Kambing,2017_Sapi,2017_Kerbau,2017_Babi,2017_Kambing,2018_Sapi,2018_Kerbau,2018_Babi,2018_Kambing,2019_Sapi,2019_Kerbau,2019_Babi,2019_Kambing,2020_Sapi,2020_Kerbau,2020_Babi,2020_Kambing,2021_Sapi,2021_Kerbau,2021_Babi,2021_Kambing,2022_Sapi,2022_Kerbau,2022_Babi,2022_Kambing
0,Kab. Jembrana,443.23,NaN,4741.89,202.02,473.56,0.00,6184.37,397.40,481.95,1.64,7354.63,549.06,439.23,2.11,8224.87,497.52,404.01,NaN,6406.38,437.86,400.79,2.05,6148.37,392.86,368.86,1.39,40168.52,269.85,357.0,0.62,4650.0,290.0,330.0,0.0,5178.0,347.0,285.0,0.0,1357.0,152.0
1,Kab. Tabanan,524.47,1.00,14558.03,338.68,605.72,1.34,17263.59,491.54,549.05,0.00,17184.82,405.82,492.63,NaN,20685.15,302.55,473.10,NaN,21275.12,309.37,436.15,0.77,25080.02,315.29,420.52,0.00,23552.14,265.75,407.0,0.00,14770.0,370.0,261.0,0.0,13497.0,331.0,246.0,0.0,879.0,98.0
2,Kab. Badung,1511.94,NaN,7533.59,110.83,2035.92,0.00,8938.99,73.20,2155.16,0.00,7829.49,83.37,2264.08,NaN,20334.02,82.62,2354.30,NaN,14418.69,95.49,2268.73,0.00,13865.33,56.75,2096.97,0.00,13561.81,69.59,1518.0,0.00,12847.0,27.0,1218.0,0.0,12415.0,36.0,1052.0,0.0,17.0,2.0
3,Kab. Gianyar,488.20,NaN,5446.57,84.67,285.40,0.00,6388.72,160.79,190.19,0.00,11751.21,229.78,146.60,NaN,12675.51,205.99,174.28,NaN,14634.81,238.60,160.04,0.00,23343.09,254.22,162.46,0.00,40789.69,230.19,172.0,0.00,67113.0,206.0,188.0,0.0,61003.0,56.0,220.0,0.0,191.0,21.0
4,Kab. Klungkung,327.50,NaN,2569.89,73.64,158.91,0.00,2378.33,62.49,147.35,0.00,3915.87,76.48,232.23,NaN,3578.61,91.59,144.97,NaN,3061.71,66.12,150.30,0.00,3938.38,83.56,124.82,0.00,3549.20,62.06,145.0,0.00,4058.0,39.0,142.0,0.0,3365.0,47.0,109.0,0.0,131.0,15.0
5,Kab. Bangli,59.12,0.16,10990.72,54.90,59.17,1.26,13403.98,79.98,35.36,5.04,19316.04,87.04,24.77,1.26,16285.75,94.31,17.72,1.26,15073.72,101.64,20.40,0.00,10012.47,151.00,18.70,0.65,11840.01,145.84,11.0,0.63,10974.0,71.0,7.0,0.0,10746.0,66.0,318.0,0.0,85.0,9.0
6,Kab. Karangasem,519.76,2.00,10033.63,58.72,543.61,5.39,10720.48,70.14,566.96,4.04,11731.41,110.33,617.15,4.42,9464.52,75.70,394.47,2.53,10721.11,103.07,379.03,0.21,4603.62,39.47,426.83,0.00,5599.40,76.10,497.0,0.16,9897.0,46.0,396.0,0.0,11437.0,21.0,5.0,0.0,209.0,23.0
7,Kab. Buleleng,1399.14,5.99,24295.32,205.59,1229.12,16.17,27404.79,258.21,1478.48,7.35,40240.74,272.48,1418.80,20.21,46819.53,282.15,1414.26,16.42,59434.44,281.38,1614.04,1.89,48332.64,466.50,1602.59,0.00,40923.37,514.09,1226.0,13.00,37190.0,371.0,882.0,19.0,31807.0,366.0,462.0,19.0,836.0,94.0
8,Kota Denpasar,2261.66,NaN,22035.10,172.25,2289.59,0.00,33024.29,386.63,2139.06,0.00,31635.28,785.12,2175.24,0.21,28467.09,1770.50,1838.26,0.21,16014.38,1278.21,1777.49,0.00,19936.28,146.90,1963.50,0.00,16349.03,112.69,1565.0,0.00,13691.0,419.0,1226.0,0.0,9257.0,690.0,1367.0,0.0,1514.0,169.0
9,Provinsi Bali,7535.02,9.15,102204.74,1301.30,7681.00,24.16,125707.54,1980.38,7743.56,18.07,150959.49,2599.48,7810.73,28.21,166535.05,3402.93,7215.37,20.42,161040.36,2911.74,7206.97,4.92,155260.20,1906.55,7185.25,2.04,196333.17,1746.16,5898.0,14.41,175190.0,1839.0,4651.0,19.0,158705.0,1959.0,4064.0,19.0,5218.0,584.0


In [274]:
data_188.to_excel("data_188.xlsx", sheet_name="Hewan Ternak", index=False)

Persentase Rumah Tangga yang Memiliki Akses Terhadap Sumber Air Minum Layak Menurut Kabupaten/Kota di Provinsi Bali(422)

In [275]:
data_422=getDataByVarId(domain='5100',varId='422')
data_422

,verval_val,vervar_label,var_val,turvar_val,turvar_label,tahun_val,tahun_label,turtahun_val,turtahun_label,id,content_val,content_label
0,1,Kab. Jembrana,422,0,Tidak ada,124,2024,0,Tahun,142201240,142201240,95.14
1,2,Kab. Tabanan,422,0,Tidak ada,124,2024,0,Tahun,242201240,242201240,98.38
2,3,Kab. Badung,422,0,Tidak ada,124,2024,0,Tahun,342201240,342201240,100.00
3,4,Kab. Gianyar,422,0,Tidak ada,124,2024,0,Tahun,442201240,442201240,99.55
4,5,Kab. Klungkung,422,0,Tidak ada,124,2024,0,Tahun,542201240,542201240,99.02
...,...,...,...,...,...,...,...,...,...,...,...,...
5,6,Kab. Bangli,422,0,Tidak ada,116,2016,0,Tahun,642201160,642201160,91.99
6,7,Kab. Karangasem,422,0,Tidak ada,116,2016,0,Tahun,742201160,742201160,89.92
7,8,Kab. Buleleng,422,0,Tidak ada,116,2016,0,Tahun,842201160,842201160,76.73
8,9,Kota Denpasar,422,0,Tidak ada,116,2016,0,Tahun,942201160,942201160,98.57


In [276]:
# --- proses cleaning ---
data_422 = data_422[['verval_val','vervar_label','tahun_label','content_label']].copy()
data_422["vervar_label"] = data_422["vervar_label"].str.replace(r"<.*?>", "", regex=True)

# --- rename kolom ---
data_tb = data_422.rename(columns={
    'vervar_label'   : 'Kabupaten/Kota',
    'tahun_label'    : 'Tahun',
    'content_label'  : 'Nilai'
})

# --- ambil urutan otomatis dari API ---
api_order = data_422['vervar_label'].drop_duplicates().tolist()
data_tb['Kabupaten/Kota'] = pd.Categorical(data_tb['Kabupaten/Kota'], categories=api_order, ordered=True)

# --- pivoting tabel ---
data_tb = data_tb.pivot_table(
    index=['Kabupaten/Kota'],
    columns=['Tahun'],
    values='Nilai'
)

# --- gabungkan multi index jadi satu kolom ---
data_tb.columns = ['_'.join([str(c) for c in col if c]).strip() if isinstance(col, tuple) else str(col) 
                   for col in data_tb.columns.values]

# --- reset index (urut sesuai API otomatis) ---
data_tb = data_tb.reset_index()

data_tb


C:\Users\user\AppData\Local\Temp\ipykernel_13044\2425402639.py:17: FutureWarning: The default value of observed=False is deprecated and will change to observed=True in a future version of pandas. Specify observed=False to silence this warning and retain the current behavior
  data_tb = data_tb.pivot_table(


,Kabupaten/Kota,2016,2017,2018,2019,2020,2021,2022,2023,2024
0,Kab. Jembrana,74.79,91.66,75.60,90.58,93.82,92.11,97.23,96.02,95.14
1,Kab. Tabanan,95.24,99.50,95.94,92.73,96.88,98.15,94.60,98.41,98.38
2,Kab. Badung,96.63,99.84,98.20,100.00,99.31,100.00,98.79,99.52,100.00
3,Kab. Gianyar,95.03,99.10,93.94,99.90,100.00,98.71,98.98,99.88,99.55
4,Kab. Klungkung,95.95,96.74,98.85,99.79,98.35,99.96,98.25,98.64,99.02
5,Kab. Bangli,91.99,83.98,98.83,95.30,97.24,91.16,93.12,96.32,96.07
6,Kab. Karangasem,89.92,90.01,89.49,99.91,93.72,98.45,100.00,100.00,99.50
7,Kab. Buleleng,76.73,92.26,88.42,90.21,94.59,93.38,98.99,95.49,94.36
8,Kota Denpasar,98.57,99.81,98.85,100.00,99.25,100.00,100.00,99.84,99.96
9,Provinsi Bali,91.17,96.09,93.76,96.84,97.36,97.56,98.42,98.31,98.32


In [277]:
data_422.to_excel("data_422.xlsx", sheet_name="Persentase_RT_Air_Bersih", index=False)

Persentase Rumah Tangga yang Memiliki Akses Terhadap Sanitasi Layak Menurut Kabupaten/Kota di Provinsi Bali (421)

In [278]:
data_421=getDataByVarId(domain='5100',varId='421')
data_421

,verval_val,vervar_label,var_val,turvar_val,turvar_label,tahun_val,tahun_label,turtahun_val,turtahun_label,id,content_val,content_label
0,1,Kab. Jembrana,421,0,Tidak ada,124,2024,0,Tahun,142101240,142101240,96.13
1,2,Kab. Tabanan,421,0,Tidak ada,124,2024,0,Tahun,242101240,242101240,96.25
2,3,Kab. Badung,421,0,Tidak ada,124,2024,0,Tahun,342101240,342101240,99.30
3,4,Kab. Gianyar,421,0,Tidak ada,124,2024,0,Tahun,442101240,442101240,99.52
4,5,Kab. Klungkung,421,0,Tidak ada,124,2024,0,Tahun,542101240,542101240,99.23
...,...,...,...,...,...,...,...,...,...,...,...,...
5,6,Kab. Bangli,421,0,Tidak ada,115,2015,0,Tahun,642101150,642101150,59.44
6,7,Kab. Karangasem,421,0,Tidak ada,115,2015,0,Tahun,742101150,742101150,57.44
7,8,Kab. Buleleng,421,0,Tidak ada,115,2015,0,Tahun,842101150,842101150,73.87
8,9,Kota Denpasar,421,0,Tidak ada,115,2015,0,Tahun,942101150,942101150,90.52


In [279]:

data_421 = data_421[['verval_val','vervar_label','tahun_label','content_label']].copy()
data_421["vervar_label"] = data_421["vervar_label"].str.replace(r"<.*?>", "", regex=True)


data_tb = data_421.rename(columns={
    'vervar_label'   : 'Kabupaten/Kota',
    'tahun_label'    : 'Tahun',
    'content_label'  : 'Nilai'
})

api_order = data_421['vervar_label'].drop_duplicates().tolist()
data_tb['Kabupaten/Kota'] = pd.Categorical(data_tb['Kabupaten/Kota'], categories=api_order, ordered=True)

data_tb = data_tb.pivot_table(
    index=['Kabupaten/Kota'],
    columns=['Tahun'],
    values='Nilai'
)

data_tb.columns = ['_'.join([str(c) for c in col if c]).strip() if isinstance(col, tuple) else str(col) 
                   for col in data_tb.columns.values]

data_tb = data_tb.reset_index()

data_tb


C:\Users\user\AppData\Local\Temp\ipykernel_13044\1830601075.py:14: FutureWarning: The default value of observed=False is deprecated and will change to observed=True in a future version of pandas. Specify observed=False to silence this warning and retain the current behavior
  data_tb = data_tb.pivot_table(


,Kabupaten/Kota,2015,2016,2017,2018,2019,2020,2021,2022,2023,2024
0,Kab. Jembrana,81.71,85.51,91.05,91.93,94.18,93.03,96.24,95.20,96.05,96.13
1,Kab. Tabanan,92.77,89.80,91.13,91.63,94.92,94.70,98.34,93.34,95.56,96.25
2,Kab. Badung,90.76,96.07,93.06,97.14,100.00,98.88,99.59,97.50,98.98,99.30
3,Kab. Gianyar,89.77,93.80,96.09,95.66,97.71,97.98,93.48,99.95,99.00,99.52
4,Kab. Klungkung,80.34,85.23,91.19,89.96,93.36,94.44,95.91,96.04,97.70,99.23
5,Kab. Bangli,59.44,73.65,74.12,79.93,89.18,90.53,87.01,92.41,91.58,92.49
6,Kab. Karangasem,57.44,74.24,70.30,79.97,81.12,85.44,87.36,90.24,90.55,91.31
7,Kab. Buleleng,73.87,78.44,81.15,80.17,89.56,91.71,94.15,92.41,90.99,93.79
8,Kota Denpasar,90.52,90.35,93.73,94.38,99.68,98.74,99.97,100.00,99.92,99.01
9,Provinsi Bali,82.03,86.59,88.07,89.92,94.59,95.01,95.95,95.94,95.70,96.83


In [280]:
data_421.to_excel("data_421.xlsx", sheet_name= "Akses_Sanitasi", index=False)

Persentase Penduduk 5 Tahun ke Atas yang Merokok Tembakau selama Sebulan Terakhir Menurut Kabupaten/Kota dan Kebiasaan Merokok (391)

In [281]:
data_391=getDataByVarId(domain='5100',varId='391')
data_391

,verval_val,vervar_label,var_val,turvar_val,turvar_label,tahun_val,tahun_label,turtahun_val,turtahun_label,id,content_val,content_label
0,1,Kab. Jembrana,391,185,"Ya, Setiap Hari",124,2024,0,Tahun,13911851240,13911851240,20.31
1,1,Kab. Jembrana,391,186,"Ya, Tidak Setiap Hari",124,2024,0,Tahun,13911861240,13911861240,0.77
2,1,Kab. Jembrana,391,187,Tidak,124,2024,0,Tahun,13911871240,13911871240,28.92
4,2,Kab. Tabanan,391,185,"Ya, Setiap Hari",124,2024,0,Tahun,23911851240,23911851240,18.51
5,2,Kab. Tabanan,391,186,"Ya, Tidak Setiap Hari",124,2024,0,Tahun,23911861240,23911861240,1.15
...,...,...,...,...,...,...,...,...,...,...,...,...
35,9,Kota Denpasar,391,188,Tidak Tahu,119,2019,0,Tahun,93911881190,93911881190,0.20
36,10,<b>Provinsi Bali</b>,391,185,"Ya, Setiap Hari",119,2019,0,Tahun,103911851190,103911851190,15.97
37,10,<b>Provinsi Bali</b>,391,186,"Ya, Tidak Setiap Hari",119,2019,0,Tahun,103911861190,103911861190,1.46
38,10,<b>Provinsi Bali</b>,391,187,Tidak,119,2019,0,Tahun,103911871190,103911871190,82.44


In [284]:

data_391 = data_391[['verval_val','vervar_label','turvar_label','tahun_label','content_label']].copy()
data_391["vervar_label"] = data_391["vervar_label"].str.replace(r"<.*?>", "", regex=True)


data_tb = data_391.rename(columns={
    'vervar_label'   : 'Kabupaten/Kota',
    'tahun_label'    : 'Tahun',
    'turvar_label'   : 'Merokok',
    'content_label'  : 'Nilai'
})

api_order = data_391['vervar_label'].drop_duplicates().tolist()
data_tb['Kabupaten/Kota'] = pd.Categorical(data_tb['Kabupaten/Kota'], categories=api_order, ordered=True)

data_tb = data_tb.pivot_table(
    index=['Kabupaten/Kota'],
    columns=['Tahun', 'Merokok'],
    values='Nilai'
)

data_tb.columns = ['_'.join([str(c) for c in col if c]).strip() if isinstance(col, tuple) else str(col) 
                   for col in data_tb.columns.values]

data_tb = data_tb.reset_index()

data_tb


C:\Users\user\AppData\Local\Temp\ipykernel_13044\794400139.py:15: FutureWarning: The default value of observed=False is deprecated and will change to observed=True in a future version of pandas. Specify observed=False to silence this warning and retain the current behavior
  data_tb = data_tb.pivot_table(


,Kabupaten/Kota,2019_Tidak,2019_Tidak Tahu,"2019_Ya, Setiap Hari","2019_Ya, Tidak Setiap Hari",2020_Tidak,2020_Tidak Tahu,"2020_Ya, Setiap Hari","2020_Ya, Tidak Setiap Hari",2021_Tidak,2021_Tidak Tahu,"2021_Ya, Setiap Hari","2021_Ya, Tidak Setiap Hari",2022_Tidak,2022_Tidak Tahu,"2022_Ya, Setiap Hari","2022_Ya, Tidak Setiap Hari",2023_Tidak,2023_Tidak Tahu,"2023_Ya, Setiap Hari","2023_Ya, Tidak Setiap Hari",2024_Tidak,2024_Tidak Tahu,"2024_Ya, Setiap Hari","2024_Ya, Tidak Setiap Hari"
0,Kab. Jembrana,77.43,0.00,20.75,1.82,77.75,0.00,20.98,1.27,78.43,0.00,20.46,1.10,79.77,0.00,19.16,1.08,78.77,0.00,20.53,0.70,28.92,NaN,20.31,0.77
1,Kab. Tabanan,81.65,0.00,17.22,1.13,80.45,0.06,18.64,0.86,81.95,0.00,16.63,1.43,82.29,0.00,15.61,2.10,81.70,0.30,16.41,1.58,80.33,NaN,18.51,1.15
2,Kab. Badung,81.67,0.04,16.41,1.89,83.12,0.70,14.78,1.41,84.97,0.04,13.21,1.78,85.47,0.99,12.15,1.39,86.06,0.30,12.80,0.83,85.19,NaN,14.17,0.57
3,Kab. Gianyar,87.21,0.00,11.56,1.22,86.14,0.05,12.80,1.01,86.46,0.00,12.45,1.09,87.13,0.00,11.96,0.91,86.74,0.10,12.18,0.98,86.96,NaN,12.17,0.86
4,Kab. Klungkung,87.42,0.06,11.09,1.44,85.87,0.53,12.36,1.24,85.39,0.42,12.88,1.31,82.97,3.34,12.21,1.48,88.24,0.94,9.23,1.59,87.10,NaN,11.85,0.82
5,Kab. Bangli,81.98,0.53,15.70,1.78,83.29,0.08,15.07,1.56,85.66,0.00,13.01,1.32,83.54,1.61,13.72,1.14,81.09,2.28,15.24,1.39,85.20,NaN,13.91,0.80
6,Kab. Karangasem,84.63,0.51,13.53,1.33,83.83,0.06,14.14,1.97,85.99,0.00,12.88,1.13,83.52,0.00,15.19,1.29,84.09,0.02,13.89,2.00,84.58,NaN,14.27,1.14
7,Kab. Buleleng,80.81,0.00,17.43,1.77,81.37,0.00,17.42,1.20,81.03,0.00,18.01,0.96,82.35,0.01,15.80,1.84,79.11,0.05,19.51,1.33,79.10,NaN,20.00,0.76
8,Kota Denpasar,81.56,0.20,17.11,1.13,83.00,0.15,15.78,1.06,82.63,0.17,16.07,1.13,86.78,0.00,10.95,2.28,85.67,0.02,13.02,1.29,85.12,NaN,13.98,0.91
9,Provinsi Bali,82.44,0.13,15.97,1.46,82.76,0.19,15.81,1.25,83.46,0.06,15.23,1.25,84.43,0.38,13.55,1.63,83.78,0.26,14.70,1.26,83.63,0.05,15.47,0.85


In [289]:
data_391.to_excel("data_391.xlsx", sheet_name= "Jawaban Merokok", index=False)

Jumlah Bayi Lahir, Bayi Berat Badan Lahir Rendah (BBLR), dan Bergizi Kurang Menurut Kabupaten/Kota di Bali (390)

In [ ]:
data_390=getDataByVarId(domain='5100',varId='390')
data_390

,verval_val,vervar_label,var_val,turvar_val,turvar_label,tahun_val,tahun_label,turtahun_val,turtahun_label,id,content_val,content_label
0,1,Kab. Jembrana,390,182,Bayi Lahir,123,2023,0,Tahun,13901821230,13901821230,3963
1,1,Kab. Jembrana,390,183,Berat Bayi Lahir Rendah (BBLR),123,2023,0,Tahun,13901831230,13901831230,170
2,1,Kab. Jembrana,390,184,Gizi Kurang,123,2023,0,Tahun,13901841230,13901841230,197
3,2,Kab. Tabanan,390,182,Bayi Lahir,123,2023,0,Tahun,23901821230,23901821230,5130
4,2,Kab. Tabanan,390,183,Berat Bayi Lahir Rendah (BBLR),123,2023,0,Tahun,23901831230,23901831230,153
...,...,...,...,...,...,...,...,...,...,...,...,...
25,9,Kota Denpasar,390,183,Berat Bayi Lahir Rendah (BBLR),118,2018,0,Tahun,93901831180,93901831180,189
26,9,Kota Denpasar,390,184,Gizi Kurang,118,2018,0,Tahun,93901841180,93901841180,611
27,10,<b>Provinsi Bali</b>,390,182,Bayi Lahir,118,2018,0,Tahun,103901821180,103901821180,65722
28,10,<b>Provinsi Bali</b>,390,183,Berat Bayi Lahir Rendah (BBLR),118,2018,0,Tahun,103901831180,103901831180,1823


In [291]:

data_390 = data_390[['verval_val','vervar_label','turvar_label','tahun_label','content_label']].copy()
data_390["vervar_label"] = data_390["vervar_label"].str.replace(r"<.*?>", "", regex=True)


data_tb = data_390.rename(columns={
    'vervar_label'   : 'Kabupaten/Kota',
    'tahun_label'    : 'Tahun',
    'turvar_label'   : 'Jumlah Bayi',
    'content_label'  : 'Nilai'
})

api_order = data_390['vervar_label'].drop_duplicates().tolist()
data_tb['Kabupaten/Kota'] = pd.Categorical(data_tb['Kabupaten/Kota'], categories=api_order, ordered=True)

data_tb = data_tb.pivot_table(
    index=['Kabupaten/Kota'],
    columns=['Tahun', 'Jumlah Bayi'],
    values='Nilai'
)

data_tb.columns = ['_'.join([str(c) for c in col if c]).strip() if isinstance(col, tuple) else str(col) 
                   for col in data_tb.columns.values]

data_tb = data_tb.reset_index()

data_tb


C:\Users\user\AppData\Local\Temp\ipykernel_13044\3109333819.py:15: FutureWarning: The default value of observed=False is deprecated and will change to observed=True in a future version of pandas. Specify observed=False to silence this warning and retain the current behavior
  data_tb = data_tb.pivot_table(


,Kabupaten/Kota,2018_Bayi Lahir,2018_Berat Bayi Lahir Rendah (BBLR),2018_Gizi Kurang,2019_Bayi Lahir,2019_Berat Bayi Lahir Rendah (BBLR),2019_Gizi Kurang,2020_Bayi Lahir,2020_Berat Bayi Lahir Rendah (BBLR),2020_Gizi Kurang,2021_Bayi Lahir,2021_Berat Bayi Lahir Rendah (BBLR),2021_Gizi Kurang,2022_Bayi Lahir,2022_Berat Bayi Lahir Rendah (BBLR),2022_Gizi Kurang,2023_Bayi Lahir,2023_Berat Bayi Lahir Rendah (BBLR),2023_Gizi Kurang
0,Kab. Jembrana,4505.0,180.0,218.0,3954.0,223.0,221.0,4228.0,179.0,122.0,4322.0,165.0,59.0,3671.0,160.0,224.0,3963.0,170.0,197.0
1,Kab. Tabanan,5256.0,163.0,603.0,5215.0,213.0,179.0,5176.0,220.0,433.0,5254.0,218.0,257.0,4723.0,195.0,439.0,5130.0,153.0,284.0
2,Kab. Badung,10380.0,214.0,110.0,10653.0,179.0,66.0,10485.0,185.0,312.0,9251.0,267.0,88.0,8233.0,260.0,109.0,11379.0,337.0,153.0
3,Kab. Gianyar,7177.0,246.0,637.0,6568.0,267.0,366.0,6480.0,233.0,503.0,6543.0,291.0,525.0,5448.0,252.0,513.0,7202.0,276.0,410.0
4,Kab. Klungkung,2516.0,126.0,36.0,2665.0,125.0,303.0,2742.0,167.0,250.0,2732.0,160.0,236.0,2300.0,100.0,227.0,2446.0,161.0,187.0
5,Kab. Bangli,3258.0,121.0,40.0,3262.0,123.0,117.0,3404.0,113.0,179.0,3393.0,160.0,131.0,2772.0,108.0,169.0,3324.0,172.0,189.0
6,Kab. Karangasem,6418.0,277.0,112.0,6344.0,294.0,137.0,6949.0,279.0,569.0,6753.0,300.0,314.0,6086.0,206.0,583.0,6283.0,284.0,404.0
7,Kab. Buleleng,10684.0,307.0,758.0,10466.0,350.0,536.0,7828.0,199.0,679.0,10997.0,308.0,626.0,8130.0,367.0,662.0,10008.0,327.0,554.0
8,Kota Denpasar,15528.0,189.0,611.0,16538.0,19.0,28.0,16453.0,230.0,108.0,16619.0,233.0,68.0,17089.0,239.0,99.0,16858.0,266.0,111.0
9,Provinsi Bali,65722.0,1823.0,3125.0,65665.0,1793.0,1953.0,63745.0,1805.0,3155.0,65864.0,2102.0,2304.0,58452.0,1887.0,3025.0,66593.0,2146.0,2489.0


In [292]:
data_390.to_excel("data_390.xlsx", sheet_name= "Bayi Lahir", index=False)

Persentase Penduduk yang Mempunyai Keluhan Kesehatan dan Berobat Jalan Selama Sebulan Terakhir Menurut Kabupaten/Kota di Bali (389)

In [293]:
data_389=getDataByVarId(domain='5100',varId='389')
data_389

,verval_val,vervar_label,var_val,turvar_val,turvar_label,tahun_val,tahun_label,turtahun_val,turtahun_label,id,content_val,content_label
0,1,Kab. Jembrana,389,0,Tidak ada,124,2024,0,Tahun,138901240,138901240,57.80
1,2,Kab. Tabanan,389,0,Tidak ada,124,2024,0,Tahun,238901240,238901240,57.45
2,3,Kab. Badung,389,0,Tidak ada,124,2024,0,Tahun,338901240,338901240,60.84
3,4,Kab. Gianyar,389,0,Tidak ada,124,2024,0,Tahun,438901240,438901240,78.16
4,5,Kab. Klungkung,389,0,Tidak ada,124,2024,0,Tahun,538901240,538901240,51.99
...,...,...,...,...,...,...,...,...,...,...,...,...
5,6,Kab. Bangli,389,0,Tidak ada,115,2015,0,Tahun,638901150,638901150,72.30
6,7,Kab. Karangasem,389,0,Tidak ada,115,2015,0,Tahun,738901150,738901150,70.28
7,8,Kab. Buleleng,389,0,Tidak ada,115,2015,0,Tahun,838901150,838901150,63.35
8,9,Kota Denpasar,389,0,Tidak ada,115,2015,0,Tahun,938901150,938901150,53.62


In [294]:

data_389 = data_389[['verval_val','vervar_label','tahun_label','content_label']].copy()
data_389["vervar_label"] = data_389["vervar_label"].str.replace(r"<.*?>", "", regex=True)


data_tb = data_389.rename(columns={
    'vervar_label'   : 'Kabupaten/Kota',
    'tahun_label'    : 'Tahun',
    'content_label'  : 'Nilai'
})

api_order = data_389['vervar_label'].drop_duplicates().tolist()
data_tb['Kabupaten/Kota'] = pd.Categorical(data_tb['Kabupaten/Kota'], categories=api_order, ordered=True)

data_tb = data_tb.pivot_table(
    index=['Kabupaten/Kota'],
    columns=['Tahun'],
    values='Nilai'
)

data_tb.columns = ['_'.join([str(c) for c in col if c]).strip() if isinstance(col, tuple) else str(col) 
                   for col in data_tb.columns.values]

data_tb = data_tb.reset_index()

data_tb


C:\Users\user\AppData\Local\Temp\ipykernel_13044\1339428518.py:14: FutureWarning: The default value of observed=False is deprecated and will change to observed=True in a future version of pandas. Specify observed=False to silence this warning and retain the current behavior
  data_tb = data_tb.pivot_table(


,Kabupaten/Kota,2015,2016,2017,2018,2019,2020,2021,2022,2023,2024
0,Kab. Jembrana,66.78,70.56,50.40,63.98,73.11,54.80,48.94,49.74,48.77,57.80
1,Kab. Tabanan,75.09,74.29,65.44,64.31,73.52,64.78,52.48,63.40,53.16,57.45
2,Kab. Badung,66.04,54.06,53.81,60.88,51.22,57.82,52.87,53.87,45.51,60.84
3,Kab. Gianyar,74.27,68.24,70.33,65.57,73.60,70.24,57.74,69.52,69.02,78.16
4,Kab. Klungkung,73.47,70.75,51.00,67.02,79.77,68.33,73.00,83.42,67.15,51.99
5,Kab. Bangli,72.30,75.04,68.67,69.37,68.82,59.59,47.87,74.79,53.83,61.11
6,Kab. Karangasem,70.28,65.07,64.77,70.11,76.02,63.48,53.63,56.55,53.39,66.61
7,Kab. Buleleng,63.35,71.43,54.77,63.33,70.77,65.97,62.73,53.17,70.84,77.49
8,Kota Denpasar,53.62,48.91,51.15,51.53,56.63,48.52,48.99,50.49,32.95,43.15
9,Provinsi Bali,66.30,65.00,58.85,62.77,67.69,61.32,53.31,58.80,53.08,62.07


In [295]:
data_389.to_excel("data_389.xlsx", sheet_name= "Keluhan Kesehatan", index=False)

Persentase Penduduk yang Mempunyai Keluhan Kesehatan Selama Sebulan Terakhir Menurut Kabupaten/Kota di Bali (388)

In [296]:
data_388=getDataByVarId(domain='5100',varId='388')
data_388

,verval_val,vervar_label,var_val,turvar_val,turvar_label,tahun_val,tahun_label,turtahun_val,turtahun_label,id,content_val,content_label
0,1,Kab. Jembrana,388,0,Tidak ada,123,2023,0,Tahun,138801230,138801230,17.41
1,2,Kab. Tabanan,388,0,Tidak ada,123,2023,0,Tahun,238801230,238801230,21.44
2,3,Kab. Badung,388,0,Tidak ada,123,2023,0,Tahun,338801230,338801230,21.41
3,4,Kab. Gianyar,388,0,Tidak ada,123,2023,0,Tahun,438801230,438801230,19.71
4,5,Kab. Klungkung,388,0,Tidak ada,123,2023,0,Tahun,538801230,538801230,12.76
...,...,...,...,...,...,...,...,...,...,...,...,...
5,6,Kab. Bangli,388,0,Tidak ada,114,2014,0,Tahun,638801140,638801140,39.62
6,7,Kab. Karangasem,388,0,Tidak ada,114,2014,0,Tahun,738801140,738801140,40.19
7,8,Kab. Buleleng,388,0,Tidak ada,114,2014,0,Tahun,838801140,838801140,57.50
8,9,Kota Denpasar,388,0,Tidak ada,114,2014,0,Tahun,938801140,938801140,25.47


In [297]:

data_388 = data_388[['verval_val','vervar_label','tahun_label','content_label']].copy()
data_388["vervar_label"] = data_388["vervar_label"].str.replace(r"<.*?>", "", regex=True)


data_tb = data_388.rename(columns={
    'vervar_label'   : 'Kabupaten/Kota',
    'tahun_label'    : 'Tahun',
    'content_label'  : 'Nilai'
})

api_order = data_388['vervar_label'].drop_duplicates().tolist()
data_tb['Kabupaten/Kota'] = pd.Categorical(data_tb['Kabupaten/Kota'], categories=api_order, ordered=True)

data_tb = data_tb.pivot_table(
    index=['Kabupaten/Kota'],
    columns=['Tahun'],
    values='Nilai'
)

data_tb.columns = ['_'.join([str(c) for c in col if c]).strip() if isinstance(col, tuple) else str(col) 
                   for col in data_tb.columns.values]

data_tb = data_tb.reset_index()

data_tb


C:\Users\user\AppData\Local\Temp\ipykernel_13044\2028408397.py:14: FutureWarning: The default value of observed=False is deprecated and will change to observed=True in a future version of pandas. Specify observed=False to silence this warning and retain the current behavior
  data_tb = data_tb.pivot_table(


,Kabupaten/Kota,2014,2015,2016,2017,2018,2019,2020,2021,2022,2023
0,Kab. Jembrana,31.96,38.03,29.86,30.92,26.09,23.29,24.07,12.85,20.14,17.41
1,Kab. Tabanan,39.36,37.60,34.03,31.39,30.54,28.95,24.64,16.35,16.98,21.44
2,Kab. Badung,23.42,22.41,22.82,21.03,26.42,22.39,18.55,11.95,21.30,21.41
3,Kab. Gianyar,36.59,34.05,34.12,31.51,38.02,35.29,34.08,16.98,20.95,19.71
4,Kab. Klungkung,41.01,33.46,24.80,22.91,24.87,19.71,18.60,18.83,21.24,12.76
5,Kab. Bangli,39.62,42.86,39.29,34.54,40.23,41.57,34.81,23.34,26.65,30.90
6,Kab. Karangasem,40.19,34.32,31.23,34.24,35.56,35.00,30.42,23.97,36.20,36.46
7,Kab. Buleleng,57.50,50.93,41.89,35.41,33.50,36.07,31.22,21.39,21.96,26.56
8,Kota Denpasar,25.47,30.49,25.85,23.85,24.08,20.94,19.54,44.10,12.09,18.65
9,Provinsi Bali,36.24,35.29,31.18,28.90,30.33,28.53,25.48,23.62,20.45,22.64


In [298]:
data_388.to_excel("data_388.xlsx", sheet_name= "Keluhan Kesehatan", index=False)

Angka Partisipasi Sekolah (APS) Provinsi Bali Menurut Jenjang Pendidikan dan Kabupaten/Kota (282)

In [299]:
data_282=getDataByVarId(domain='5100',varId='282')
data_282

,verval_val,vervar_label,var_val,turvar_val,turvar_label,tahun_val,tahun_label,turtahun_val,turtahun_label,id,content_val,content_label
0,1,Kab. Jembrana,282,172,7-12 Tahun,124,2024,0,Tahun,12821721240,12821721240,99.62
1,1,Kab. Jembrana,282,173,13-15 Tahun,124,2024,0,Tahun,12821731240,12821731240,99.65
2,1,Kab. Jembrana,282,174,16-18 Tahun,124,2024,0,Tahun,12821741240,12821741240,81.70
3,1,Kab. Jembrana,282,175,19-24 Tahun,124,2024,0,Tahun,12821751240,12821751240,20.95
4,2,Kab. Tabanan,282,172,7-12 Tahun,124,2024,0,Tahun,22821721240,22821721240,99.82
...,...,...,...,...,...,...,...,...,...,...,...,...
35,9,Kota Denpasar,282,175,19-24 Tahun,115,2015,0,Tahun,92821751150,92821751150,28.65
36,10,<b>Provinsi Bali</b>,282,172,7-12 Tahun,115,2015,0,Tahun,102821721150,102821721150,99.41
37,10,<b>Provinsi Bali</b>,282,173,13-15 Tahun,115,2015,0,Tahun,102821731150,102821731150,97.41
38,10,<b>Provinsi Bali</b>,282,174,16-18 Tahun,115,2015,0,Tahun,102821741150,102821741150,81.69


In [310]:

data_282 = data_282[['verval_val','vervar_label','turvar_label','tahun_label','content_label']].copy()
data_282["vervar_label"] = data_282["vervar_label"].str.replace(r"<.*?>", "", regex=True)


data_tb = data_282.rename(columns={
    'vervar_label'   : 'Kabupaten/Kota',
    'tahun_label'    : 'Tahun',
    'turvar_label'   : 'Umur',
    'content_label'  : 'Nilai'
})

api_order = data_282['vervar_label'].drop_duplicates().tolist()
data_tb['Kabupaten/Kota'] = pd.Categorical(data_tb['Kabupaten/Kota'], categories=api_order, ordered=True)

data_tb = data_tb.pivot_table(
    index=['Kabupaten/Kota'],
    columns=['Tahun', 'Umur'],
    values='Nilai'
)

data_tb.columns = ['_'.join([str(c) for c in col if c]).strip() if isinstance(col, tuple) else str(col) 
                   for col in data_tb.columns.values]

data_tb = data_tb.reset_index()

data_tb


C:\Users\user\AppData\Local\Temp\ipykernel_13044\2934597020.py:15: FutureWarning: The default value of observed=False is deprecated and will change to observed=True in a future version of pandas. Specify observed=False to silence this warning and retain the current behavior
  data_tb = data_tb.pivot_table(


,Kabupaten/Kota,2015_13-15 Tahun,2015_16-18 Tahun,2015_19-24 Tahun,2015_7-12 Tahun,2016_13-15 Tahun,2016_16-18 Tahun,2016_19-24 Tahun,2016_7-12 Tahun,2017_13-15 Tahun,2017_16-18 Tahun,2017_19-24 Tahun,2017_7-12 Tahun,2018_13-15 Tahun,2018_16-18 Tahun,2018_19-24 Tahun,2018_7-12 Tahun,2019_13-15 Tahun,2019_16-18 Tahun,2019_7-12 Tahun,2020_13-15 Tahun,2020_16-18 Tahun,2020_19-24 Tahun,2020_7-12 Tahun,2021_13-15 Tahun,2021_16-18 Tahun,2021_19-24 Tahun,2021_7-12 Tahun,2022_13-15 Tahun,2022_16-18 Tahun,2022_19-24 Tahun,2022_7-12 Tahun,2023_13-15 Tahun,2023_16-18 Tahun,2023_19-24 Tahun,2023_7-12 Tahun,2024_13-15 Tahun,2024_16-18 Tahun,2024_19-24 Tahun,2024_7-12 Tahun
0,Kab. Jembrana,97.46,80.53,7.64,100.00,100.00,81.04,17.86,99.66,98.97,77.34,14.33,99.85,99.24,84.37,14.79,99.41,99.69,84.43,99.44,98.69,84.84,15.34,99.68,99.05,84.48,15.71,99.96,98.37,84.24,23.36,99.65,98.60,84.30,23.56,99.82,99.65,81.70,20.95,99.62
1,Kab. Tabanan,98.25,75.00,24.44,99.26,96.85,88.41,23.75,99.50,98.27,86.97,21.36,98.98,98.55,84.45,15.31,98.96,99.19,85.69,99.22,98.65,87.49,16.29,99.19,98.49,86.33,16.83,99.28,98.70,86.68,19.84,99.23,97.78,86.18,26.37,99.76,99.21,80.98,25.78,99.82
2,Kab. Badung,100.00,89.68,29.46,99.22,99.09,93.77,31.57,99.67,100.00,90.79,34.03,100.00,100.00,92.07,32.88,99.71,98.81,92.09,100.00,99.19,91.27,31.65,99.86,99.20,90.79,31.94,99.97,98.75,90.59,36.47,99.97,99.49,90.30,42.88,99.99,99.81,94.52,43.76,99.99
3,Kab. Gianyar,97.40,86.79,29.30,99.60,100.00,82.57,28.29,100.00,99.51,80.95,27.17,100.00,99.47,90.17,35.70,99.78,99.37,90.12,100.00,99.39,90.04,35.54,99.31,99.32,89.89,35.42,99.72,98.24,89.89,32.38,98.81,98.84,89.81,43.21,99.75,99.79,92.35,43.26,99.49
4,Kab. Klungkung,97.53,94.28,17.13,99.66,97.28,78.96,18.26,98.45,97.56,86.51,19.64,98.68,97.58,85.69,14.93,99.21,97.60,86.07,99.68,97.94,87.12,19.80,99.75,97.96,86.92,18.98,99.72,96.81,86.79,25.28,99.28,97.06,87.27,24.55,99.41,98.78,84.19,26.87,99.21
5,Kab. Bangli,98.53,74.30,13.11,98.30,97.47,81.67,18.95,99.61,96.58,76.36,16.90,98.88,95.57,74.64,14.62,99.02,95.89,75.58,99.26,96.19,76.34,14.60,99.72,96.21,74.76,15.42,99.54,96.15,74.47,20.71,99.40,98.55,74.01,19.44,99.49,98.22,77.73,18.27,99.98
6,Kab. Karangasem,92.61,72.88,8.86,99.75,96.24,66.74,10.61,99.71,96.17,71.61,13.82,99.47,96.48,80.11,8.54,99.35,96.97,81.26,99.32,96.77,80.96,8.99,99.86,96.43,80.54,9.52,99.72,96.92,80.13,14.28,99.94,95.96,78.87,20.57,99.79,99.45,84.21,22.91,99.84
7,Kab. Buleleng,97.25,74.78,21.43,99.48,94.90,83.97,18.19,99.66,94.48,83.93,16.51,99.63,94.43,80.08,25.94,100.00,94.70,80.52,99.67,96.41,81.00,26.21,99.40,96.53,80.58,25.80,99.41,95.86,80.33,29.31,99.83,96.45,81.57,26.52,99.69,96.24,82.46,28.39,99.92
8,Kota Denpasar,97.54,85.94,28.65,99.33,97.85,78.19,31.99,98.23,98.57,80.22,35.68,98.93,99.17,73.10,34.24,99.62,98.42,73.74,100.00,99.23,75.47,39.06,99.51,98.98,80.40,39.53,99.84,98.84,80.80,39.92,99.37,98.50,83.63,41.06,99.12,98.79,82.29,41.25,99.18
9,Provinsi Bali,97.41,81.69,23.75,99.41,97.55,81.98,25.36,99.35,97.72,82.16,26.56,99.44,97.92,82.35,27.24,99.56,97.72,82.60,99.71,98.21,82.96,28.67,99.57,98.22,83.96,28.95,99.70,97.85,83.84,30.18,99.55,97.95,84.73,34.31,99.61,98.79,85.17,35.39,99.65


In [309]:
data_282.to_excel("data_282.xlsx", sheet_name= "APS", index=False)

Angka Partisipasi Murni (APM) Provinsi Bali Menurut Jenjang Pendidikan dan Kabupaten/Kota (281) URUTAN BLM RAPI

In [315]:
data_281=getDataByVarId (domain='5100',varId='281')
data_281

,verval_val,vervar_label,var_val,turvar_val,turvar_label,tahun_val,tahun_label,turtahun_val,turtahun_label,id,content_val,content_label
0,1,Kab. Jembrana,281,75,SD/Sederajat,124,2024,0,Tahun,1281751240,1281751240,96.98
1,1,Kab. Jembrana,281,76,SMP/Sederajat,124,2024,0,Tahun,1281761240,1281761240,85.36
2,1,Kab. Jembrana,281,77,SMA/Sederajat,124,2024,0,Tahun,1281771240,1281771240,77.31
3,1,Kab. Jembrana,281,78,Perguruan Tinggi,124,2024,0,Tahun,1281781240,1281781240,17.77
4,2,Kab. Tabanan,281,75,SD/Sederajat,124,2024,0,Tahun,2281751240,2281751240,99.82
...,...,...,...,...,...,...,...,...,...,...,...,...
35,9,Kota Denpasar,281,78,Perguruan Tinggi,115,2015,0,Tahun,9281781150,9281781150,26.47
36,10,<b>Provinsi Bali</b>,281,75,SD/Sederajat,115,2015,0,Tahun,10281751150,10281751150,95.64
37,10,<b>Provinsi Bali</b>,281,76,SMP/Sederajat,115,2015,0,Tahun,10281761150,10281761150,84.78
38,10,<b>Provinsi Bali</b>,281,77,SMA/Sederajat,115,2015,0,Tahun,10281771150,10281771150,71.53


In [316]:

data_281 = data_281[['verval_val','vervar_label','turvar_label','tahun_label','content_label']].copy()
data_281["vervar_label"] = data_281["vervar_label"].str.replace(r"<.*?>", "", regex=True)


data_tb = data_281.rename(columns={
    'vervar_label'   : 'Kabupaten/Kota',
    'tahun_label'    : 'Tahun',
    'turvar_label'   : 'Jenjang Pendidikan',
    'content_label'  : 'Nilai'
})

api_order = data_281['vervar_label'].drop_duplicates().tolist()
data_tb['Kabupaten/Kota'] = pd.Categorical(data_tb['Kabupaten/Kota'], categories=api_order, ordered=True)

data_tb = data_tb.pivot_table(
    index=['Kabupaten/Kota'],
    columns=['Tahun', 'Jenjang Pendidikan'],
    values='Nilai'
)

data_tb.columns = ['_'.join([str(c) for c in col if c]).strip() if isinstance(col, tuple) else str(col) 
                   for col in data_tb.columns.values]

data_tb = data_tb.reset_index()

data_tb


C:\Users\user\AppData\Local\Temp\ipykernel_13044\2628664472.py:15: FutureWarning: The default value of observed=False is deprecated and will change to observed=True in a future version of pandas. Specify observed=False to silence this warning and retain the current behavior
  data_tb = data_tb.pivot_table(


,Kabupaten/Kota,2015_Perguruan Tinggi,2015_SD/Sederajat,2015_SMA/Sederajat,2015_SMP/Sederajat,2016_Perguruan Tinggi,2016_SD/Sederajat,2016_SMA/Sederajat,2016_SMP/Sederajat,2017_Perguruan Tinggi,2017_SD/Sederajat,2017_SMA/Sederajat,2017_SMP/Sederajat,2018_Perguruan Tinggi,2018_SD/Sederajat,2018_SMA/Sederajat,2018_SMP/Sederajat,2019_SD/Sederajat,2019_SMA/Sederajat,2019_SMP/Sederajat,2020_Perguruan Tinggi,2020_SD/Sederajat,2020_SMA/Sederajat,2020_SMP/Sederajat,2021_Perguruan Tinggi,2021_SD/Sederajat,2021_SMA/Sederajat,2021_SMP/Sederajat,2022_Perguruan Tinggi,2022_SD/Sederajat,2022_SMA/Sederajat,2022_SMP/Sederajat,2023_Perguruan Tinggi,2023_SD/Sederajat,2023_SMA/Sederajat,2023_SMP/Sederajat,2024_Perguruan Tinggi,2024_SD/Sederajat,2024_SMA/Sederajat,2024_SMP/Sederajat
0,Kab. Jembrana,3.92,95.66,77.08,85.74,9.62,99.66,67.12,95.23,12.67,99.85,68.39,93.77,11.02,99.41,71.87,94.49,99.43,72.45,94.16,10.07,99.24,72.81,94.26,11.15,99.37,71.81,94.05,19.12,97.83,72.86,94.11,16.89,97.77,77.73,88.12,17.77,96.98,77.31,85.36
1,Kab. Tabanan,24.44,95.74,65.43,87.66,16.63,97.61,81.54,91.99,19.08,96.26,82.91,92.38,14.41,97.32,80.97,91.98,97.80,80.72,91.91,15.26,97.43,80.74,91.69,15.73,99.07,80.57,91.90,16.72,98.62,80.59,91.51,23.44,99.22,71.46,88.46,24.22,99.82,72.81,89.92
2,Kab. Badung,27.29,95.96,72.18,82.21,27.25,99.67,80.21,82.70,30.65,99.13,80.88,84.77,32.21,99.71,79.22,84.30,99.74,80.14,84.27,31.27,99.75,80.15,84.48,31.56,99.41,82.02,84.71,33.52,98.20,81.94,84.23,39.89,98.03,81.57,86.64,39.72,98.72,83.40,88.57
3,Kab. Gianyar,27.39,95.20,77.41,85.65,22.92,94.49,74.09,96.20,24.28,97.63,74.80,94.66,23.61,98.13,77.15,97.41,98.44,77.15,97.29,24.77,98.07,77.59,97.45,24.42,98.32,77.44,97.70,29.36,97.47,77.26,97.70,41.29,97.40,79.26,90.04,41.16,98.59,82.58,89.52
4,Kab. Klungkung,14.09,96.47,82.29,87.68,15.98,95.33,71.16,93.91,15.88,96.60,71.79,91.62,13.95,99.21,76.58,9.09,99.04,77.49,90.28,14.52,98.90,77.38,90.18,14.59,98.90,77.29,90.79,21.64,98.25,77.12,90.15,18.37,98.18,78.39,74.00,19.30,97.85,74.47,75.63
5,Kab. Bangli,12.45,94.30,68.95,86.56,7.84,88.85,77.86,92.46,9.32,91.81,74.86,91.27,10.98,90.20,72.41,90.13,94.51,72.82,90.60,11.51,94.59,73.77,90.85,12.12,94.23,73.74,91.18,15.65,93.53,73.50,90.83,12.08,96.77,65.76,74.90,12.99,99.18,67.15,76.31
6,Kab. Karangasem,6.21,95.95,61.34,75.02,7.90,95.65,56.39,84.26,7.54,95.86,58.00,82.28,6.16,96.55,59.30,82.87,96.86,60.02,85.20,7.52,96.91,60.06,85.06,8.65,96.85,60.06,84.51,13.00,99.01,59.93,83.99,16.66,98.85,64.38,84.20,18.75,98.89,67.71,87.83
7,Kab. Buleleng,18.61,96.93,65.50,85.67,16.77,91.70,78.11,79.33,14.04,93.26,78.08,81.10,19.63,94.34,79.23,83.35,94.86,78.74,85.06,19.44,95.16,79.81,86.40,18.35,94.45,79.22,87.15,22.87,95.10,78.58,86.87,18.95,95.12,64.34,85.44,19.76,98.83,66.01,87.30
8,Kota Denpasar,26.47,94.44,76.61,87.83,31.06,96.68,63.76,74.92,31.19,95.08,64.24,75.06,33.26,93.20,63.47,77.19,94.50,64.42,76.94,35.38,94.27,64.96,80.04,36.86,96.29,70.44,79.13,38.51,98.20,71.12,79.40,35.17,98.83,82.95,82.25,33.75,98.67,73.66,84.62
9,Provinsi Bali,21.61,95.64,71.53,84.78,21.80,95.75,71.71,84.99,22.78,96.09,72.40,85.32,24.06,96.19,73.00,86.00,96.78,72.93,86.75,24.68,96.84,73.29,87.26,25.28,97.20,74.82,87.11,26.97,97.46,74.73,86.88,29.40,97.77,75.60,84.78,29.96,98.70,74.19,86.30


In [323]:
data_281.to_excel("data_281.xlsx", sheet_name= "APM", index=False)

Angka Partisipasi Kasar (APK) Provinsi Bali Menurut Jenjang Pendidikan dan Kabupaten/Kota (280)

In [318]:
data_280=getDataByVarId(domain='5100',varId='280')
data_280

,verval_val,vervar_label,var_val,turvar_val,turvar_label,tahun_val,tahun_label,turtahun_val,turtahun_label,id,content_val,content_label
0,1,Kab. Jembrana,280,75,SD/Sederajat,124,2024,0,Tahun,1280751240,1280751240,102.57
1,1,Kab. Jembrana,280,76,SMP/Sederajat,124,2024,0,Tahun,1280761240,1280761240,96.05
2,1,Kab. Jembrana,280,77,SMA/Sederajat,124,2024,0,Tahun,1280771240,1280771240,93.67
3,1,Kab. Jembrana,280,78,Perguruan Tinggi,124,2024,0,Tahun,1280781240,1280781240,25.12
4,2,Kab. Tabanan,280,75,SD/Sederajat,124,2024,0,Tahun,2280751240,2280751240,103.36
...,...,...,...,...,...,...,...,...,...,...,...,...
35,9,Kota Denpasar,280,78,Perguruan Tinggi,115,2015,0,Tahun,9280781150,9280781150,31.06
36,10,<b>Provinsi Bali</b>,280,75,SD/Sederajat,115,2015,0,Tahun,10280751150,10280751150,105.00
37,10,<b>Provinsi Bali</b>,280,76,SMP/Sederajat,115,2015,0,Tahun,10280761150,10280761150,96.82
38,10,<b>Provinsi Bali</b>,280,77,SMA/Sederajat,115,2015,0,Tahun,10280771150,10280771150,86.61


In [319]:

data_280 = data_280[['verval_val','vervar_label','turvar_label','tahun_label','content_label']].copy()
data_280["vervar_label"] = data_280["vervar_label"].str.replace(r"<.*?>", "", regex=True)


data_tb = data_280.rename(columns={
    'vervar_label'   : 'Kabupaten/Kota',
    'tahun_label'    : 'Tahun',
    'turvar_label'   : 'Jenjang Pendidikan',
    'content_label'  : 'Nilai'
})

api_order = data_280['vervar_label'].drop_duplicates().tolist()
data_tb['Kabupaten/Kota'] = pd.Categorical(data_tb['Kabupaten/Kota'], categories=api_order, ordered=True)

data_tb = data_tb.pivot_table(
    index=['Kabupaten/Kota'],
    columns=['Tahun', 'Jenjang Pendidikan'],
    values='Nilai'
)

data_tb.columns = ['_'.join([str(c) for c in col if c]).strip() if isinstance(col, tuple) else str(col) 
                   for col in data_tb.columns.values]

data_tb = data_tb.reset_index()

data_tb


C:\Users\user\AppData\Local\Temp\ipykernel_13044\1455397255.py:15: FutureWarning: The default value of observed=False is deprecated and will change to observed=True in a future version of pandas. Specify observed=False to silence this warning and retain the current behavior
  data_tb = data_tb.pivot_table(


,Kabupaten/Kota,2015_Perguruan Tinggi,2015_SD/Sederajat,2015_SMA/Sederajat,2015_SMP/Sederajat,2016_Perguruan Tinggi,2016_SD/Sederajat,2016_SMA/Sederajat,2016_SMP/Sederajat,2017_Perguruan Tinggi,2017_SD/Sederajat,2017_SMA/Sederajat,2017_SMP/Sederajat,2018_Perguruan Tinggi,2018_SD/Sederajat,2018_SMA/Sederajat,2018_SMP/Sederajat,2019_Perguruan Tinggi,2019_SD/Sederajat,2019_SMA/Sederajat,2019_SMP/Sederajat,2020_Perguruan Tinggi,2020_SD/Sederajat,2020_SMA/Sederajat,2020_SMP/Sederajat,2021_Perguruan Tinggi,2021_SD/Sederajat,2021_SMA/Sederajat,2021_SMP/Sederajat,2022_Perguruan Tinggi,2022_SD/Sederajat,2022_SMA/Sederajat,2022_SMP/Sederajat,2023_Perguruan Tinggi,2023_SD/Sederajat,2023_SMA/Sederajat,2023_SMP/Sederajat,2024_Perguruan Tinggi,2024_SD/Sederajat,2024_SMA/Sederajat,2024_SMP/Sederajat
0,Kab. Jembrana,9.64,103.42,91.80,99.68,12.29,111.93,81.94,106.41,18.43,105.28,74.19,97.67,13.65,108.41,77.35,104.19,11.12,104.69,83.97,104.94,14.11,103.15,83.46,105.01,16.17,104.85,85.61,103.73,22.00,104.21,85.67,106.05,22.64,101.48,97.96,97.68,25.12,102.57,93.67,96.05
1,Kab. Tabanan,31.02,108.02,71.07,96.82,20.74,103.54,98.38,99.51,21.96,106.25,91.71,98.88,16.88,102.90,87.29,97.38,18.36,104.61,90.20,96.29,18.36,102.88,91.15,96.05,18.24,102.81,89.26,96.92,18.32,102.38,90.71,99.90,27.01,102.69,85.56,100.40,29.69,103.36,97.98,93.59
2,Kab. Badung,33.81,102.06,98.75,94.24,33.53,109.04,108.13,85.53,40.48,108.66,100.97,86.70,38.98,107.01,92.10,86.36,39.23,107.89,91.41,86.72,41.21,106.00,92.55,89.68,37.76,105.61,95.73,90.69,39.47,105.84,98.12,94.71,48.42,104.28,93.84,99.00,48.70,106.56,95.75,95.01
3,Kab. Gianyar,32.19,104.20,91.51,98.18,27.62,98.91,85.44,110.49,29.59,105.26,84.79,100.43,31.36,104.56,97.92,101.77,28.77,103.36,99.43,107.59,30.47,101.47,96.52,106.84,32.05,102.62,96.97,105.77,35.84,102.60,96.48,101.30,47.57,102.23,88.67,104.89,48.39,103.16,96.20,95.40
4,Kab. Klungkung,20.26,105.96,101.91,94.29,18.80,102.32,75.79,104.68,20.94,105.05,82.91,105.11,21.72,105.02,83.40,93.27,16.86,109.18,92.83,95.53,18.08,107.88,93.32,98.21,17.42,108.94,91.27,98.40,23.67,108.25,91.25,101.24,26.21,108.09,105.78,85.46,23.39,105.50,96.38,87.55
5,Kab. Bangli,12.45,103.34,80.08,98.90,10.86,95.94,98.07,115.42,12.69,96.40,92.58,106.39,12.78,96.20,81.16,107.92,13.27,97.27,82.64,104.13,12.48,100.01,83.49,104.97,13.91,99.44,81.56,108.38,16.19,99.61,83.03,110.31,17.65,100.65,97.18,91.41,18.03,103.70,95.39,82.64
6,Kab. Karangasem,9.41,109.17,75.60,91.60,8.44,107.57,64.06,99.78,9.11,104.04,77.55,100.40,13.38,106.86,71.32,99.32,9.43,105.88,73.68,105.36,11.51,104.37,74.79,103.50,13.29,102.65,74.13,104.99,22.87,103.54,74.34,93.99,24.05,103.10,83.52,95.16,26.76,106.21,86.14,106.03
7,Kab. Buleleng,23.13,106.65,81.40,94.62,18.86,103.30,86.62,98.32,15.71,100.87,95.31,96.42,20.86,101.25,92.70,95.36,21.64,99.60,99.42,94.76,22.42,101.28,98.72,95.26,21.66,101.24,97.43,102.99,24.54,101.76,98.24,100.51,25.38,99.80,85.82,108.42,28.17,102.00,92.07,102.64
8,Kota Denpasar,31.06,103.21,89.65,102.01,40.10,106.76,79.38,79.37,41.84,104.41,91.78,91.16,43.30,98.35,88.27,92.52,43.34,102.51,79.20,93.04,42.73,101.38,82.46,93.12,43.64,102.49,86.80,92.05,45.70,102.36,88.88,84.34,43.14,103.63,102.41,90.68,43.40,105.99,95.11,89.25
9,Provinsi Bali,26.50,105.00,86.61,96.82,27.02,104.97,86.41,96.19,29.40,104.35,90.04,96.29,30.51,103.02,87.74,96.14,29.84,103.52,87.96,97.26,30.55,102.87,88.67,97.40,30.64,103.09,89.87,98.68,31.94,103.16,90.82,96.23,36.46,102.70,93.40,98.18,38.15,104.51,94.16,95.25


In [322]:
data_280.to_excel("data_280.xlsx", sheet_name= "APK", index=False)

Banyaknya Sekolah Menengah Kejuruan Provinsi Bali Menurut Kabupaten/Kota (279)

In [324]:
data_279 = getDataByVarId(domain='5100',varId='279')
data_279

,verval_val,vervar_label,var_val,turvar_val,turvar_label,tahun_val,tahun_label,turtahun_val,turtahun_label,id,content_val,content_label
0,1,Kab. Jembrana,279,0,Tidak ada,123,2023,0,Tahun,127901230,127901230,9
1,2,Kab. Tabanan,279,0,Tidak ada,123,2023,0,Tahun,227901230,227901230,14
2,3,Kab. Badung,279,0,Tidak ada,123,2023,0,Tahun,327901230,327901230,29
3,4,Kab. Gianyar,279,0,Tidak ada,123,2023,0,Tahun,427901230,427901230,27
4,5,Kab. Klungkung,279,0,Tidak ada,123,2023,0,Tahun,527901230,527901230,7
...,...,...,...,...,...,...,...,...,...,...,...,...
5,6,Kab. Bangli,279,0,Tidak ada,114,2014,0,Tahun,627901140,627901140,14
6,7,Kab. Karangasem,279,0,Tidak ada,114,2014,0,Tahun,727901140,727901140,11
7,8,Kab. Buleleng,279,0,Tidak ada,114,2014,0,Tahun,827901140,827901140,27
8,9,Kota Denpasar,279,0,Tidak ada,114,2014,0,Tahun,927901140,927901140,32


In [325]:

data_279 = data_279[['verval_val','vervar_label','tahun_label','content_label']].copy()
data_279["vervar_label"] = data_279["vervar_label"].str.replace(r"<.*?>", "", regex=True)


data_tb = data_279.rename(columns={
    'vervar_label'   : 'Kabupaten/Kota',
    'tahun_label'    : 'Tahun',
    'content_label'  : 'Nilai'
})

api_order = data_279['vervar_label'].drop_duplicates().tolist()
data_tb['Kabupaten/Kota'] = pd.Categorical(data_tb['Kabupaten/Kota'], categories=api_order, ordered=True)

data_tb = data_tb.pivot_table(
    index=['Kabupaten/Kota'],
    columns=['Tahun'],
    values='Nilai'
)

data_tb.columns = ['_'.join([str(c) for c in col if c]).strip() if isinstance(col, tuple) else str(col) 
                   for col in data_tb.columns.values]

data_tb = data_tb.reset_index()

data_tb


C:\Users\user\AppData\Local\Temp\ipykernel_13044\1828549323.py:14: FutureWarning: The default value of observed=False is deprecated and will change to observed=True in a future version of pandas. Specify observed=False to silence this warning and retain the current behavior
  data_tb = data_tb.pivot_table(


,Kabupaten/Kota,2014,2015,2016,2017,2018,2019,2020,2021,2022,2023
0,Kab. Jembrana,10.0,10.0,10.0,10.0,10.0,13.0,9.0,9.0,9.0,9.0
1,Kab. Tabanan,15.0,15.0,15.0,14.0,14.0,14.0,14.0,14.0,14.0,14.0
2,Kab. Badung,22.0,22.0,24.0,25.0,25.0,22.0,26.0,26.0,28.0,29.0
3,Kab. Gianyar,28.0,30.0,31.0,31.0,29.0,12.0,28.0,27.0,28.0,27.0
4,Kab. Klungkung,7.0,7.0,8.0,8.0,8.0,11.0,6.0,6.0,6.0,7.0
5,Kab. Bangli,14.0,14.0,13.0,13.0,13.0,6.0,12.0,12.0,13.0,12.0
6,Kab. Karangasem,11.0,12.0,13.0,13.0,13.0,18.0,12.0,13.0,13.0,12.0
7,Kab. Buleleng,27.0,29.0,30.0,29.0,29.0,32.0,26.0,27.0,27.0,26.0
8,Kota Denpasar,32.0,32.0,32.0,33.0,33.0,33.0,32.0,32.0,33.0,34.0
9,Provinsi Bali,166.0,171.0,176.0,176.0,174.0,161.0,165.0,166.0,171.0,170.0


In [330]:
data_279 = data_279[['verval_val','vervar_label','tahun_label','content_label']].copy()
data_279["vervar_label"] = data_279["vervar_label"].str.replace(r"<.*?>", "", regex=True)

# --- rename kolom ---
data_tb = data_279.rename(columns={
    'vervar_label'   : 'Kabupaten/Kota',
    'tahun_label'    : 'Tahun',
    'content_label'  : 'Nilai'
})

# --- urutan sesuai API ---
api_order = data_279['vervar_label'].drop_duplicates().tolist()
data_tb['Kabupaten/Kota'] = pd.Categorical(
    data_tb['Kabupaten/Kota'],
    categories=api_order,
    ordered=True
)

# --- pivot ---
data_tb = data_tb.pivot_table(
    index=['Kabupaten/Kota'],
    columns=['Tahun'],
    values='Nilai'
)

# --- rapikan nama kolom, hapus .0 ---
data_tb.columns = [
    str(int(float(c))) if str(c).replace('.','',1).isdigit() else str(c)
    for c in data_tb.columns
]

# --- reset index ---
data_tb = data_tb.reset_index()

data_tb


C:\Users\user\AppData\Local\Temp\ipykernel_13044\1408965728.py:20: FutureWarning: The default value of observed=False is deprecated and will change to observed=True in a future version of pandas. Specify observed=False to silence this warning and retain the current behavior
  data_tb = data_tb.pivot_table(


,Kabupaten/Kota,2014,2015,2016,2017,2018,2019,2020,2021,2022,2023
0,Kab. Jembrana,10.0,10.0,10.0,10.0,10.0,13.0,9.0,9.0,9.0,9.0
1,Kab. Tabanan,15.0,15.0,15.0,14.0,14.0,14.0,14.0,14.0,14.0,14.0
2,Kab. Badung,22.0,22.0,24.0,25.0,25.0,22.0,26.0,26.0,28.0,29.0
3,Kab. Gianyar,28.0,30.0,31.0,31.0,29.0,12.0,28.0,27.0,28.0,27.0
4,Kab. Klungkung,7.0,7.0,8.0,8.0,8.0,11.0,6.0,6.0,6.0,7.0
5,Kab. Bangli,14.0,14.0,13.0,13.0,13.0,6.0,12.0,12.0,13.0,12.0
6,Kab. Karangasem,11.0,12.0,13.0,13.0,13.0,18.0,12.0,13.0,13.0,12.0
7,Kab. Buleleng,27.0,29.0,30.0,29.0,29.0,32.0,26.0,27.0,27.0,26.0
8,Kota Denpasar,32.0,32.0,32.0,33.0,33.0,33.0,32.0,32.0,33.0,34.0
9,Provinsi Bali,166.0,171.0,176.0,176.0,174.0,161.0,165.0,166.0,171.0,170.0


In [ ]:
data_279 = data_279[['verval_val','vervar_label','tahun_label','content_label']].copy()
data_279["vervar_label"] = data_279["vervar_label"].str.replace(r"<.*?>", "", regex=True)

# --- rename kolom ---
data_tb = data_279.rename(columns={
    'vervar_label'   : 'Kabupaten/Kota',
    'tahun_label'    : 'Tahun',
    'content_label'  : 'Nilai'
})

# --- urutan sesuai API ---
api_order = data_279['vervar_label'].drop_duplicates().tolist()
data_tb['Kabupaten/Kota'] = pd.Categorical(data_tb['Kabupaten/Kota'], categories=api_order, ordered=True)

# --- pivot ---
data_tb = data_tb.pivot_table(
    index=['Kabupaten/Kota'],
    columns=['Tahun'],
    values='Nilai'
)

# --- rapikan nama kolom ---
data_tb.columns = [
    '_'.join([str(c) for c in col if c]).strip() if isinstance(col, tuple) else str(col) 
    for col in data_tb.columns.values
]

data_tb = data_tb.reset_index()

# --- hapus .0 pada angka bulat ---
data_tb = data_tb.applymap(
    lambda x: int(x) if isinstance(x, (int, float)) and x.is_integer() else x
)

data_tb


C:\Users\user\AppData\Local\Temp\ipykernel_13044\1668241557.py:19: FutureWarning: The default value of observed=False is deprecated and will change to observed=True in a future version of pandas. Specify observed=False to silence this warning and retain the current behavior
  data_tb = data_tb.pivot_table(
C:\Users\user\AppData\Local\Temp\ipykernel_13044\1668241557.py:34: FutureWarning: DataFrame.applymap has been deprecated. Use DataFrame.map instead.
  data_tb = data_tb.applymap(


,Kabupaten/Kota,2014,2015,2016,2017,2018,2019,2020,2021,2022,2023
0,Kab. Jembrana,10,10,10,10,10,13,9,9,9,9
1,Kab. Tabanan,15,15,15,14,14,14,14,14,14,14
2,Kab. Badung,22,22,24,25,25,22,26,26,28,29
3,Kab. Gianyar,28,30,31,31,29,12,28,27,28,27
4,Kab. Klungkung,7,7,8,8,8,11,6,6,6,7
5,Kab. Bangli,14,14,13,13,13,6,12,12,13,12
6,Kab. Karangasem,11,12,13,13,13,18,12,13,13,12
7,Kab. Buleleng,27,29,30,29,29,32,26,27,27,26
8,Kota Denpasar,32,32,32,33,33,33,32,32,33,34
9,Provinsi Bali,166,171,176,176,174,161,165,166,171,170


In [332]:
data_279.to_excel("data_279.xlsx", sheet_name= "Jumlah_SMK", index=False)

Banyaknya Sekolah Menengah Atas Provinsi Bali Menurut Kabupaten/Kota (278)

In [ ]:
data_278 = getDataByVarId (domain='5100',varId='278')
data_278

,verval_val,vervar_label,var_val,turvar_val,turvar_label,tahun_val,tahun_label,turtahun_val,turtahun_label,id,content_val,content_label
0,1,Kab. Jembrana,278,0,Tidak ada,123,2023,0,Tahun,127801230,127801230,22
1,2,Kab. Tabanan,278,0,Tidak ada,123,2023,0,Tahun,227801230,227801230,14
2,3,Kab. Badung,278,0,Tidak ada,123,2023,0,Tahun,327801230,327801230,33
3,4,Kab. Gianyar,278,0,Tidak ada,123,2023,0,Tahun,427801230,427801230,12
4,5,Kab. Klungkung,278,0,Tidak ada,123,2023,0,Tahun,527801230,527801230,12
...,...,...,...,...,...,...,...,...,...,...,...,...
5,6,Kab. Bangli,278,0,Tidak ada,114,2014,0,Tahun,627801140,627801140,6
6,7,Kab. Karangasem,278,0,Tidak ada,114,2014,0,Tahun,727801140,727801140,19
7,8,Kab. Buleleng,278,0,Tidak ada,114,2014,0,Tahun,827801140,827801140,39
8,9,Kota Denpasar,278,0,Tidak ada,114,2014,0,Tahun,927801140,927801140,33


In [334]:
data_278 = data_278[['verval_val','vervar_label','tahun_label','content_label']].copy()
data_278["vervar_label"] = data_278["vervar_label"].str.replace(r"<.*?>", "", regex=True)

data_tb = data_278.rename(columns={
    'vervar_label'   : 'Kabupaten/Kota',
    'tahun_label'    : 'Tahun',
    'content_label'  : 'Nilai'
})

api_order = data_278['vervar_label'].drop_duplicates().tolist()
data_tb['Kabupaten/Kota'] = pd.Categorical(data_tb['Kabupaten/Kota'], categories=api_order, ordered=True)

data_tb = data_tb.pivot_table(
    index=['Kabupaten/Kota'],
    columns=['Tahun'],
    values='Nilai'
)

data_tb.columns = [
    '_'.join([str(c) for c in col if c]).strip() if isinstance(col, tuple) else str(col) 
    for col in data_tb.columns.values
]

data_tb = data_tb.reset_index()

data_tb = data_tb.applymap(
    lambda x: int(x) if isinstance(x, (int, float)) and x.is_integer() else x
)

data_tb


C:\Users\user\AppData\Local\Temp\ipykernel_13044\1598260358.py:13: FutureWarning: The default value of observed=False is deprecated and will change to observed=True in a future version of pandas. Specify observed=False to silence this warning and retain the current behavior
  data_tb = data_tb.pivot_table(
C:\Users\user\AppData\Local\Temp\ipykernel_13044\1598260358.py:26: FutureWarning: DataFrame.applymap has been deprecated. Use DataFrame.map instead.
  data_tb = data_tb.applymap(


,Kabupaten/Kota,2014,2015,2016,2017,2018,2019,2020,2021,2022,2023
0,Kab. Jembrana,19,18,18,21,21,21,21,21,24,22
1,Kab. Tabanan,19,18,18,18,18,18,15,15,14,14
2,Kab. Badung,19,20,19,24,22,24,25,26,30,33
3,Kab. Gianyar,15,15,13,12,12,13,11,10,12,12
4,Kab. Klungkung,11,12,12,12,12,12,12,12,12,12
5,Kab. Bangli,6,6,6,6,6,6,6,6,6,6
6,Kab. Karangasem,19,19,19,20,20,20,21,21,21,21
7,Kab. Buleleng,39,39,39,39,39,38,38,38,37,36
8,Kota Denpasar,33,34,34,35,36,37,39,39,41,41
9,Provinsi Bali,180,181,178,187,186,189,188,188,197,197


In [335]:
data_278.to_excel("data_278.xlsx", sheet_name= "Jumlah_SMA", index=False)

Banyaknya Sekolah Lanjutan Pertama/Sederajat Provinsi Bali Menurut Kabupaten/Kota (277)

In [336]:
data_277 = getDataByVarId (domain='5100',varId='277')
data_277

,verval_val,vervar_label,var_val,turvar_val,turvar_label,tahun_val,tahun_label,turtahun_val,turtahun_label,id,content_val,content_label
0,1,Kab. Jembrana,277,0,Tidak ada,123,2023,0,Tahun,127701230,127701230,38
1,2,Kab. Tabanan,277,0,Tidak ada,123,2023,0,Tahun,227701230,227701230,45
2,3,Kab. Badung,277,0,Tidak ada,123,2023,0,Tahun,327701230,327701230,69
3,4,Kab. Gianyar,277,0,Tidak ada,123,2023,0,Tahun,427701230,427701230,34
4,5,Kab. Klungkung,277,0,Tidak ada,123,2023,0,Tahun,527701230,527701230,23
...,...,...,...,...,...,...,...,...,...,...,...,...
5,6,Kab. Bangli,277,0,Tidak ada,114,2014,0,Tahun,627701140,627701140,28
6,7,Kab. Karangasem,277,0,Tidak ada,114,2014,0,Tahun,727701140,727701140,53
7,8,Kab. Buleleng,277,0,Tidak ada,114,2014,0,Tahun,827701140,827701140,87
8,9,Kota Denpasar,277,0,Tidak ada,114,2014,0,Tahun,927701140,927701140,63


In [ ]:
# data_277 = data_277[['verval_val','vervar_label','tahun_label','content_label']].copy()
# data_277["vervar_label"] = data_278["vervar_label"].str.replace(r"<.*?>", "", regex=True)

# data_tb = data_277.rename(columns={
#     'vervar_label'   : 'Kabupaten/Kota',
#     'tahun_label'    : 'Tahun',
#     'content_label'  : 'Nilai'
# })

# api_order = data_277['vervar_label'].drop_duplicates().tolist()
# data_tb['Kabupaten/Kota'] = pd.Categorical(data_tb['Kabupaten/Kota'], categories=api_order, ordered=True)

# data_tb = data_tb.pivot_table(
#     index=['Kabupaten/Kota'],
#     columns=['Tahun'],
#     values='Nilai'
# )

# data_tb.columns = [
#     '_'.join([str(c) for c in col if c]).strip() if isinstance(col, tuple) else str(col) 
#     for col in data_tb.columns.values
# ]

# data_tb = data_tb.reset_index()

# data_tb = data_tb.applymap(
#     lambda x: int(x) if isinstance(x, (int, float)) and x.is_integer() else x
# )

# data_tb


C:\Users\user\AppData\Local\Temp\ipykernel_13044\1079845433.py:13: FutureWarning: The default value of observed=False is deprecated and will change to observed=True in a future version of pandas. Specify observed=False to silence this warning and retain the current behavior
  data_tb = data_tb.pivot_table(
C:\Users\user\AppData\Local\Temp\ipykernel_13044\1079845433.py:26: FutureWarning: DataFrame.applymap has been deprecated. Use DataFrame.map instead.
  data_tb = data_tb.applymap(


,Kabupaten/Kota,2014,2015,2016,2017,2018,2019,2020,2021,2022,2023
0,Kab. Jembrana,35,35,34,38,38,38,38,39,39,38
1,Kab. Tabanan,43,43,42,43,45,46,46,45,45,45
2,Kab. Badung,51,55,59,64,68,71,69,72,69,69
3,Kab. Gianyar,46,46,44,44,44,46,42,40,38,34
4,Kab. Klungkung,24,24,24,24,24,24,24,23,23,23
5,Kab. Bangli,28,28,29,29,29,29,28,28,28,28
6,Kab. Karangasem,53,53,53,51,50,51,51,51,51,52
7,Kab. Buleleng,87,88,88,87,88,89,87,84,82,83
8,Kota Denpasar,63,66,66,70,72,74,78,79,82,87
9,Provinsi Bali,430,438,439,450,458,468,463,461,457,459


In [ ]:
data_277.to_excel("data_277.xlsx", sheet_name= "Sekolah_Lanjutan", index=False)

Banyaknya Sekolah Dasar/Sederajat Provinsi Bali Menurut Kabupaten/Kota (276)

In [ ]:
data_276 = getDataByVarId (domain='5100',varId='276')
data_276

,verval_val,vervar_label,var_val,turvar_val,turvar_label,tahun_val,tahun_label,turtahun_val,turtahun_label,id,content_val,content_label
0,1,Kab. Jembrana,276,0,Tidak ada,123,2023,0,Tahun,127601230,127601230,206
1,2,Kab. Tabanan,276,0,Tidak ada,123,2023,0,Tahun,227601230,227601230,293
2,3,Kab. Badung,276,0,Tidak ada,123,2023,0,Tahun,327601230,327601230,295
3,4,Kab. Gianyar,276,0,Tidak ada,123,2023,0,Tahun,427601230,427601230,294
4,5,Kab. Klungkung,276,0,Tidak ada,123,2023,0,Tahun,527601230,527601230,136
...,...,...,...,...,...,...,...,...,...,...,...,...
5,6,Kab. Bangli,276,0,Tidak ada,114,2014,0,Tahun,627601140,627601140,163
6,7,Kab. Karangasem,276,0,Tidak ada,114,2014,0,Tahun,727601140,727601140,363
7,8,Kab. Buleleng,276,0,Tidak ada,114,2014,0,Tahun,827601140,827601140,504
8,9,Kota Denpasar,276,0,Tidak ada,114,2014,0,Tahun,927601140,927601140,226


In [10]:
data_276 = data_276[['verval_val','vervar_label','tahun_label','content_label']].copy()
data_276["vervar_label"] = data_276["vervar_label"].str.replace(r"<.*?>", "", regex=True)

data_tb = data_276.rename(columns={
    'vervar_label'   : 'Kabupaten/Kota',
    'tahun_label'    : 'Tahun',
    'content_label'  : 'Nilai'
})

api_order = data_276['vervar_label'].drop_duplicates().tolist()
data_tb['Kabupaten/Kota'] = pd.Categorical(data_tb['Kabupaten/Kota'], categories=api_order, ordered=True)

data_tb = data_tb.pivot_table(
    index=['Kabupaten/Kota'],
    columns=['Tahun'],
    values='Nilai'
)

data_tb.columns = [
    '_'.join([str(c) for c in col if c]).strip() if isinstance(col, tuple) else str(col) 
    for col in data_tb.columns.values
]

data_tb = data_tb.reset_index()

data_tb = data_tb.applymap(
    lambda x: int(x) if isinstance(x, (int, float)) and x.is_integer() else x
)

data_tb


C:\Users\user\AppData\Local\Temp\ipykernel_2568\2822272322.py:13: FutureWarning: The default value of observed=False is deprecated and will change to observed=True in a future version of pandas. Specify observed=False to silence this warning and retain the current behavior
  data_tb = data_tb.pivot_table(
C:\Users\user\AppData\Local\Temp\ipykernel_2568\2822272322.py:26: FutureWarning: DataFrame.applymap has been deprecated. Use DataFrame.map instead.
  data_tb = data_tb.applymap(


,Kabupaten/Kota,2014,2015,2016,2017,2018,2019,2020,2021,2022,2023
0,Kab. Jembrana,196,196,198,205,204,204,204,205,206,206
1,Kab. Tabanan,328,329,327,326,322,322,313,300,298,293
2,Kab. Badung,274,279,283,292,292,293,292,294,295,295
3,Kab. Gianyar,291,292,294,294,294,294,292,293,295,294
4,Kab. Klungkung,139,139,139,138,138,138,138,138,136,136
5,Kab. Bangli,163,164,164,165,165,166,166,167,165,165
6,Kab. Karangasem,363,363,363,363,363,364,364,364,364,365
7,Kab. Buleleng,504,504,503,504,507,508,509,492,491,490
8,Kota Denpasar,226,230,232,240,244,247,248,251,255,255
9,Provinsi Bali,2484,2496,2503,2527,2529,2536,2526,2504,2505,2499


In [11]:
data_276.to_excel("data_276.xlsx", sheet_name= "Sekolah_Dasar", index=False)

Banyaknya Murid Sekolah Menengah Kejuruan Provinsi Bali Menurut Kabupaten/Kota (275)

In [12]:
data_275 = getDataByVarId (domain='5100',varId='275')
data_275

,verval_val,vervar_label,var_val,turvar_val,turvar_label,tahun_val,tahun_label,turtahun_val,turtahun_label,id,content_val,content_label
0,1,Kab. Jembrana,275,0,Tidak ada,123,2023,0,Tahun,127501230,127501230,4405
1,2,Kab. Tabanan,275,0,Tidak ada,123,2023,0,Tahun,227501230,227501230,6419
2,3,Kab. Badung,275,0,Tidak ada,123,2023,0,Tahun,327501230,327501230,11502
3,4,Kab. Gianyar,275,0,Tidak ada,123,2023,0,Tahun,427501230,427501230,14885
4,5,Kab. Klungkung,275,0,Tidak ada,123,2023,0,Tahun,527501230,527501230,3551
...,...,...,...,...,...,...,...,...,...,...,...,...
5,6,Kab. Bangli,275,0,Tidak ada,114,2014,0,Tahun,627501140,627501140,3631
6,7,Kab. Karangasem,275,0,Tidak ada,114,2014,0,Tahun,727501140,727501140,4982
7,8,Kab. Buleleng,275,0,Tidak ada,114,2014,0,Tahun,827501140,827501140,10871
8,9,Kota Denpasar,275,0,Tidak ada,114,2014,0,Tahun,927501140,927501140,21522


In [ ]:
data_275 = data_275[['verval_val','vervar_label','tahun_label','content_label']].copy()
data_275["vervar_label"] = data_275["vervar_label"].str.replace(r"<.*?>", "", regex=True)

data_tb = data_275.rename(columns={
    'vervar_label'   : 'Kabupaten/Kota',
    'tahun_label'    : 'Tahun',
    'content_label'  : 'Nilai'
})

api_order = data_275['vervar_label'].drop_duplicates().tolist()
data_tb['Kabupaten/Kota'] = pd.Categorical(data_tb['Kabupaten/Kota'], categories=api_order, ordered=True)

data_tb = data_tb.pivot_table(
    index=['Kabupaten/Kota'],
    columns=['Tahun'],
    values='Nilai'
)

data_tb.columns = [
    '_'.join([str(c) for c in col if c]).strip() if isinstance(col, tuple) else str(col) 
    for col in data_tb.columns.values
]

data_tb = data_tb.reset_index()

data_tb = data_tb.applymap(
    lambda x: int(x) if isinstance(x, (int, float)) and x.is_integer() else x
)

data_tb


C:\Users\user\AppData\Local\Temp\ipykernel_2568\2007863065.py:13: FutureWarning: The default value of observed=False is deprecated and will change to observed=True in a future version of pandas. Specify observed=False to silence this warning and retain the current behavior
  data_tb = data_tb.pivot_table(
C:\Users\user\AppData\Local\Temp\ipykernel_2568\2007863065.py:26: FutureWarning: DataFrame.applymap has been deprecated. Use DataFrame.map instead.
  data_tb = data_tb.applymap(


,Kabupaten/Kota,2014,2015,2016,2017,2018,2019,2020,2021,2022,2023
0,Kab. Jembrana,4827,4703,5395,5428,5386,5830,5377,5126,4548,4405
1,Kab. Tabanan,6209,6776,7033,7327,7192,8010,7152,7528,6384,6419
2,Kab. Badung,13546,14007,14440,15255,15403,13318,14394,12696,11202,11502
3,Kab. Gianyar,14146,14807,15666,16398,16945,7755,16524,15388,14539,14885
4,Kab. Klungkung,3445,3680,3815,3767,3645,6955,3517,3402,3469,3551
5,Kab. Bangli,3631,3896,4162,4592,4961,4082,5392,5106,5132,5270
6,Kab. Karangasem,4982,5537,6333,6613,6568,9822,6507,6527,6647,6935
7,Kab. Buleleng,10871,11682,12911,14122,14763,15700,15128,18024,15059,15406
8,Kota Denpasar,21522,21812,22333,23141,24032,18709,23811,22981,21019,20756
9,Provinsi Bali,83179,86900,92088,96643,98895,90181,97802,96778,87999,89129


In [15]:
data_275.to_excel("data_275.xlsx", sheet_name= "Banyak_Murid_SMK", index=False)

Banyaknya Murid Sekolah Menengah Atas/Sederajat Provinsi Bali Menurut Kabupaten/Kota (274)

In [16]:
data_274 = getDataByVarId (domain='5100',varId='274')
data_274

,verval_val,vervar_label,var_val,turvar_val,turvar_label,tahun_val,tahun_label,turtahun_val,turtahun_label,id,content_val,content_label
0,1,Kab. Jembrana,274,0,Tidak ada,123,2023,0,Tahun,127401230,127401230,8527
1,2,Kab. Tabanan,274,0,Tidak ada,123,2023,0,Tahun,227401230,227401230,8504
2,3,Kab. Badung,274,0,Tidak ada,123,2023,0,Tahun,327401230,327401230,16004
3,4,Kab. Gianyar,274,0,Tidak ada,123,2023,0,Tahun,427401230,427401230,9211
4,5,Kab. Klungkung,274,0,Tidak ada,123,2023,0,Tahun,527401230,527401230,6703
...,...,...,...,...,...,...,...,...,...,...,...,...
5,6,Kab. Bangli,274,0,Tidak ada,114,2014,0,Tahun,627401140,627401140,3487
6,7,Kab. Karangasem,274,0,Tidak ada,114,2014,0,Tahun,727401140,727401140,8157
7,8,Kab. Buleleng,274,0,Tidak ada,114,2014,0,Tahun,827401140,827401140,14946
8,9,Kota Denpasar,274,0,Tidak ada,114,2014,0,Tahun,927401140,927401140,18192


In [17]:
data_274 = data_274[['verval_val','vervar_label','tahun_label','content_label']].copy()
data_274["vervar_label"] = data_274["vervar_label"].str.replace(r"<.*?>", "", regex=True)

data_tb = data_274.rename(columns={
    'vervar_label'   : 'Kabupaten/Kota',
    'tahun_label'    : 'Tahun',
    'content_label'  : 'Nilai'
})

api_order = data_274['vervar_label'].drop_duplicates().tolist()
data_tb['Kabupaten/Kota'] = pd.Categorical(data_tb['Kabupaten/Kota'], categories=api_order, ordered=True)

data_tb = data_tb.pivot_table(
    index=['Kabupaten/Kota'],
    columns=['Tahun'],
    values='Nilai'
)

data_tb.columns = [
    '_'.join([str(c) for c in col if c]).strip() if isinstance(col, tuple) else str(col) 
    for col in data_tb.columns.values
]

data_tb = data_tb.reset_index()

data_tb = data_tb.applymap(
    lambda x: int(x) if isinstance(x, (int, float)) and x.is_integer() else x
)

data_tb


C:\Users\user\AppData\Local\Temp\ipykernel_2568\2419880795.py:13: FutureWarning: The default value of observed=False is deprecated and will change to observed=True in a future version of pandas. Specify observed=False to silence this warning and retain the current behavior
  data_tb = data_tb.pivot_table(
C:\Users\user\AppData\Local\Temp\ipykernel_2568\2419880795.py:26: FutureWarning: DataFrame.applymap has been deprecated. Use DataFrame.map instead.
  data_tb = data_tb.applymap(


,Kabupaten/Kota,2014,2015,2016,2017,2018,2019,2020,2021,2022,2023
0,Kab. Jembrana,6577,5411,7252,7269,7551,7870,8037,7837,8072,8527
1,Kab. Tabanan,7666,7139,8338,8316,8450,8336,8456,8503,8762,8504
2,Kab. Badung,10672,12233,12018,12534,12749,13359,13973,14719,15473,16004
3,Kab. Gianyar,7668,10766,7844,7992,7905,7874,7895,7949,8942,9211
4,Kab. Klungkung,5469,4277,6004,6271,6820,7093,7145,6755,6791,6703
5,Kab. Bangli,3487,3608,3637,3778,3901,4082,4111,4239,4509,4704
6,Kab. Karangasem,8157,6172,8596,9275,9761,10305,10380,10288,10391,10330
7,Kab. Buleleng,14946,12533,15550,15824,16098,16413,16440,16002,15818,15619
8,Kota Denpasar,18192,19583,19698,19266,19021,19156,19117,19046,20138,21010
9,Provinsi Bali,82834,81722,88937,90525,92256,94488,95554,95338,98896,100612


In [18]:
data_274.to_excel("data_274.xlsx", sheet_name= "Banyak_Murid_SMA", index=False)

Banyaknya Murid Sekolah Lanjutan Pertama/Sederajat Provinsi Bali Menurut Kabupaten/Kota (273)

In [19]:
data_273 = getDataByVarId (domain='5100',varId='273')
data_273

,verval_val,vervar_label,var_val,turvar_val,turvar_label,tahun_val,tahun_label,turtahun_val,turtahun_label,id,content_val,content_label
0,1,Kab. Jembrana,273,0,Tidak ada,123,2023,0,Tahun,127301230,127301230,14167
1,2,Kab. Tabanan,273,0,Tidak ada,123,2023,0,Tahun,227301230,227301230,16775
2,3,Kab. Badung,273,0,Tidak ada,123,2023,0,Tahun,327301230,327301230,27804
3,4,Kab. Gianyar,273,0,Tidak ada,123,2023,0,Tahun,427301230,427301230,22650
4,5,Kab. Klungkung,273,0,Tidak ada,123,2023,0,Tahun,527301230,527301230,8686
...,...,...,...,...,...,...,...,...,...,...,...,...
5,6,Kab. Bangli,273,0,Tidak ada,114,2014,0,Tahun,627301140,627301140,10742
6,7,Kab. Karangasem,273,0,Tidak ada,114,2014,0,Tahun,727301140,727301140,20889
7,8,Kab. Buleleng,273,0,Tidak ada,114,2014,0,Tahun,827301140,827301140,34468
8,9,Kota Denpasar,273,0,Tidak ada,114,2014,0,Tahun,927301140,927301140,38996


In [ ]:
data_273 = data_273[['verval_val','vervar_label','tahun_label','content_label']].copy()
data_273["vervar_label"] = data_273["vervar_label"].str.replace(r"<.*?>", "", regex=True)

data_tb = data_273.rename(columns={
    'vervar_label'   : 'Kabupaten/Kota',
    'tahun_label'    : 'Tahun',
    'content_label'  : 'Nilai'
})

api_order = data_273['vervar_label'].drop_duplicates().tolist()
data_tb['Kabupaten/Kota'] = pd.Categorical(data_tb['Kabupaten/Kota'], categories=api_order, ordered=True)

data_tb = data_tb.pivot_table(
    index=['Kabupaten/Kota'],
    columns=['Tahun'],
    values='Nilai'
)

data_tb.columns = [
    '_'.join([str(c) for c in col if c]).strip() if isinstance(col, tuple) else str(col) 
    for col in data_tb.columns.values
]

data_tb = data_tb.reset_index()

data_tb = data_tb.applymap(
    lambda x: int(x) if isinstance(x, (int, float)) and x.is_integer() else x
)

data_tb


C:\Users\user\AppData\Local\Temp\ipykernel_2568\713974550.py:13: FutureWarning: The default value of observed=False is deprecated and will change to observed=True in a future version of pandas. Specify observed=False to silence this warning and retain the current behavior
  data_tb = data_tb.pivot_table(
C:\Users\user\AppData\Local\Temp\ipykernel_2568\713974550.py:26: FutureWarning: DataFrame.applymap has been deprecated. Use DataFrame.map instead.
  data_tb = data_tb.applymap(


,Kabupaten/Kota,2014,2015,2016,2017,2018,2019,2020,2021,2022,2023
0,Kab. Jembrana,14297,14379,14547,14154,13706,13412,13068,13977,14165,14167
1,Kab. Tabanan,19814,19604,19368,19047,18527,18261,18086,18218,17514,16775
2,Kab. Badung,29212,29819,30039,29738,29177,29033,29076,28537,28174,27804
3,Kab. Gianyar,23274,23551,23455,22995,22407,22394,22916,23263,23021,22650
4,Kab. Klungkung,9517,9689,9540,9399,9091,9172,9218,9286,8913,8686
5,Kab. Bangli,10742,11019,11230,11344,11084,10951,11001,11068,11191,11310
6,Kab. Karangasem,20889,21081,21359,21496,21354,21515,21632,21443,21714,21949
7,Kab. Buleleng,34468,35164,35511,35220,34220,33964,34035,35403,35439,35531
8,Kota Denpasar,38996,39618,39740,38866,38191,39245,39613,38287,36267,36094
9,Provinsi Bali,201209,203924,204789,202259,197757,197947,198645,199482,196398,194966


In [21]:
data_273.to_excel("data_273.xlsx", sheet_name= "Banyak_Murid_SLP", index=False)

Banyaknya Murid Sekolah Dasar/Sederajat Provinsi Bali Menurut Kabupaten/Kota (272)

In [22]:
data_272 = getDataByVarId (domain='5100',varId='272')
data_272

,verval_val,vervar_label,var_val,turvar_val,turvar_label,tahun_val,tahun_label,turtahun_val,turtahun_label,id,content_val,content_label
0,1,Kab. Jembrana,272,0,Tidak ada,123,2023,0,Tahun,127201230,127201230,27398
1,2,Kab. Tabanan,272,0,Tidak ada,123,2023,0,Tahun,227201230,227201230,34978
2,3,Kab. Badung,272,0,Tidak ada,123,2023,0,Tahun,327201230,327201230,59528
3,4,Kab. Gianyar,272,0,Tidak ada,123,2023,0,Tahun,427201230,427201230,44288
4,5,Kab. Klungkung,272,0,Tidak ada,123,2023,0,Tahun,527201230,527201230,17325
...,...,...,...,...,...,...,...,...,...,...,...,...
5,6,Kab. Bangli,272,0,Tidak ada,114,2014,0,Tahun,627201140,627201140,23208
6,7,Kab. Karangasem,272,0,Tidak ada,114,2014,0,Tahun,727201140,727201140,46698
7,8,Kab. Buleleng,272,0,Tidak ada,114,2014,0,Tahun,827201140,827201140,73783
8,9,Kota Denpasar,272,0,Tidak ada,114,2014,0,Tahun,927201140,927201140,88610


In [23]:
data_272 = data_272[['verval_val','vervar_label','tahun_label','content_label']].copy()
data_272["vervar_label"] = data_272["vervar_label"].str.replace(r"<.*?>", "", regex=True)

data_tb = data_273.rename(columns={
    'vervar_label'   : 'Kabupaten/Kota',
    'tahun_label'    : 'Tahun',
    'content_label'  : 'Nilai'
})

api_order = data_272['vervar_label'].drop_duplicates().tolist()
data_tb['Kabupaten/Kota'] = pd.Categorical(data_tb['Kabupaten/Kota'], categories=api_order, ordered=True)

data_tb = data_tb.pivot_table(
    index=['Kabupaten/Kota'],
    columns=['Tahun'],
    values='Nilai'
)

data_tb.columns = [
    '_'.join([str(c) for c in col if c]).strip() if isinstance(col, tuple) else str(col) 
    for col in data_tb.columns.values
]

data_tb = data_tb.reset_index()

data_tb = data_tb.applymap(
    lambda x: int(x) if isinstance(x, (int, float)) and x.is_integer() else x
)

data_tb


C:\Users\user\AppData\Local\Temp\ipykernel_2568\6542740.py:13: FutureWarning: The default value of observed=False is deprecated and will change to observed=True in a future version of pandas. Specify observed=False to silence this warning and retain the current behavior
  data_tb = data_tb.pivot_table(
C:\Users\user\AppData\Local\Temp\ipykernel_2568\6542740.py:26: FutureWarning: DataFrame.applymap has been deprecated. Use DataFrame.map instead.
  data_tb = data_tb.applymap(


,Kabupaten/Kota,2014,2015,2016,2017,2018,2019,2020,2021,2022,2023
0,Kab. Jembrana,14297,14379,14547,14154,13706,13412,13068,13977,14165,14167
1,Kab. Tabanan,19814,19604,19368,19047,18527,18261,18086,18218,17514,16775
2,Kab. Badung,29212,29819,30039,29738,29177,29033,29076,28537,28174,27804
3,Kab. Gianyar,23274,23551,23455,22995,22407,22394,22916,23263,23021,22650
4,Kab. Klungkung,9517,9689,9540,9399,9091,9172,9218,9286,8913,8686
5,Kab. Bangli,10742,11019,11230,11344,11084,10951,11001,11068,11191,11310
6,Kab. Karangasem,20889,21081,21359,21496,21354,21515,21632,21443,21714,21949
7,Kab. Buleleng,34468,35164,35511,35220,34220,33964,34035,35403,35439,35531
8,Kota Denpasar,38996,39618,39740,38866,38191,39245,39613,38287,36267,36094
9,Provinsi Bali,201209,203924,204789,202259,197757,197947,198645,199482,196398,194966


In [24]:
data_272.to_excel("data_272.xlsx", sheet_name= "Banyak_Murid_SD", index=False)

Banyaknya Guru Sekolah Menengah Kejuruan Provinsi Bali Menurut Kabupaten/Kota (271)

In [25]:
data_271 = getDataByVarId (domain='5100',varId='271')
data_271

,verval_val,vervar_label,var_val,turvar_val,turvar_label,tahun_val,tahun_label,turtahun_val,turtahun_label,id,content_val,content_label
0,1,Kab. Jembrana,271,0,Tidak ada,123,2023,0,Tahun,127101230,127101230,311
1,2,Kab. Tabanan,271,0,Tidak ada,123,2023,0,Tahun,227101230,227101230,431
2,3,Kab. Badung,271,0,Tidak ada,123,2023,0,Tahun,327101230,327101230,733
3,4,Kab. Gianyar,271,0,Tidak ada,123,2023,0,Tahun,427101230,427101230,908
4,5,Kab. Klungkung,271,0,Tidak ada,123,2023,0,Tahun,527101230,527101230,236
...,...,...,...,...,...,...,...,...,...,...,...,...
5,6,Kab. Bangli,271,0,Tidak ada,114,2014,0,Tahun,627101140,627101140,482
6,7,Kab. Karangasem,271,0,Tidak ada,114,2014,0,Tahun,727101140,727101140,391
7,8,Kab. Buleleng,271,0,Tidak ada,114,2014,0,Tahun,827101140,827101140,1225
8,9,Kota Denpasar,271,0,Tidak ada,114,2014,0,Tahun,927101140,927101140,1507


In [26]:
data_271 = data_271[['verval_val','vervar_label','tahun_label','content_label']].copy()
data_271["vervar_label"] = data_271["vervar_label"].str.replace(r"<.*?>", "", regex=True)

data_tb = data_271.rename(columns={
    'vervar_label'   : 'Kabupaten/Kota',
    'tahun_label'    : 'Tahun',
    'content_label'  : 'Nilai'
})

api_order = data_271['vervar_label'].drop_duplicates().tolist()
data_tb['Kabupaten/Kota'] = pd.Categorical(data_tb['Kabupaten/Kota'], categories=api_order, ordered=True)

data_tb = data_tb.pivot_table(
    index=['Kabupaten/Kota'],
    columns=['Tahun'],
    values='Nilai'
)

data_tb.columns = [
    '_'.join([str(c) for c in col if c]).strip() if isinstance(col, tuple) else str(col) 
    for col in data_tb.columns.values
]

data_tb = data_tb.reset_index()

data_tb = data_tb.applymap(
    lambda x: int(x) if isinstance(x, (int, float)) and x.is_integer() else x
)

data_tb


C:\Users\user\AppData\Local\Temp\ipykernel_2568\3971766139.py:13: FutureWarning: The default value of observed=False is deprecated and will change to observed=True in a future version of pandas. Specify observed=False to silence this warning and retain the current behavior
  data_tb = data_tb.pivot_table(
C:\Users\user\AppData\Local\Temp\ipykernel_2568\3971766139.py:26: FutureWarning: DataFrame.applymap has been deprecated. Use DataFrame.map instead.
  data_tb = data_tb.applymap(


,Kabupaten/Kota,2014,2015,2016,2017,2018,2019,2020,2021,2022,2023
0,Kab. Jembrana,354,362,242,328,331,339,359,335,331,311
1,Kab. Tabanan,588,644,367,448,471,467,484,470,451,431
2,Kab. Badung,973,926,440,750,767,787,843,806,757,733
3,Kab. Gianyar,1439,1390,659,959,955,981,1031,940,963,908
4,Kab. Klungkung,226,317,203,247,219,217,224,224,235,236
5,Kab. Bangli,482,483,253,446,431,465,464,473,509,482
6,Kab. Karangasem,391,408,234,380,388,391,398,394,425,422
7,Kab. Buleleng,1225,1142,571,913,926,987,1021,1007,1023,1027
8,Kota Denpasar,1507,1528,712,1111,1114,1123,1233,1212,1187,1177
9,Provinsi Bali,7185,7200,3681,5582,5602,5757,6057,5861,5881,5727


In [27]:
data_271.to_excel("data_271.xlsx", sheet_name= "Banyak_Guru_SMK", index=False)

Banyaknya Guru Sekolah Menengah Atas/Sederajat Provinsi Bali Menurut Kabupaten/Kota (270)

In [28]:
data_270 = getDataByVarId (domain='5100',varId='270')
data_270

,verval_val,vervar_label,var_val,turvar_val,turvar_label,tahun_val,tahun_label,turtahun_val,turtahun_label,id,content_val,content_label
0,1,Kab. Jembrana,270,0,Tidak ada,123,2023,0,Tahun,127001230,127001230,649
1,2,Kab. Tabanan,270,0,Tidak ada,123,2023,0,Tahun,227001230,227001230,594
2,3,Kab. Badung,270,0,Tidak ada,123,2023,0,Tahun,327001230,327001230,999
3,4,Kab. Gianyar,270,0,Tidak ada,123,2023,0,Tahun,427001230,427001230,540
4,5,Kab. Klungkung,270,0,Tidak ada,123,2023,0,Tahun,527001230,527001230,465
...,...,...,...,...,...,...,...,...,...,...,...,...
5,6,Kab. Bangli,270,0,Tidak ada,114,2014,0,Tahun,627001140,627001140,310
6,7,Kab. Karangasem,270,0,Tidak ada,114,2014,0,Tahun,727001140,727001140,737
7,8,Kab. Buleleng,270,0,Tidak ada,114,2014,0,Tahun,827001140,827001140,1305
8,9,Kota Denpasar,270,0,Tidak ada,114,2014,0,Tahun,927001140,927001140,1258


In [29]:
data_270 = data_270[['verval_val','vervar_label','tahun_label','content_label']].copy()
data_270["vervar_label"] = data_270["vervar_label"].str.replace(r"<.*?>", "", regex=True)

data_tb = data_270.rename(columns={
    'vervar_label'   : 'Kabupaten/Kota',
    'tahun_label'    : 'Tahun',
    'content_label'  : 'Nilai'
})

api_order = data_270['vervar_label'].drop_duplicates().tolist()
data_tb['Kabupaten/Kota'] = pd.Categorical(data_tb['Kabupaten/Kota'], categories=api_order, ordered=True)

data_tb = data_tb.pivot_table(
    index=['Kabupaten/Kota'],
    columns=['Tahun'],
    values='Nilai'
)

data_tb.columns = [
    '_'.join([str(c) for c in col if c]).strip() if isinstance(col, tuple) else str(col) 
    for col in data_tb.columns.values
]

data_tb = data_tb.reset_index()

data_tb = data_tb.applymap(
    lambda x: int(x) if isinstance(x, (int, float)) and x.is_integer() else x
)

data_tb


C:\Users\user\AppData\Local\Temp\ipykernel_2568\205252474.py:13: FutureWarning: The default value of observed=False is deprecated and will change to observed=True in a future version of pandas. Specify observed=False to silence this warning and retain the current behavior
  data_tb = data_tb.pivot_table(
C:\Users\user\AppData\Local\Temp\ipykernel_2568\205252474.py:26: FutureWarning: DataFrame.applymap has been deprecated. Use DataFrame.map instead.
  data_tb = data_tb.applymap(


,Kabupaten/Kota,2014,2015,2016,2017,2018,2019,2020,2021,2022,2023
0,Kab. Jembrana,536,520,379,488,510,504,560,616,641,649
1,Kab. Tabanan,973,997,669,727,719,672,640,583,585,594
2,Kab. Badung,815,851,568,723,756,773,866,834,942,999
3,Kab. Gianyar,600,610,453,517,514,496,482,463,503,540
4,Kab. Klungkung,440,435,317,437,442,467,481,462,472,465
5,Kab. Bangli,310,331,254,307,301,287,295,289,288,287
6,Kab. Karangasem,737,695,533,570,586,592,614,594,619,610
7,Kab. Buleleng,1305,1356,911,1094,1120,1143,1172,1128,1168,1147
8,Kota Denpasar,1258,1356,885,1036,1067,1084,1187,1135,1144,1190
9,Provinsi Bali,6974,7151,4969,5899,6015,6018,6297,6104,6362,6481


In [30]:
data_270.to_excel("data_270.xlsx", sheet_name= "Banyak_Guru_SMA", index=False)

Banyaknya Guru Sekolah Lanjutan Pertama/Sederajat Provinsi Bali Menurut Kabupaten/Kota (269)

In [31]:
data_269 = getDataByVarId (domain='5100',varId='269')
data_269

,verval_val,vervar_label,var_val,turvar_val,turvar_label,tahun_val,tahun_label,turtahun_val,turtahun_label,id,content_val,content_label
0,1,Kab. Jembrana,269,0,Tidak ada,123,2023,0,Tahun,126901230,126901230,1020
1,2,Kab. Tabanan,269,0,Tidak ada,123,2023,0,Tahun,226901230,226901230,1240
2,3,Kab. Badung,269,0,Tidak ada,123,2023,0,Tahun,326901230,326901230,1833
3,4,Kab. Gianyar,269,0,Tidak ada,123,2023,0,Tahun,426901230,426901230,1171
4,5,Kab. Klungkung,269,0,Tidak ada,123,2023,0,Tahun,526901230,526901230,632
...,...,...,...,...,...,...,...,...,...,...,...,...
5,6,Kab. Bangli,269,0,Tidak ada,114,2014,0,Tahun,626901140,626901140,881
6,7,Kab. Karangasem,269,0,Tidak ada,114,2014,0,Tahun,726901140,726901140,1482
7,8,Kab. Buleleng,269,0,Tidak ada,114,2014,0,Tahun,826901140,826901140,2197
8,9,Kota Denpasar,269,0,Tidak ada,114,2014,0,Tahun,926901140,926901140,2208


In [32]:
data_269 = data_269[['verval_val','vervar_label','tahun_label','content_label']].copy()
data_269["vervar_label"] = data_269["vervar_label"].str.replace(r"<.*?>", "", regex=True)

data_tb = data_269.rename(columns={
    'vervar_label'   : 'Kabupaten/Kota',
    'tahun_label'    : 'Tahun',
    'content_label'  : 'Nilai'
})

api_order = data_269['vervar_label'].drop_duplicates().tolist()
data_tb['Kabupaten/Kota'] = pd.Categorical(data_tb['Kabupaten/Kota'], categories=api_order, ordered=True)

data_tb = data_tb.pivot_table(
    index=['Kabupaten/Kota'],
    columns=['Tahun'],
    values='Nilai'
)

data_tb.columns = [
    '_'.join([str(c) for c in col if c]).strip() if isinstance(col, tuple) else str(col) 
    for col in data_tb.columns.values
]

data_tb = data_tb.reset_index()

data_tb = data_tb.applymap(
    lambda x: int(x) if isinstance(x, (int, float)) and x.is_integer() else x
)

data_tb


C:\Users\user\AppData\Local\Temp\ipykernel_2568\2676141297.py:13: FutureWarning: The default value of observed=False is deprecated and will change to observed=True in a future version of pandas. Specify observed=False to silence this warning and retain the current behavior
  data_tb = data_tb.pivot_table(
C:\Users\user\AppData\Local\Temp\ipykernel_2568\2676141297.py:26: FutureWarning: DataFrame.applymap has been deprecated. Use DataFrame.map instead.
  data_tb = data_tb.applymap(


,Kabupaten/Kota,2014,2015,2016,2017,2018,2019,2020,2021,2022,2023
0,Kab. Jembrana,860,885,890,865,891,875,883,987,1002,1020
1,Kab. Tabanan,1752,1735,1707,1660,1655,1588,1616,1509,1298,1240
2,Kab. Badung,1824,1912,1810,1768,1827,1872,1947,1906,1840,1833
3,Kab. Gianyar,1593,1599,1568,1383,1334,1265,1292,1245,1238,1171
4,Kab. Klungkung,718,747,761,735,747,717,721,682,641,632
5,Kab. Bangli,881,895,904,876,871,850,852,858,889,880
6,Kab. Karangasem,1482,1395,1471,1202,1214,1227,1254,1183,1166,1137
7,Kab. Buleleng,2197,2124,2143,2028,2061,2104,2141,2153,2172,2201
8,Kota Denpasar,2208,2296,2133,1770,1771,1873,1990,1965,1899,1994
9,Provinsi Bali,13515,13588,13387,12287,12371,12371,12696,12488,12145,12108


In [33]:
data_269.to_excel("data_269.xlsx", sheet_name= "Banyak_Guru_SLP", index=False)

Banyaknya Guru Sekolah Dasar/Sederajat Provinsi Bali Menurut Kabupaten/Kota (268)

In [ ]:
# data_268 = getDataByVarId (domain='5100',varId='268')
# data_268

,verval_val,vervar_label,var_val,turvar_val,turvar_label,tahun_val,tahun_label,turtahun_val,turtahun_label,id,content_val,content_label
0,1,Kab. Jembrana,268,0,Tidak ada,123,2023,0,Tahun,126801230,126801230,2031
1,2,Kab. Tabanan,268,0,Tidak ada,123,2023,0,Tahun,226801230,226801230,2626
2,3,Kab. Badung,268,0,Tidak ada,123,2023,0,Tahun,326801230,326801230,4172
3,4,Kab. Gianyar,268,0,Tidak ada,123,2023,0,Tahun,426801230,426801230,2920
4,5,Kab. Klungkung,268,0,Tidak ada,123,2023,0,Tahun,526801230,526801230,1297
...,...,...,...,...,...,...,...,...,...,...,...,...
5,6,Kab. Bangli,268,0,Tidak ada,114,2014,0,Tahun,626801140,626801140,1550
6,7,Kab. Karangasem,268,0,Tidak ada,114,2014,0,Tahun,726801140,726801140,3759
7,8,Kab. Buleleng,268,0,Tidak ada,114,2014,0,Tahun,826801140,826801140,4817
8,9,Kota Denpasar,268,0,Tidak ada,114,2014,0,Tahun,926801140,926801140,4101


In [ ]:
# data_268 = data_268[['verval_val','vervar_label','tahun_label','content_label']].copy()
# data_268["vervar_label"] = data_268["vervar_label"].str.replace(r"<.*?>", "", regex=True)

# data_tb = data_268.rename(columns={
#     'vervar_label'   : 'Kabupaten/Kota',
#     'tahun_label'    : 'Tahun',
#     'content_label'  : 'Nilai'
# })

# api_order = data_268['vervar_label'].drop_duplicates().tolist()
# data_tb['Kabupaten/Kota'] = pd.Categorical(data_tb['Kabupaten/Kota'], categories=api_order, ordered=True)

# data_tb = data_tb.pivot_table(
#     index=['Kabupaten/Kota'],
#     columns=['Tahun'],
#     values='Nilai'
# )

# data_tb.columns = [
#     '_'.join([str(c) for c in col if c]).strip() if isinstance(col, tuple) else str(col) 
#     for col in data_tb.columns.values
# ]

# data_tb = data_tb.reset_index()

# data_tb = data_tb.applymap(
#     lambda x: int(x) if isinstance(x, (int, float)) and x.is_integer() else x
# )

# data_tb


C:\Users\user\AppData\Local\Temp\ipykernel_2568\506169200.py:13: FutureWarning: The default value of observed=False is deprecated and will change to observed=True in a future version of pandas. Specify observed=False to silence this warning and retain the current behavior
  data_tb = data_tb.pivot_table(
C:\Users\user\AppData\Local\Temp\ipykernel_2568\506169200.py:26: FutureWarning: DataFrame.applymap has been deprecated. Use DataFrame.map instead.
  data_tb = data_tb.applymap(


,Kabupaten/Kota,2014,2015,2016,2017,2018,2019,2020,2021,2022,2023
0,Kab. Jembrana,1534,1617,1625,1623,1781,1605,1871,1893,1995,2031
1,Kab. Tabanan,3413,3401,3370,2910,3033,2824,2993,2700,2708,2626
2,Kab. Badung,3718,3727,3700,3661,4134,3821,4277,4145,4146,4172
3,Kab. Gianyar,3211,3197,3275,2885,3091,2801,3148,2960,2967,2920
4,Kab. Klungkung,1551,1515,1509,1369,1456,1367,1460,1353,1328,1297
5,Kab. Bangli,1550,1586,1581,1462,1545,1444,1595,1553,1637,1647
6,Kab. Karangasem,3759,3702,3665,3306,3577,3256,3590,3385,3340,3146
7,Kab. Buleleng,4817,4751,4696,4261,4660,4265,4776,4743,4703,4675
8,Kota Denpasar,4101,4170,4234,3863,4438,4141,4604,4459,4431,4468
9,Provinsi Bali,27654,27666,27655,25340,27715,25524,28314,27191,27255,26982


In [ ]:
# data_268.to_excel("data_268.xlsx", sheet_name= "Banyak_Guru_SD", index=False)

Persentase Penduduk Usia 15 Tahun ke Atas Menurut Pendidikan Tertinggi yang Ditamatkan dan Kabupaten/Kota di Bali (267)

In [ ]:
# data_267 = getDataByVarId (domain='5100',varId='267')
# data_267

,verval_val,vervar_label,var_val,turvar_val,turvar_label,tahun_val,tahun_label,turtahun_val,turtahun_label,id,content_val,content_label
0,1,Kab. Jembrana,267,166,Tidak Punya,123,2023,0,Tahun,12671661230,12671661230,13.75
1,1,Kab. Jembrana,267,167,SD Sederajat,123,2023,0,Tahun,12671671230,12671671230,24.35
2,1,Kab. Jembrana,267,168,SLTP Sederajat,123,2023,0,Tahun,12671681230,12671681230,23.42
3,1,Kab. Jembrana,267,169,SLTA Sederajat,123,2023,0,Tahun,12671691230,12671691230,28.54
4,1,Kab. Jembrana,267,170,DI / DII / DIII,123,2023,0,Tahun,12671701230,12671701230,3.10
...,...,...,...,...,...,...,...,...,...,...,...,...
51,9,Kota Denpasar,267,169,SLTA Sederajat,114,2014,0,Tahun,92671691140,92671691140,42.17
54,10,<b>Provinsi Bali</b>,267,166,Tidak Punya,114,2014,0,Tahun,102671661140,102671661140,19.65
55,10,<b>Provinsi Bali</b>,267,167,SD Sederajat,114,2014,0,Tahun,102671671140,102671671140,21.66
56,10,<b>Provinsi Bali</b>,267,168,SLTP Sederajat,114,2014,0,Tahun,102671681140,102671681140,17.54


In [ ]:

# data_267 = data_267[['verval_val','vervar_label','turvar_label','tahun_label','content_label']].copy()
# data_267["vervar_label"] = data_267["vervar_label"].str.replace(r"<.*?>", "", regex=True)


# data_tb = data_267.rename(columns={
#     'vervar_label'   : 'Kabupaten/Kota',
#     'tahun_label'    : 'Tahun',
#     'turvar_label'   : 'Jenjang Pendidikan',
#     'content_label'  : 'Nilai'
# })

# api_order = data_267['vervar_label'].drop_duplicates().tolist()
# data_tb['Kabupaten/Kota'] = pd.Categorical(data_tb['Kabupaten/Kota'], categories=api_order, ordered=True)

# data_tb = data_tb.pivot_table(
#     index=['Kabupaten/Kota'],
#     columns=['Tahun', 'Jenjang Pendidikan'],
#     values='Nilai'
# )

# data_tb.columns = ['_'.join([str(c) for c in col if c]).strip() if isinstance(col, tuple) else str(col) 
#                    for col in data_tb.columns.values]

# data_tb = data_tb.reset_index()

# data_tb


C:\Users\user\AppData\Local\Temp\ipykernel_2772\2265396899.py:15: FutureWarning: The default value of observed=False is deprecated and will change to observed=True in a future version of pandas. Specify observed=False to silence this warning and retain the current behavior
  data_tb = data_tb.pivot_table(


,Kabupaten/Kota,2014_SD Sederajat,2014_SLTA Sederajat,2014_SLTP Sederajat,2014_Tidak Punya,2015_DI / DII / DIII,2015_DIV / S1 / S2 / S3,2015_SD Sederajat,2015_SLTA Sederajat,2015_SLTP Sederajat,...,2022_SD Sederajat,2022_SLTA Sederajat,2022_SLTP Sederajat,2022_Tidak Punya,2023_DI / DII / DIII,2023_DIV / S1 / S2 / S3,2023_SD Sederajat,2023_SLTA Sederajat,2023_SLTP Sederajat,2023_Tidak Punya
0,Kab. Jembrana,33.08,24.41,18.82,18.02,1.58,5.50,31.37,27.04,17.61,...,23.60,29.73,21.70,13.36,3.10,6.83,24.35,28.54,23.42,13.75
1,Kab. Tabanan,22.93,29.45,19.07,18.88,3.71,6.50,26.60,29.19,17.53,...,25.80,31.09,18.21,10.07,4.16,10.49,25.21,30.73,19.56,9.85
2,Kab. Badung,18.77,38.88,15.71,14.12,4.90,9.56,17.95,39.76,15.57,...,14.45,38.10,18.74,7.52,7.40,14.86,13.10,39.11,20.29,5.24
3,Kab. Gianyar,18.42,35.16,18.09,19.87,2.95,8.05,22.14,34.06,16.15,...,17.91,35.28,18.56,11.63,3.92,12.76,16.81,37.83,17.83,10.85
4,Kab. Klungkung,22.52,22.06,16.61,31.50,2.19,5.48,20.63,25.48,16.81,...,19.34,31.84,18.84,16.36,2.93,9.77,19.72,29.59,19.11,18.87
5,Kab. Bangli,36.49,17.28,16.75,24.28,2.07,4.75,29.80,19.17,20.66,...,27.59,22.05,20.81,20.06,1.52,7.99,26.63,21.89,19.86,22.11
6,Kab. Karangasem,25.39,16.77,16.94,37.07,1.08,3.88,27.03,14.76,16.08,...,25.79,21.99,19.41,25.00,1.78,4.17,25.70,19.31,20.09,28.95
7,Kab. Buleleng,23.73,22.18,19.01,30.12,1.39,4.21,28.21,22.33,15.74,...,27.05,27.09,22.35,16.82,0.63,4.34,28.58,23.89,24.17,18.39
8,Kota Denpasar,14.24,42.17,16.92,5.17,6.83,13.03,14.61,41.54,17.91,...,15.81,39.97,18.57,2.58,4.46,16.88,14.14,41.02,21.03,2.47
9,Provinsi Bali,21.66,30.60,17.54,19.65,3.56,7.73,22.73,30.73,16.88,...,20.67,32.77,19.51,11.50,3.75,11.05,20.09,32.57,20.75,11.78


In [ ]:
# # --- urutan jenjang yang diinginkan ---
# urutan_jenjang = [
#     "Tidak Punya",
#     "SD Sederajat",
#     "SLTP Sederajat",
#     "SLTA Sederajat",
#     "DI / DII / DIII",
#     "DIV / S1 / S2 / S3"
# ]

# # ambil daftar tahun unik
# tahun_list = sorted({c.split("_")[0] for c in data_tb.columns if c != "Kabupaten/Kota"})

# # buat urutan kolom per tahun sesuai jenjang
# kolom_urut = []
# for th in tahun_list:
#     for jenjang in urutan_jenjang:
#         kolom_urut.append(f"{th}_{jenjang}")

# # pastikan kolom ada di data
# kolom_urut = [c for c in kolom_urut if c in data_tb.columns]

# # susun ulang dataframe
# data_tb = data_tb[["Kabupaten/Kota"] + kolom_urut]
# data_tb

,Kabupaten/Kota,2014_Tidak Punya,2014_SD Sederajat,2014_SLTP Sederajat,2014_SLTA Sederajat,2015_Tidak Punya,2015_SD Sederajat,2015_SLTP Sederajat,2015_SLTA Sederajat,2015_DI / DII / DIII,...,2022_SLTP Sederajat,2022_SLTA Sederajat,2022_DI / DII / DIII,2022_DIV / S1 / S2 / S3,2023_Tidak Punya,2023_SD Sederajat,2023_SLTP Sederajat,2023_SLTA Sederajat,2023_DI / DII / DIII,2023_DIV / S1 / S2 / S3
0,Kab. Jembrana,18.02,33.08,18.82,24.41,16.91,31.37,17.61,27.04,1.58,...,21.70,29.73,4.29,7.32,13.75,24.35,23.42,28.54,3.10,6.83
1,Kab. Tabanan,18.88,22.93,19.07,29.45,16.46,26.60,17.53,29.19,3.71,...,18.21,31.09,4.81,10.04,9.85,25.21,19.56,30.73,4.16,10.49
2,Kab. Badung,14.12,18.77,15.71,38.88,12.26,17.95,15.57,39.76,4.90,...,18.74,38.10,7.25,13.94,5.24,13.10,20.29,39.11,7.40,14.86
3,Kab. Gianyar,19.87,18.42,18.09,35.16,16.65,22.14,16.15,34.06,2.95,...,18.56,35.28,4.26,12.36,10.85,16.81,17.83,37.83,3.92,12.76
4,Kab. Klungkung,31.50,22.52,16.61,22.06,29.41,20.63,16.81,25.48,2.19,...,18.84,31.84,3.86,9.75,18.87,19.72,19.11,29.59,2.93,9.77
5,Kab. Bangli,24.28,36.49,16.75,17.28,23.56,29.80,20.66,19.17,2.07,...,20.81,22.05,1.18,8.32,22.11,26.63,19.86,21.89,1.52,7.99
6,Kab. Karangasem,37.07,25.39,16.94,16.77,37.16,27.03,16.08,14.76,1.08,...,19.41,21.99,2.55,5.26,28.95,25.70,20.09,19.31,1.78,4.17
7,Kab. Buleleng,30.12,23.73,19.01,22.18,28.11,28.21,15.74,22.33,1.39,...,22.35,27.09,1.47,5.22,18.39,28.58,24.17,23.89,0.63,4.34
8,Kota Denpasar,5.17,14.24,16.92,42.17,6.08,14.61,17.91,41.54,6.83,...,18.57,39.97,5.36,17.72,2.47,14.14,21.03,41.02,4.46,16.88
9,Provinsi Bali,19.65,21.66,17.54,30.60,18.37,22.73,16.88,30.73,3.56,...,19.51,32.77,4.31,11.25,11.78,20.09,20.75,32.57,3.75,11.05


In [ ]:
# import re

# # --- urutan jenjang yang diinginkan ---
# urutan_jenjang = [
#     "Tidak Punya",
#     "SD Sederajat",
#     "SLTP Sederajat",
#     "SLTA Sederajat",
#     "DI / DII / DIII",
#     "DIV / S1 / S2 / S3"
# ]

# # ambil tahun unik pakai regex
# tahun_list = sorted({
#     re.match(r"^(\d{4})", c).group(1)
#     for c in data_tb.columns if c != "Kabupaten/Kota" and re.match(r"^(\d{4})", c)
# })

# # buat urutan kolom per tahun sesuai jenjang
# kolom_urut = []
# for th in tahun_list:
#     for jenjang in urutan_jenjang:
#         kolom_urut.append(f"{th}_{jenjang}")

# # pastikan kolom ada di data
# kolom_urut = [c for c in kolom_urut if c in data_tb.columns]

# # susun ulang dataframe
# data_tb = data_tb[["Kabupaten/Kota"] + kolom_urut]

# data_tb


,Kabupaten/Kota,2014_Tidak Punya,2014_SD Sederajat,2014_SLTP Sederajat,2014_SLTA Sederajat,2015_Tidak Punya,2015_SD Sederajat,2015_SLTP Sederajat,2015_SLTA Sederajat,2015_DI / DII / DIII,...,2022_SLTP Sederajat,2022_SLTA Sederajat,2022_DI / DII / DIII,2022_DIV / S1 / S2 / S3,2023_Tidak Punya,2023_SD Sederajat,2023_SLTP Sederajat,2023_SLTA Sederajat,2023_DI / DII / DIII,2023_DIV / S1 / S2 / S3
0,Kab. Jembrana,18.02,33.08,18.82,24.41,16.91,31.37,17.61,27.04,1.58,...,21.70,29.73,4.29,7.32,13.75,24.35,23.42,28.54,3.10,6.83
1,Kab. Tabanan,18.88,22.93,19.07,29.45,16.46,26.60,17.53,29.19,3.71,...,18.21,31.09,4.81,10.04,9.85,25.21,19.56,30.73,4.16,10.49
2,Kab. Badung,14.12,18.77,15.71,38.88,12.26,17.95,15.57,39.76,4.90,...,18.74,38.10,7.25,13.94,5.24,13.10,20.29,39.11,7.40,14.86
3,Kab. Gianyar,19.87,18.42,18.09,35.16,16.65,22.14,16.15,34.06,2.95,...,18.56,35.28,4.26,12.36,10.85,16.81,17.83,37.83,3.92,12.76
4,Kab. Klungkung,31.50,22.52,16.61,22.06,29.41,20.63,16.81,25.48,2.19,...,18.84,31.84,3.86,9.75,18.87,19.72,19.11,29.59,2.93,9.77
5,Kab. Bangli,24.28,36.49,16.75,17.28,23.56,29.80,20.66,19.17,2.07,...,20.81,22.05,1.18,8.32,22.11,26.63,19.86,21.89,1.52,7.99
6,Kab. Karangasem,37.07,25.39,16.94,16.77,37.16,27.03,16.08,14.76,1.08,...,19.41,21.99,2.55,5.26,28.95,25.70,20.09,19.31,1.78,4.17
7,Kab. Buleleng,30.12,23.73,19.01,22.18,28.11,28.21,15.74,22.33,1.39,...,22.35,27.09,1.47,5.22,18.39,28.58,24.17,23.89,0.63,4.34
8,Kota Denpasar,5.17,14.24,16.92,42.17,6.08,14.61,17.91,41.54,6.83,...,18.57,39.97,5.36,17.72,2.47,14.14,21.03,41.02,4.46,16.88
9,Provinsi Bali,19.65,21.66,17.54,30.60,18.37,22.73,16.88,30.73,3.56,...,19.51,32.77,4.31,11.25,11.78,20.09,20.75,32.57,3.75,11.05


In [ ]:
# data_267.to_excel("data_267.xlsx", sheet_name= "Pendidikan_Tertinggi", index=False)

Banyaknya Murid Taman Kanak-kanak/Sederajat Provinsi Bali Menurut Kabupaten/Kota (88)

In [21]:
data_88 = getDataByVarId (domain='5100',varId='88')
data_88

,verval_val,vervar_label,var_val,turvar_val,turvar_label,tahun_val,tahun_label,turtahun_val,turtahun_label,id,content_val,content_label
0,1,Kab. Jembrana,88,0,Tidak ada,123,2023,0,Tahun,18801230,18801230,5750
1,2,Kab. Tabanan,88,0,Tidak ada,123,2023,0,Tahun,28801230,28801230,7567
2,3,Kab. Badung,88,0,Tidak ada,123,2023,0,Tahun,38801230,38801230,12210
3,4,Kab. Gianyar,88,0,Tidak ada,123,2023,0,Tahun,48801230,48801230,8864
4,5,Kab. Klungkung,88,0,Tidak ada,123,2023,0,Tahun,58801230,58801230,4065
...,...,...,...,...,...,...,...,...,...,...,...,...
5,6,Kab. Bangli,88,0,Tidak ada,114,2014,0,Tahun,68801140,68801140,4670
6,7,Kab. Karangasem,88,0,Tidak ada,114,2014,0,Tahun,78801140,78801140,6678
7,8,Kab. Buleleng,88,0,Tidak ada,114,2014,0,Tahun,88801140,88801140,10572
8,9,Kota Denpasar,88,0,Tidak ada,114,2014,0,Tahun,98801140,98801140,22148


In [22]:
data_88 = data_88[['verval_val','vervar_label','tahun_label','content_label']].copy()
data_88["vervar_label"] = data_88["vervar_label"].str.replace(r"<.*?>", "", regex=True)

data_tb = data_88.rename(columns={
    'vervar_label'   : 'Kabupaten/Kota',
    'tahun_label'    : 'Tahun',
    'content_label'  : 'Nilai'
})

api_order = data_88['vervar_label'].drop_duplicates().tolist()
data_tb['Kabupaten/Kota'] = pd.Categorical(data_tb['Kabupaten/Kota'], categories=api_order, ordered=True)

data_tb = data_tb.pivot_table(
    index=['Kabupaten/Kota'],
    columns=['Tahun'],
    values='Nilai'
)

data_tb.columns = [
    '_'.join([str(c) for c in col if c]).strip() if isinstance(col, tuple) else str(col) 
    for col in data_tb.columns.values
]

data_tb = data_tb.reset_index()

data_tb = data_tb.applymap(
    lambda x: int(x) if isinstance(x, (int, float)) and x.is_integer() else x
)

data_tb


C:\Users\user\AppData\Local\Temp\ipykernel_2772\1696740151.py:13: FutureWarning: The default value of observed=False is deprecated and will change to observed=True in a future version of pandas. Specify observed=False to silence this warning and retain the current behavior
  data_tb = data_tb.pivot_table(
C:\Users\user\AppData\Local\Temp\ipykernel_2772\1696740151.py:26: FutureWarning: DataFrame.applymap has been deprecated. Use DataFrame.map instead.
  data_tb = data_tb.applymap(


,Kabupaten/Kota,2014,2015,2016,2017,2018,2019,2020,2021,2022,2023
0,Kab. Jembrana,6205,5240,5138,3215,14480,5664,5790,5420,5747,5750
1,Kab. Tabanan,12498,6972,5944,7763,3315,7337,7005,6840,7586,7567
2,Kab. Badung,14586,11694,12143,13459,10697,13403,11427,10135,12216,12210
3,Kab. Gianyar,7579,5468,8188,9584,9192,8863,8283,7992,8852,8864
4,Kab. Klungkung,4728,3534,4244,4618,4397,4376,4259,3807,4065,4065
5,Kab. Bangli,4670,2457,2264,2821,5047,2984,2782,2600,2907,2910
6,Kab. Karangasem,6678,4223,5669,5831,4760,5200,4769,4388,5109,5124
7,Kab. Buleleng,10572,8349,16112,10812,8279,11328,11276,9798,11481,11461
8,Kota Denpasar,22148,18555,17920,21768,21120,19954,17166,14132,17863,18018
9,Provinsi Bali,89664,66492,77622,79871,81287,79109,72757,65112,75826,75969


In [23]:
data_88.to_excel("data_88.xlsx", sheet_name= "Banyak_Murid_TK", index=False)

Banyaknya Guru Taman Kanak-kanak/Sederajat Provinsi Bali Menurut Kabupaten/Kota (87)

In [24]:
data_87 = getDataByVarId (domain='5100',varId='87')
data_87

,verval_val,vervar_label,var_val,turvar_val,turvar_label,tahun_val,tahun_label,turtahun_val,turtahun_label,id,content_val,content_label
0,1,Kab. Jembrana,87,0,Tidak ada,123,2023,0,Tahun,18701230,18701230,517
1,2,Kab. Tabanan,87,0,Tidak ada,123,2023,0,Tahun,28701230,28701230,611
2,3,Kab. Badung,87,0,Tidak ada,123,2023,0,Tahun,38701230,38701230,1137
3,4,Kab. Gianyar,87,0,Tidak ada,123,2023,0,Tahun,48701230,48701230,859
4,5,Kab. Klungkung,87,0,Tidak ada,123,2023,0,Tahun,58701230,58701230,498
...,...,...,...,...,...,...,...,...,...,...,...,...
5,6,Kab. Bangli,87,0,Tidak ada,114,2014,0,Tahun,68701140,68701140,549
6,7,Kab. Karangasem,87,0,Tidak ada,114,2014,0,Tahun,78701140,78701140,630
7,8,Kab. Buleleng,87,0,Tidak ada,114,2014,0,Tahun,88701140,88701140,1044
8,9,Kota Denpasar,87,0,Tidak ada,114,2014,0,Tahun,98701140,98701140,2209


In [25]:
data_87 = data_87[['verval_val','vervar_label','tahun_label','content_label']].copy()
data_87["vervar_label"] = data_87["vervar_label"].str.replace(r"<.*?>", "", regex=True)

data_tb = data_87.rename(columns={
    'vervar_label'   : 'Kabupaten/Kota',
    'tahun_label'    : 'Tahun',
    'content_label'  : 'Nilai'
})

api_order = data_87['vervar_label'].drop_duplicates().tolist()
data_tb['Kabupaten/Kota'] = pd.Categorical(data_tb['Kabupaten/Kota'], categories=api_order, ordered=True)

data_tb = data_tb.pivot_table(
    index=['Kabupaten/Kota'],
    columns=['Tahun'],
    values='Nilai'
)

data_tb.columns = [
    '_'.join([str(c) for c in col if c]).strip() if isinstance(col, tuple) else str(col) 
    for col in data_tb.columns.values
]

data_tb = data_tb.reset_index()

data_tb = data_tb.applymap(
    lambda x: int(x) if isinstance(x, (int, float)) and x.is_integer() else x
)

data_tb


C:\Users\user\AppData\Local\Temp\ipykernel_2772\2091465574.py:13: FutureWarning: The default value of observed=False is deprecated and will change to observed=True in a future version of pandas. Specify observed=False to silence this warning and retain the current behavior
  data_tb = data_tb.pivot_table(
C:\Users\user\AppData\Local\Temp\ipykernel_2772\2091465574.py:26: FutureWarning: DataFrame.applymap has been deprecated. Use DataFrame.map instead.
  data_tb = data_tb.applymap(


,Kabupaten/Kota,2014,2015,2016,2017,2018,2019,2020,2021,2022,2023
0,Kab. Jembrana,1522,2443,432,493,926,524,536,515,513,517
1,Kab. Tabanan,1446,1743,579,474,255,634,609,561,609,611
2,Kab. Badung,1562,1500,1057,1113,645,1129,1155,996,1134,1137
3,Kab. Gianyar,750,810,678,773,663,793,781,793,859,859
4,Kab. Klungkung,441,687,473,491,295,463,471,447,496,498
5,Kab. Bangli,549,609,266,200,313,357,375,318,392,393
6,Kab. Karangasem,630,725,772,494,363,443,415,430,467,467
7,Kab. Buleleng,1044,1090,737,916,563,909,946,915,951,955
8,Kota Denpasar,2209,2409,1290,1631,1347,1635,1670,1569,1618,1624
9,Provinsi Bali,10153,12016,6284,6585,5370,6887,6958,6544,7039,7061


In [26]:
data_87.to_excel("data_87.xlsx", sheet_name= "Banyak_Guru_TK", index=False)

Banyaknya Sekolah Taman Kanak-kanak/Sederajat Provinsi Bali Menurut Kabupaten/Kota (85)

In [27]:
data_85 = getDataByVarId (domain='5100',varId='85')
data_85

,verval_val,vervar_label,var_val,turvar_val,turvar_label,tahun_val,tahun_label,turtahun_val,turtahun_label,id,content_val,content_label
0,1,Kab. Jembrana,85,0,Tidak ada,123,2023,0,Tahun,18501230,18501230,157
1,2,Kab. Tabanan,85,0,Tidak ada,123,2023,0,Tahun,28501230,28501230,228
2,3,Kab. Badung,85,0,Tidak ada,123,2023,0,Tahun,38501230,38501230,240
3,4,Kab. Gianyar,85,0,Tidak ada,123,2023,0,Tahun,48501230,48501230,161
4,5,Kab. Klungkung,85,0,Tidak ada,123,2023,0,Tahun,58501230,58501230,102
...,...,...,...,...,...,...,...,...,...,...,...,...
5,6,Kab. Bangli,85,0,Tidak ada,114,2014,0,Tahun,68501140,68501140,67
6,7,Kab. Karangasem,85,0,Tidak ada,114,2014,0,Tahun,78501140,78501140,116
7,8,Kab. Buleleng,85,0,Tidak ada,114,2014,0,Tahun,88501140,88501140,205
8,9,Kota Denpasar,85,0,Tidak ada,114,2014,0,Tahun,98501140,98501140,274


In [28]:
data_85 = data_85[['verval_val','vervar_label','tahun_label','content_label']].copy()
data_85["vervar_label"] = data_85["vervar_label"].str.replace(r"<.*?>", "", regex=True)

data_tb = data_85.rename(columns={
    'vervar_label'   : 'Kabupaten/Kota',
    'tahun_label'    : 'Tahun',
    'content_label'  : 'Nilai'
})

api_order = data_85['vervar_label'].drop_duplicates().tolist()
data_tb['Kabupaten/Kota'] = pd.Categorical(data_tb['Kabupaten/Kota'], categories=api_order, ordered=True)

data_tb = data_tb.pivot_table(
    index=['Kabupaten/Kota'],
    columns=['Tahun'],
    values='Nilai'
)

data_tb.columns = [
    '_'.join([str(c) for c in col if c]).strip() if isinstance(col, tuple) else str(col) 
    for col in data_tb.columns.values
]

data_tb = data_tb.reset_index()

data_tb = data_tb.applymap(
    lambda x: int(x) if isinstance(x, (int, float)) and x.is_integer() else x
)

data_tb


C:\Users\user\AppData\Local\Temp\ipykernel_2772\3946143808.py:13: FutureWarning: The default value of observed=False is deprecated and will change to observed=True in a future version of pandas. Specify observed=False to silence this warning and retain the current behavior
  data_tb = data_tb.pivot_table(
C:\Users\user\AppData\Local\Temp\ipykernel_2772\3946143808.py:26: FutureWarning: DataFrame.applymap has been deprecated. Use DataFrame.map instead.
  data_tb = data_tb.applymap(


,Kabupaten/Kota,2014,2015,2016,2017,2018,2019,2020,2021,2022,2023
0,Kab. Jembrana,139,140,158,163,159,159,155,158,156,157
1,Kab. Tabanan,230,230,238,240,247,248,230,229,228,228
2,Kab. Badung,202,202,223,237,233,238,233,237,240,240
3,Kab. Gianyar,106,106,179,157,159,161,154,155,161,161
4,Kab. Klungkung,110,110,123,124,109,108,104,103,102,102
5,Kab. Bangli,67,67,79,82,84,86,79,83,84,84
6,Kab. Karangasem,116,119,134,163,132,133,116,121,127,127
7,Kab. Buleleng,205,204,228,265,236,241,244,246,245,245
8,Kota Denpasar,274,273,314,314,322,322,304,307,308,309
9,Provinsi Bali,1449,1451,1676,1745,1681,1696,1619,1639,1651,1653


In [29]:
data_85.to_excel("data_85.xlsx", sheet_name= "Banyak_Sekolah_TK", index=False)

Angka Partisipasi Sekolah (APS) Provinsi Bali Menurut Jenjang Pendidikan dan Jenis Kelamin (83)

In [17]:
data_83 = getDataByVarId (domain='5100',varId='83')
data_83

,verval_val,vervar_label,var_val,turvar_val,turvar_label,tahun_val,tahun_label,turtahun_val,turtahun_label,id,content_val,content_label
0,1,7-12 Tahun,83,41,Laki-laki,122,2022,0,Tahun,183411220,183411220,99.35
1,1,7-12 Tahun,83,42,Perempuan,122,2022,0,Tahun,183421220,183421220,99.76
2,1,7-12 Tahun,83,43,Laki-Laki + Perempuan,122,2022,0,Tahun,183431220,183431220,99.55
3,2,13-15 Tahun,83,41,Laki-laki,122,2022,0,Tahun,283411220,283411220,97.21
4,2,13-15 Tahun,83,42,Perempuan,122,2022,0,Tahun,283421220,283421220,98.53
...,...,...,...,...,...,...,...,...,...,...,...,...
7,3,16-18 Tahun,83,42,Perempuan,113,2013,0,Tahun,383421130,383421130,70.98
8,3,16-18 Tahun,83,43,Laki-Laki + Perempuan,113,2013,0,Tahun,383431130,383431130,73.95
9,4,19-24 Tahun,83,41,Laki-laki,113,2013,0,Tahun,483411130,483411130,21.34
10,4,19-24 Tahun,83,42,Perempuan,113,2013,0,Tahun,483421130,483421130,17.56


In [18]:
data_83 = data_83[['verval_val','vervar_label','turvar_label','tahun_label','content_label']].copy()
data_83["vervar_label"] = data_83["vervar_label"].str.replace(r"<.*?>", "", regex=True)


data_tb = data_83.rename(columns={
    'vervar_label'   : 'Umur',
    'tahun_label'    : 'Tahun',
    'turvar_label'   : 'Jenis Kelamin',
    'content_label'  : 'Nilai'
})

api_order = data_83['vervar_label'].drop_duplicates().tolist()
data_tb['Umur'] = pd.Categorical(data_tb['Umur'], categories=api_order, ordered=True)

data_tb = data_tb.pivot_table(
    index=['Umur'],
    columns=['Tahun', 'Jenis Kelamin'],
    values='Nilai'
)

data_tb.columns = ['_'.join([str(c) for c in col if c]).strip() if isinstance(col, tuple) else str(col) 
                   for col in data_tb.columns.values]

data_tb = data_tb.reset_index()

data_tb


C:\Users\user\AppData\Local\Temp\ipykernel_16028\3795776694.py:15: FutureWarning: The default value of observed=False is deprecated and will change to observed=True in a future version of pandas. Specify observed=False to silence this warning and retain the current behavior
  data_tb = data_tb.pivot_table(


,Umur,2013_Laki-Laki + Perempuan,2013_Laki-laki,2013_Perempuan,2014_Laki-Laki + Perempuan,2014_Laki-laki,2014_Perempuan,2015_Laki-Laki + Perempuan,2015_Laki-laki,2015_Perempuan,...,2019_Perempuan,2020_Laki-Laki + Perempuan,2020_Laki-laki,2020_Perempuan,2021_Laki-Laki + Perempuan,2021_Laki-laki,2021_Perempuan,2022_Laki-Laki + Perempuan,2022_Laki-laki,2022_Perempuan
0,7-12 Tahun,99.27,99.29,99.26,99.36,99.58,99.12,99.41,99.70,99.12,...,99.55,99.57,99.43,99.64,99.70,99.67,99.73,99.55,99.35,99.76
1,13-15 Tahun,95.83,95.35,96.29,97.23,97.18,97.27,97.41,97.62,97.19,...,97.82,98.21,98.05,98.38,98.22,98.15,98.30,97.85,97.21,98.53
2,16-18 Tahun,73.95,76.55,70.98,81.59,83.99,78.97,81.69,87.32,74.83,...,82.29,82.96,84.94,80.77,83.96,83.55,84.39,83.84,83.36,84.36
3,19-24 Tahun,19.48,21.34,17.56,23.59,25.32,21.76,23.75,25.90,21.62,...,NaN,28.67,NaN,NaN,28.95,NaN,NaN,30.18,NaN,NaN


In [ ]:
# ============= 1) Ambil & bersihkan dari API =============
d = data_83[['vervar_label','tahun_label','turvar_label','content_label']].copy()

d['Umur'] = (d['vervar_label'].astype(str)
                 .str.replace(r'<.*?>','', regex=True)
                 .str.replace(r'\s+',' ', regex=True)
                 .str.strip())
order_kab = d['Kab/Kota'].drop_duplicates().tolist()

# Label jenis kelamin dari API (tanpa mapping, cuma dirapikan)
d['JK'] = (d['turvar_label'].astype(str)
           .str.replace(r'<.*?>','', regex=True)
           .str.replace(r'\s+',' ', regex=True)
           .str.strip())

# Tahun numerik
d['Tahun'] = pd.to_numeric(d['tahun_label'], errors='coerce')
d = d.dropna(subset=['Tahun']).copy()
d['Tahun'] = d['Tahun'].astype(int)

# Nilai numerik (support format BPS)
def parse_bps_number(s):
    if pd.isna(s): return np.nan
    s = str(s).strip()
    if s == "": return np.nan
    if ',' in s: s = s.replace('.', '').replace(',', '.')
    else:        s = s.replace('.', '')
    s = re.sub(r'[^0-9\.\-]', '', s)
    return float(s) if s not in ("", ".", "-", "-.", ".-") else np.nan

d['Nilai'] = d['content_label'].map(parse_bps_number)

# ============= 2) Pivot =============
wide = d.pivot_table(index='Umur',
                     columns=['Tahun','JK'],
                     values='Nilai',
                     aggfunc='first',
                     sort=False)

# Susun kolom per tahun mengikuti preferensi JK, tapi hanya yang ADA di data
years = sorted(d['Tahun'].unique())
jk_pref = ['Laki-laki', 'Perempuan', 'Laki-laki + Perempuan']  # pakai label API kalau ada
present = [j for j in jk_pref if j in wide.columns.get_level_values(1).unique()]
# kalau ada label lain dari API, ikutkan di belakang tanpa dihitung apa-apa
others  = [j for j in wide.columns.get_level_values(1).unique() if j not in present]
jk_order = present + others

full_cols = pd.MultiIndex.from_product([years, jk_order], names=['Tahun','JK'])
wide = wide.reindex(columns=full_cols)

# Urut baris sesuai kemunculan kab/kota
wide.index = pd.CategoricalIndex(wide.index,
                                 categories=[k for k in order_kab if k in wide.index],
                                 ordered=True)
wide = wide.sort_index()

wide

Tahun            2013                                      2014            \
JK          Laki-laki Perempuan Laki-Laki + Perempuan Laki-laki Perempuan   
Kab/Kota                                                                    
7-12 Tahun     9929.0    9926.0                9927.0    9958.0    9912.0   
13-15 Tahun    9535.0    9629.0                9583.0    9718.0    9727.0   
16-18 Tahun    7655.0    7098.0                7395.0    8399.0    7897.0   
19-24 Tahun    2134.0    1756.0                1948.0    2532.0    2176.0   

Tahun                                  2015                                  \
JK          Laki-Laki + Perempuan Laki-laki Perempuan Laki-Laki + Perempuan   
Kab/Kota                                                                      
7-12 Tahun                 9936.0     997.0    9912.0                9941.0   
13-15 Tahun                9723.0    9762.0    9719.0                9741.0   
16-18 Tahun                8159.0    8732.0    7483.0                8169.0   
19-24 Tahun                2359.0     259.0    2162.0                2375.0   

Tahun            2016  ...                  2019      2020            \
JK          Laki-laki  ... Laki-Laki + Perempuan Laki-laki Perempuan   
Kab/Kota               ...                                             
7-12 Tahun     9962.0  ...                9971.0    9943.0    9964.0   
13-15 Tahun    9777.0  ...                9772.0    9805.0    9838.0   
16-18 Tahun    8395.0  ...                 826.0    8494.0    8077.0   
19-24 Tahun    2702.0  ...                   NaN       NaN       NaN   

Tahun                                  2021                                  \
JK          Laki-Laki + Perempuan Laki-laki Perempuan Laki-Laki + Perempuan   
Kab/Kota                                                                      
7-12 Tahun                 9957.0    9967.0    9973.0                 997.0   
13-15 Tahun                9821.0    9815.0     983.0                9822.0   
16-18 Tahun                8296.0    8355.0    8439.0                8396.0   
19-24 Tahun                2867.0       NaN       NaN                2895.0   

Tahun            2022                                  
JK          Laki-laki Perempuan Laki-Laki + Perempuan  
Kab/Kota                                               
7-12 Tahun     9935.0    9976.0                9955.0  
13-15 Tahun    9721.0    9853.0                9785.0  
16-18 Tahun    8336.0    8436.0                8384.0  
19-24 Tahun       NaN       NaN                3018.0  

[4 rows x 30 columns]

In [21]:
wide_flat = wide.copy()
wide_flat.columns = [f"{yr} - {bln}" for yr, bln in wide_flat.columns]
wide_flat.reset_index().to_excel("data_83.xlsx", index=False, engine="openpyxl")

Angka Melek Huruf Provinsi Bali Menurut Kabupaten/Kota dan Jenis Kelamin (71)

In [22]:
data_71 = getDataByVarId (domain='5100',varId='71')
data_71

,verval_val,vervar_label,var_val,turvar_val,turvar_label,tahun_val,tahun_label,turtahun_val,turtahun_label,id,content_val,content_label
0,1,Kab. Jembrana,71,41,Laki-laki,124,2024,0,Tahun,171411240,171411240,98.80
1,1,Kab. Jembrana,71,42,Perempuan,124,2024,0,Tahun,171421240,171421240,94.52
2,1,Kab. Jembrana,71,43,Laki-Laki + Perempuan,124,2024,0,Tahun,171431240,171431240,96.63
3,2,Kab. Tabanan,71,41,Laki-laki,124,2024,0,Tahun,271411240,271411240,98.28
4,2,Kab. Tabanan,71,42,Perempuan,124,2024,0,Tahun,271421240,271421240,94.21
...,...,...,...,...,...,...,...,...,...,...,...,...
25,9,Kota Denpasar,71,42,Perempuan,115,2015,0,Tahun,971421150,971421150,96.56
26,9,Kota Denpasar,71,43,Laki-Laki + Perempuan,115,2015,0,Tahun,971431150,971431150,97.96
27,10,<b>Provinsi Bali</b>,71,41,Laki-laki,115,2015,0,Tahun,1071411150,1071411150,96.57
28,10,<b>Provinsi Bali</b>,71,42,Perempuan,115,2015,0,Tahun,1071421150,1071421150,88.94


In [23]:
# ============= 1) Ambil & bersihkan dari API =============
d = data_71[['vervar_label','tahun_label','turvar_label','content_label']].copy()

# Nama kab/kota & urutan kemunculan
d['Kab/Kota'] = (d['vervar_label'].astype(str)
                 .str.replace(r'<.*?>','', regex=True)
                 .str.replace(r'\s+',' ', regex=True)
                 .str.strip())
order_kab = d['Kab/Kota'].drop_duplicates().tolist()

# Label jenis kelamin dari API (tanpa mapping, cuma dirapikan)
d['JK'] = (d['turvar_label'].astype(str)
           .str.replace(r'<.*?>','', regex=True)
           .str.replace(r'\s+',' ', regex=True)
           .str.strip())

# Tahun numerik
d['Tahun'] = pd.to_numeric(d['tahun_label'], errors='coerce')
d = d.dropna(subset=['Tahun']).copy()
d['Tahun'] = d['Tahun'].astype(int)

# Nilai numerik (support format BPS)
def parse_bps_number(s):
    if pd.isna(s): return np.nan
    s = str(s).strip()
    if s == "": return np.nan
    if ',' in s: s = s.replace('.', '').replace(',', '.')
    else:        s = s.replace('.', '')
    s = re.sub(r'[^0-9\.\-]', '', s)
    return float(s) if s not in ("", ".", "-", "-.", ".-") else np.nan

d['Nilai'] = d['content_label'].map(parse_bps_number)

# ============= 2) Pivot =============
wide = d.pivot_table(index='Kab/Kota',
                     columns=['Tahun','JK'],
                     values='Nilai',
                     aggfunc='first',
                     sort=False)

# Susun kolom per tahun mengikuti preferensi JK, tapi hanya yang ADA di data
years = sorted(d['Tahun'].unique())
jk_pref = ['Laki-laki', 'Perempuan', 'Laki-laki + Perempuan']  # pakai label API kalau ada
present = [j for j in jk_pref if j in wide.columns.get_level_values(1).unique()]
# kalau ada label lain dari API, ikutkan di belakang tanpa dihitung apa-apa
others  = [j for j in wide.columns.get_level_values(1).unique() if j not in present]
jk_order = present + others

full_cols = pd.MultiIndex.from_product([years, jk_order], names=['Tahun','JK'])
wide = wide.reindex(columns=full_cols)

# Urut baris sesuai kemunculan kab/kota
wide.index = pd.CategoricalIndex(wide.index,
                                 categories=[k for k in order_kab if k in wide.index],
                                 ordered=True)
wide = wide.sort_index()

wide

Tahun                2015                                      2016            \
JK              Laki-laki Perempuan Laki-Laki + Perempuan Laki-laki Perempuan   
Kab/Kota                                                                        
Kab. Jembrana      9818.0    9017.0                9414.0    9769.0    8989.0   
Kab. Tabanan        981.0    9108.0                9454.0    9756.0    9031.0   
Kab. Badung        9819.0    9193.0                9513.0    9886.0    9516.0   
Kab. Gianyar       9755.0    8928.0                9343.0    9703.0    8945.0   
Kab. Klungkung     9217.0    8072.0                8632.0    9213.0    7971.0   
Kab. Bangli        9315.0    8519.0                8921.0    9306.0    8299.0   
Kab. Karangasem    8871.0    7653.0                8252.0    8792.0    7567.0   
Kab. Buleleng      9583.0    8486.0                9034.0    9472.0    8415.0   
Kota Denpasar      9931.0    9656.0                9796.0    9969.0    9762.0   
Provinsi Bali      9657.0    8894.0                9277.0    9637.0    8923.0   

Tahun                                      2017            \
JK              Laki-Laki + Perempuan Laki-laki Perempuan   
Kab/Kota                                                    
Kab. Jembrana                  9374.0     978.0     899.0   
Kab. Tabanan                   9388.0     976.0     905.0   
Kab. Badung                    9705.0     991.0     951.0   
Kab. Gianyar                   9327.0     970.0     895.0   
Kab. Klungkung                 8582.0     907.0     813.0   
Kab. Bangli                    8809.0     941.0     821.0   
Kab. Karangasem                8174.0     890.0     748.0   
Kab. Buleleng                  8941.0     938.0     852.0   
Kota Denpasar                  9868.0     997.0     977.0   
Provinsi Bali                  9282.0     964.0     894.0   

Tahun                                      2018  ...                  2021  \
JK              Laki-Laki + Perempuan Laki-laki  ... Laki-Laki + Perempuan   
Kab/Kota                                         ...                         
Kab. Jembrana                   938.0    9707.0  ...                9576.0   
Kab. Tabanan                    940.0    9728.0  ...                9689.0   
Kab. Badung                     971.0    9835.0  ...                9861.0   
Kab. Gianyar                    933.0    9628.0  ...                9601.0   
Kab. Klungkung                  858.0    9325.0  ...                8885.0   
Kab. Bangli                     882.0    9333.0  ...                8916.0   
Kab. Karangasem                 818.0    9075.0  ...                8408.0   
Kab. Buleleng                   895.0    9564.0  ...                9274.0   
Kota Denpasar                   987.0     991.0  ...                 993.0   
Provinsi Bali                   929.0    9649.0  ...                 950.0   

Tahun                2022                                      2023            \
JK              Laki-laki Perempuan Laki-Laki + Perempuan Laki-laki Perempuan   
Kab/Kota                                                                        
Kab. Jembrana      9781.0    9438.0                9609.0    9865.0     943.0   
Kab. Tabanan       9756.0    9596.0                9676.0    9858.0    9608.0   
Kab. Badung        9902.0    9621.0                9762.0    9885.0    9593.0   
Kab. Gianyar       9662.0    9216.0                9439.0    9717.0    9359.0   
Kab. Klungkung     9658.0    8833.0                9243.0    9555.0    8895.0   
Kab. Bangli        9434.0    8549.0                8992.0    9412.0    8737.0   
Kab. Karangasem    9213.0    8195.0                8709.0    9061.0    8119.0   
Kab. Buleleng      9715.0    9348.0                9531.0    9804.0    9201.0   
Kota Denpasar      9989.0    9886.0                9938.0    9972.0    9922.0   
Provinsi Bali      9745.0    9361.0                9553.0    9757.0    9363.0   

Tahun                                      2024            \
JK              Laki-L

In [24]:
wide_flat = wide.copy()
wide_flat.columns = [f"{yr} - {bln}" for yr, bln in wide_flat.columns]
wide_flat.reset_index().to_excel("data_71.xlsx", index=False, engine="openpyxl")

Banyaknya Penerbangan dan Penumpang di Provinsi Bali (259)

In [25]:
data_259 = getDataByVarId (domain='5100',varId='259')
data_259

,verval_val,vervar_label,var_val,turvar_val,turvar_label,tahun_val,tahun_label,turtahun_val,turtahun_label,id,content_val,content_label
0,1,Pesawat Tiba,259,0,Tidak ada,124,2024,0,Tahun,125901240,125901240,71081
1,2,Pesawat Berangkat,259,0,Tidak ada,124,2024,0,Tahun,225901240,225901240,71088
2,3,Penumpang Domestik Tiba,259,0,Tidak ada,124,2024,0,Tahun,325901240,325901240,4901759
3,4,Penumpang Domestik Berangkat,259,0,Tidak ada,124,2024,0,Tahun,425901240,425901240,4719044
4,5,Penumpang Domestik Transit,259,0,Tidak ada,124,2024,0,Tahun,525901240,525901240,223836
...,...,...,...,...,...,...,...,...,...,...,...,...
3,4,Penumpang Domestik Berangkat,259,0,Tidak ada,115,2015,0,Tahun,425901150,425901150,4276790
4,5,Penumpang Domestik Transit,259,0,Tidak ada,115,2015,0,Tahun,525901150,525901150,99269
5,6,Penumpang Asing Tiba,259,0,Tidak ada,115,2015,0,Tahun,625901150,625901150,4271042
6,7,Penumpang Asing Berangkat,259,0,Tidak ada,115,2015,0,Tahun,725901150,725901150,4235580


In [26]:
data_259 = data_259[['verval_val','vervar_label','tahun_label','content_label']].copy()
data_259["vervar_label"] = data_259["vervar_label"].str.replace(r"<.*?>", "", regex=True)

data_tb = data_259.rename(columns={
    'vervar_label'   : 'Jenis Perjalanan',
    'tahun_label'    : 'Tahun',
    'content_label'  : 'Nilai'
})

api_order = data_259['vervar_label'].drop_duplicates().tolist()
data_tb['Jenis Perjalanan'] = pd.Categorical(data_tb['Jenis Perjalanan'], categories=api_order, ordered=True)

data_tb = data_tb.pivot_table(
    index=['Jenis Perjalanan'],
    columns=['Tahun'],
    values='Nilai'
)

data_tb.columns = [
    '_'.join([str(c) for c in col if c]).strip() if isinstance(col, tuple) else str(col) 
    for col in data_tb.columns.values
]

data_tb = data_tb.reset_index()

data_tb = data_tb.applymap(
    lambda x: int(x) if isinstance(x, (int, float)) and x.is_integer() else x
)

data_tb


C:\Users\user\AppData\Local\Temp\ipykernel_16028\995309974.py:13: FutureWarning: The default value of observed=False is deprecated and will change to observed=True in a future version of pandas. Specify observed=False to silence this warning and retain the current behavior
  data_tb = data_tb.pivot_table(
C:\Users\user\AppData\Local\Temp\ipykernel_16028\995309974.py:26: FutureWarning: DataFrame.applymap has been deprecated. Use DataFrame.map instead.
  data_tb = data_tb.applymap(


,Jenis Perjalanan,2015,2016,2017,2018,2019,2020,2021,2022,2023,2024
0,Pesawat Tiba,62806,69564,73200,81335,77635,28075,21753,43787,73545,71081
1,Pesawat Berangkat,62796,69536,73213,81289,77699,28098,21768,43790,73536,71088
2,Penumpang Domestik Tiba,4139706,4977673,4928000,5531125,4974558,1735628,2277471,3978953,5333319,4901759
3,Penumpang Domestik Berangkat,4276790,4912815,5055977,5500822,4956092,1775741,2190093,3878209,5053688,4719044
4,Penumpang Domestik Transit,99269,68456,46192,112163,352471,145929,81931,89965,263760,223836
5,Penumpang Asing Tiba,4271042,4928907,5317800,6112014,6864934,1183054,3072,2332339,6138907,6982674
6,Penumpang Asing Berangkat,4235580,4988254,5578065,6351957,6939208,1356003,3484,2212411,6128531,7041983
7,Penumpang Asing Transit,2588,607,2436,36226,82298,42419,33,30523,12029,61603


In [27]:
data_259.to_excel("data_259.xlsx", sheet_name= "Jenis Perjalanan", index=False)

Jumlah Kendaraan Listrik Menurut Jenis Kendaraan dan  Kabupaten/Kota di Provinsi Bali (430)

In [28]:
data_430 = getDataByVarId (domain='5100',varId='430')
data_430

,verval_val,vervar_label,var_val,turvar_val,turvar_label,tahun_val,tahun_label,turtahun_val,turtahun_label,id,content_val,content_label
0,1,Kab. Jembrana,430,198,Mobil Listrik,124,2024,0,Tahun,14301981240,14301981240,13
1,1,Kab. Jembrana,430,199,Sepeda Motor Listrik,124,2024,0,Tahun,14301991240,14301991240,41
2,1,Kab. Jembrana,430,200,Jumlah,124,2024,0,Tahun,14302001240,14302001240,54
3,2,Kab. Tabanan,430,198,Mobil Listrik,124,2024,0,Tahun,24301981240,24301981240,53
4,2,Kab. Tabanan,430,199,Sepeda Motor Listrik,124,2024,0,Tahun,24301991240,24301991240,89
5,2,Kab. Tabanan,430,200,Jumlah,124,2024,0,Tahun,24302001240,24302001240,142
6,3,Kab. Badung,430,198,Mobil Listrik,124,2024,0,Tahun,34301981240,34301981240,290
7,3,Kab. Badung,430,199,Sepeda Motor Listrik,124,2024,0,Tahun,34301991240,34301991240,571
8,3,Kab. Badung,430,200,Jumlah,124,2024,0,Tahun,34302001240,34302001240,861
9,4,Kab. Gianyar,430,198,Mobil Listrik,124,2024,0,Tahun,44301981240,44301981240,72


In [29]:

data_430 = data_430[['verval_val','vervar_label','turvar_label','tahun_label','content_label']].copy()
data_430["vervar_label"] = data_430["vervar_label"].str.replace(r"<.*?>", "", regex=True)


data_tb = data_430.rename(columns={
    'vervar_label'   : 'Kabupaten/Kota',
    'tahun_label'    : 'Tahun',
    'turvar_label'   : 'Jenis Kendaraan',
    'content_label'  : 'Nilai'
})

api_order = data_430['vervar_label'].drop_duplicates().tolist()
data_tb['Kabupaten/Kota'] = pd.Categorical(data_tb['Kabupaten/Kota'], categories=api_order, ordered=True)

data_tb = data_tb.pivot_table(
    index=['Kabupaten/Kota'],
    columns=['Tahun', 'Jenis Kendaraan'],
    values='Nilai'
)

data_tb.columns = ['_'.join([str(c) for c in col if c]).strip() if isinstance(col, tuple) else str(col) 
                   for col in data_tb.columns.values]

data_tb = data_tb.reset_index()

data_tb


C:\Users\user\AppData\Local\Temp\ipykernel_16028\3832947056.py:15: FutureWarning: The default value of observed=False is deprecated and will change to observed=True in a future version of pandas. Specify observed=False to silence this warning and retain the current behavior
  data_tb = data_tb.pivot_table(


,Kabupaten/Kota,2023_Jumlah,2023_Mobil Listrik,2023_Sepeda Motor Listrik,2024_Jumlah,2024_Mobil Listrik,2024_Sepeda Motor Listrik
0,Kab. Jembrana,42.0,3.0,39.0,54.0,13.0,41.0
1,Kab. Tabanan,211.0,18.0,193.0,142.0,53.0,89.0
2,Kab. Badung,1026.0,109.0,917.0,861.0,290.0,571.0
3,Kab. Gianyar,356.0,39.0,317.0,199.0,72.0,127.0
4,Kab. Klungkung,70.0,2.0,68.0,43.0,10.0,33.0
5,Kab. Bangli,33.0,1.0,32.0,43.0,10.0,33.0
6,Kab. Karangasem,76.0,3.0,73.0,40.0,14.0,26.0
7,Kab. Buleleng,143.0,12.0,131.0,99.0,41.0,58.0
8,Kota Denpasar,1675.0,163.0,1512.0,1457.0,522.0,935.0
9,Provinsi Bali,3632.0,350.0,3282.0,2938.0,1025.0,1913.0


In [30]:
# ============= 1) Ambil & bersihkan dari API =============
d = data_430[['vervar_label','tahun_label','turvar_label','content_label']].copy()

# Nama kab/kota & urutan kemunculan
d['Kab/Kota'] = (d['vervar_label'].astype(str)
                 .str.replace(r'<.*?>','', regex=True)
                 .str.replace(r'\s+',' ', regex=True)
                 .str.strip())
order_kab = d['Kab/Kota'].drop_duplicates().tolist()

# Label jenis kelamin dari API (tanpa mapping, cuma dirapikan)
d['JK'] = (d['turvar_label'].astype(str)
           .str.replace(r'<.*?>','', regex=True)
           .str.replace(r'\s+',' ', regex=True)
           .str.strip())

# Tahun numerik
d['Tahun'] = pd.to_numeric(d['tahun_label'], errors='coerce')
d = d.dropna(subset=['Tahun']).copy()
d['Tahun'] = d['Tahun'].astype(int)

# Nilai numerik (support format BPS)
def parse_bps_number(s):
    if pd.isna(s): return np.nan
    s = str(s).strip()
    if s == "": return np.nan
    if ',' in s: s = s.replace('.', '').replace(',', '.')
    else:        s = s.replace('.', '')
    s = re.sub(r'[^0-9\.\-]', '', s)
    return float(s) if s not in ("", ".", "-", "-.", ".-") else np.nan

d['Nilai'] = d['content_label'].map(parse_bps_number)

# ============= 2) Pivot =============
wide = d.pivot_table(index='Kab/Kota',
                     columns=['Tahun','JK'],
                     values='Nilai',
                     aggfunc='first',
                     sort=False)

# Susun kolom per tahun mengikuti preferensi JK, tapi hanya yang ADA di data
years = sorted(d['Tahun'].unique())
jk_pref = ['Mobil Listrik', 'Sepeda Motor Listrik', 'Jumlah']  # pakai label API kalau ada
present = [j for j in jk_pref if j in wide.columns.get_level_values(1).unique()]
# kalau ada label lain dari API, ikutkan di belakang tanpa dihitung apa-apa
others  = [j for j in wide.columns.get_level_values(1).unique() if j not in present]
jk_order = present + others

full_cols = pd.MultiIndex.from_product([years, jk_order], names=['Tahun','JK'])
wide = wide.reindex(columns=full_cols)

# Urut baris sesuai kemunculan kab/kota
wide.index = pd.CategoricalIndex(wide.index,
                                 categories=[k for k in order_kab if k in wide.index],
                                 ordered=True)
wide = wide.sort_index()

wide

Tahun                    2023                                       2024  \
JK              Mobil Listrik Sepeda Motor Listrik  Jumlah Mobil Listrik   
Kab/Kota                                                                   
Kab. Jembrana             3.0                 39.0    42.0          13.0   
Kab. Tabanan             18.0                193.0   211.0          53.0   
Kab. Badung             109.0                917.0  1026.0         290.0   
Kab. Gianyar             39.0                317.0   356.0          72.0   
Kab. Klungkung            2.0                 68.0    70.0          10.0   
Kab. Bangli               1.0                 32.0    33.0          10.0   
Kab. Karangasem           3.0                 73.0    76.0          14.0   
Kab. Buleleng            12.0                131.0   143.0          41.0   
Kota Denpasar           163.0               1512.0  1675.0         522.0   
Provinsi Bali           350.0               3282.0  3632.0        1025.0   

Tahun                                         
JK              Sepeda Motor Listrik  Jumlah  
Kab/Kota                                      
Kab. Jembrana                   41.0    54.0  
Kab. Tabanan                    89.0   142.0  
Kab. Badung                    571.0   861.0  
Kab. Gianyar                   127.0   199.0  
Kab. Klungkung                  33.0    43.0  
Kab. Bangli                     33.0    43.0  
Kab. Karangasem                 26.0    40.0  
Kab. Buleleng                   58.0    99.0  
Kota Denpasar                  935.0  1457.0  
Provinsi Bali                 1913.0  2938.0

In [31]:
wide_flat = wide.copy()
wide_flat.columns = [f"{yr} - {bln}" for yr, bln in wide_flat.columns]
wide_flat.reset_index().to_excel("data_430.xlsx", index=False, engine="openpyxl")

Nilai Tukar Petani Provinsi Bali (117)

In [37]:
data_117 = getDataByVarId (domain='5100',varId='117')
data_117

,verval_val,vervar_label,var_val,turvar_val,turvar_label,tahun_val,tahun_label,turtahun_val,turtahun_label,id,content_val,content_label
0,1,Indeks Diterima,117,0,Tidak ada,125,2025,1,Januari,111701251,111701251,128.87
1,1,Indeks Diterima,117,0,Tidak ada,125,2025,2,Februari,111701252,111701252,128.10
2,1,Indeks Diterima,117,0,Tidak ada,125,2025,3,Maret,111701253,111701253,130.72
3,1,Indeks Diterima,117,0,Tidak ada,125,2025,4,April,111701254,111701254,131.35
4,1,Indeks Diterima,117,0,Tidak ada,125,2025,5,Mei,111701255,111701255,128.99
...,...,...,...,...,...,...,...,...,...,...,...,...
33,3,Nilai Tukar Petani,117,0,Tidak ada,116,2016,8,Agustus,311701168,311701168,106.66
34,3,Nilai Tukar Petani,117,0,Tidak ada,116,2016,9,September,311701169,311701169,107.44
35,3,Nilai Tukar Petani,117,0,Tidak ada,116,2016,10,Oktober,3117011610,3117011610,107.13
36,3,Nilai Tukar Petani,117,0,Tidak ada,116,2016,11,November,3117011611,3117011611,107.06


In [38]:
data_117 = data_117[['verval_val','vervar_label','tahun_label','turtahun_label','content_label']].copy()
data_117["vervar_label"] = data_117["vervar_label"].str.replace(r"<.*?>", "", regex=True)

data_tb = data_117.rename(columns={
    'vervar_label'   : 'Nilai Tukar Petani',
    'tahun_label'    : 'Tahun',
    'turtahun_label' : 'Bulan',
    'content_label'  : 'Nilai'
})

api_order = data_117['vervar_label'].drop_duplicates().tolist()
data_tb['Nilai Tukar Petani'] = pd.Categorical(data_tb['Nilai Tukar Petani'], categories=api_order, ordered=True)

data_tb = data_tb.pivot_table(
    index=['Nilai Tukar Petani'],
    columns=['Tahun', 'Bulan'],
    values='Nilai'
)

data_tb.columns = [
    '_'.join([str(c) for c in col if c]).strip() if isinstance(col, tuple) else str(col) 
    for col in data_tb.columns.values
]

data_tb = data_tb.reset_index()

data_tb = data_tb.applymap(
    lambda x: int(x) if isinstance(x, (int, float)) and x.is_integer() else x
)

data_tb


C:\Users\user\AppData\Local\Temp\ipykernel_16028\274861758.py:14: FutureWarning: The default value of observed=False is deprecated and will change to observed=True in a future version of pandas. Specify observed=False to silence this warning and retain the current behavior
  data_tb = data_tb.pivot_table(
C:\Users\user\AppData\Local\Temp\ipykernel_16028\274861758.py:27: FutureWarning: DataFrame.applymap has been deprecated. Use DataFrame.map instead.
  data_tb = data_tb.applymap(


,Nilai Tukar Petani,2016_Agustus,2016_April,2016_Desember,2016_Februari,2016_Januari,2016_Juli,2016_Juni,2016_Maret,2016_Mei,...,2024_November,2024_Oktober,2024_September,2025_April,2025_Februari,2025_Januari,2025_Juli,2025_Juni,2025_Maret,2025_Mei
0,Indeks Diterima,129.66,126.37,130.72,127.26,126.30,129.43,128.83,126.93,127.66,...,122.05,120.18,119.48,131.35,128.10,128.87,128.48,128.95,130.72,128.99
1,Indeks Dibayar,121.57,120.57,122.47,120.71,120.33,121.34,120.88,121.04,120.50,...,123.13,121.66,121.47,126.51,123.93,125.09,126.35,125.83,125.53,125.21
2,Nilai Tukar Petani,106.66,104.81,106.74,105.42,104.96,106.67,106.58,104.86,105.94,...,99.13,98.78,98.36,103.83,103.37,103.02,101.69,102.47,104.14,103.02


In [40]:
# Mapping nama bulan ke angka
bulan_order = {
    'Januari': 1, 'Februari': 2, 'Maret': 3, 'April': 4,
    'Mei': 5, 'Juni': 6, 'Juli': 7, 'Agustus': 8,
    'September': 9, 'Oktober': 10, 'November': 11, 'Desember': 12
}

# Pisahkan kolom menjadi Tahun & Bulan
cols = data_tb.columns.tolist()
index_col = ['Nilai Tukar Petani']
data_cols = [c for c in cols if c not in index_col]

# Buat tuple (tahun, bulan_order) untuk sorting
def sort_key(col):
    try:
        tahun, bulan = col.split('_')
        return (int(tahun), bulan_order.get(bulan, 13))  # 13 buat yg tidak dikenali
    except:
        return (9999, 99)  # jika format beda
       
sorted_cols = index_col + sorted(data_cols, key=sort_key)

# Reorder dataframe
data_tb = data_tb[sorted_cols]


In [41]:
data_tb

,Nilai Tukar Petani,2016_Januari,2016_Februari,2016_Maret,2016_April,2016_Mei,2016_Juni,2016_Juli,2016_Agustus,2016_September,...,2024_Oktober,2024_November,2024_Desember,2025_Januari,2025_Februari,2025_Maret,2025_April,2025_Mei,2025_Juni,2025_Juli
0,Indeks Diterima,126.30,127.26,126.93,126.37,127.66,128.83,129.43,129.66,131.14,...,120.18,122.05,125.34,128.87,128.10,130.72,131.35,128.99,128.95,128.48
1,Indeks Dibayar,120.33,120.71,121.04,120.57,120.50,120.88,121.34,121.57,122.06,...,121.66,123.13,123.77,125.09,123.93,125.53,126.51,125.21,125.83,126.35
2,Nilai Tukar Petani,104.96,105.42,104.86,104.81,105.94,106.58,106.67,106.66,107.44,...,98.78,99.13,101.27,103.02,103.37,104.14,103.83,103.02,102.47,101.69


In [42]:
data_117.to_excel("data_117.xlsx", sheet_name= "Nilai_Tukar_Petani", index=False)

Nilai Tukar Petani Provinsi Bali Menurut Subsektor (167)

In [8]:
data_167 = getDataByVarId (domain='5100',varId='167')
data_167

,verval_val,vervar_label,var_val,turvar_val,turvar_label,tahun_val,tahun_label,turtahun_val,turtahun_label,id,content_val,content_label
0,1,Tanaman Pangan,167,0,Tidak ada,125,2025,1,Januari,116701251,116701251,97.85
1,1,Tanaman Pangan,167,0,Tidak ada,125,2025,2,Februari,116701252,116701252,99.41
2,1,Tanaman Pangan,167,0,Tidak ada,125,2025,3,Maret,116701253,116701253,100.41
3,1,Tanaman Pangan,167,0,Tidak ada,125,2025,4,April,116701254,116701254,98.63
4,1,Tanaman Pangan,167,0,Tidak ada,125,2025,5,Mei,116701255,116701255,100.35
...,...,...,...,...,...,...,...,...,...,...,...,...
59,5,Perikanan,167,0,Tidak ada,116,2016,8,Agustus,516701168,516701168,102.22
60,5,Perikanan,167,0,Tidak ada,116,2016,9,September,516701169,516701169,102.56
61,5,Perikanan,167,0,Tidak ada,116,2016,10,Oktober,5167011610,5167011610,103.62
62,5,Perikanan,167,0,Tidak ada,116,2016,11,November,5167011611,5167011611,104.02


In [10]:
data_167 = data_167[['verval_val','vervar_label','tahun_label','turtahun_label','content_label']].copy()
data_167["vervar_label"] = data_167["vervar_label"].str.replace(r"<.*?>", "", regex=True)

data_tb = data_167.rename(columns={
    'vervar_label'   : 'Nilai Tukar Petani',
    'tahun_label'    : 'Tahun',
    'turtahun_label' : 'Bulan',
    'content_label'  : 'Nilai'
})

api_order = data_167['vervar_label'].drop_duplicates().tolist()
data_tb['Nilai Tukar Petani'] = pd.Categorical(data_tb['Nilai Tukar Petani'], categories=api_order, ordered=True)

data_tb = data_tb.pivot_table(
    index=['Nilai Tukar Petani'],
    columns=['Tahun', 'Bulan'],
    values='Nilai'
)

data_tb.columns = [
    '_'.join([str(c) for c in col if c]).strip() if isinstance(col, tuple) else str(col) 
    for col in data_tb.columns.values
]

data_tb = data_tb.reset_index()

data_tb = data_tb.applymap(
    lambda x: int(x) if isinstance(x, (int, float)) and x.is_integer() else x
)

data_tb


C:\Users\user\AppData\Local\Temp\ipykernel_19036\864147643.py:14: FutureWarning: The default value of observed=False is deprecated and will change to observed=True in a future version of pandas. Specify observed=False to silence this warning and retain the current behavior
  data_tb = data_tb.pivot_table(
C:\Users\user\AppData\Local\Temp\ipykernel_19036\864147643.py:27: FutureWarning: DataFrame.applymap has been deprecated. Use DataFrame.map instead.
  data_tb = data_tb.applymap(


,Nilai Tukar Petani,2016_Agustus,2016_April,2016_Desember,2016_Februari,2016_Januari,2016_Juli,2016_Juni,2016_Maret,2016_Mei,...,2024_November,2024_Oktober,2024_September,2025_April,2025_Februari,2025_Januari,2025_Juli,2025_Juni,2025_Maret,2025_Mei
0,Tanaman Pangan,96.53,96.34,95.82,99.62,99.81,95.90,96.71,98.04,97.09,...,96.91,100.27,99.55,98.63,99.41,97.85,101.60,100.25,100.41,100.35
1,Hortikultura,105.71,105.03,106.60,104.96,104.41,106.58,105.64,104.20,105.34,...,98.67,90.59,86.54,111.43,107.48,119.99,103.64,99.83,108.20,96.06
2,Tanaman Perkebunan Rakyat,105.46,100.90,105.74,99.94,99.88,107.10,105.64,99.67,103.58,...,103.02,102.80,104.43,117.05,113.86,108.49,111.39,117.10,117.36,118.49
3,Peternakan,115.98,113.89,115.78,114.13,112.98,114.56,115.40,114.33,114.80,...,99.15,98.83,98.79,96.22,98.02,96.57,93.97,95.51,96.83,98.06
4,Perikanan,102.22,100.97,104.34,102.66,101.90,103.18,103.61,101.13,102.22,...,93.86,92.50,94.45,95.85,98.21,95.14,97.03,97.98,95.40,97.91


In [11]:
# Mapping nama bulan ke angka
bulan_order = {
    'Januari': 1, 'Februari': 2, 'Maret': 3, 'April': 4,
    'Mei': 5, 'Juni': 6, 'Juli': 7, 'Agustus': 8,
    'September': 9, 'Oktober': 10, 'November': 11, 'Desember': 12
}

# Pisahkan kolom menjadi Tahun & Bulan
cols = data_tb.columns.tolist()
index_col = ['Nilai Tukar Petani']
data_cols = [c for c in cols if c not in index_col]

# Buat tuple (tahun, bulan_order) untuk sorting
def sort_key(col):
    try:
        tahun, bulan = col.split('_')
        return (int(tahun), bulan_order.get(bulan, 13))  # 13 buat yg tidak dikenali
    except:
        return (9999, 99)  # jika format beda
       
sorted_cols = index_col + sorted(data_cols, key=sort_key)

# Reorder dataframe
data_tb = data_tb[sorted_cols]


In [12]:
data_tb

,Nilai Tukar Petani,2016_Januari,2016_Februari,2016_Maret,2016_April,2016_Mei,2016_Juni,2016_Juli,2016_Agustus,2016_September,...,2024_Oktober,2024_November,2024_Desember,2025_Januari,2025_Februari,2025_Maret,2025_April,2025_Mei,2025_Juni,2025_Juli
0,Tanaman Pangan,99.81,99.62,98.04,96.34,97.09,96.71,95.90,96.53,96.73,...,100.27,96.91,97.69,97.85,99.41,100.41,98.63,100.35,100.25,101.60
1,Hortikultura,104.41,104.96,104.20,105.03,105.34,105.64,106.58,105.71,106.64,...,90.59,98.67,107.10,119.99,107.48,108.20,111.43,96.06,99.83,103.64
2,Tanaman Perkebunan Rakyat,99.88,99.94,99.67,100.90,103.58,105.64,107.10,105.46,105.38,...,102.80,103.02,107.29,108.49,113.86,117.36,117.05,118.49,117.10,111.39
3,Peternakan,112.98,114.13,114.33,113.89,114.80,115.40,114.56,115.98,117.73,...,98.83,99.15,98.16,96.57,98.02,96.83,96.22,98.06,95.51,93.97
4,Perikanan,101.90,102.66,101.13,100.97,102.22,103.61,103.18,102.22,102.56,...,92.50,93.86,94.65,95.14,98.21,95.40,95.85,97.91,97.98,97.03


In [13]:
data_167.to_excel("data_167.xlsx", sheet_name= "Nilai_Tukar_Petani_Subsektor", index=False)

Jumlah Penumpang Tiba di Pelabuhan di Provinsi Bali (253)

In [ ]:
data_253 = getDataByVarId (domain='0000',varId='253')
data_253

,verval_val,vervar_label,var_val,turvar_val,turvar_label,tahun_val,tahun_label,turtahun_val,turtahun_label,id,content_val,content_label
0,1100,ACEH,253,0,Tidak ada,118,2018,0,Tahun,110025301180,110025301180,73.0
1,1200,SUMATERA UTARA,253,0,Tidak ada,118,2018,0,Tahun,120025301180,120025301180,1498.0
4,1500,JAMBI,253,0,Tidak ada,118,2018,0,Tahun,150025301180,150025301180,39.0
5,1600,SUMATERA SELATAN,253,0,Tidak ada,118,2018,0,Tahun,160025301180,160025301180,149.0
6,1700,BENGKULU,253,0,Tidak ada,118,2018,0,Tahun,170025301180,170025301180,50.0
...,...,...,...,...,...,...,...,...,...,...,...,...
27,7400,SULAWESI TENGGARA,253,0,Tidak ada,109,2009,0,Tahun,740025301090,740025301090,137.0
28,7500,GORONTALO,253,0,Tidak ada,109,2009,0,Tahun,750025301090,750025301090,43.0
30,8100,MALUKU,253,0,Tidak ada,109,2009,0,Tahun,810025301090,810025301090,68.0
34,9400,PAPUA,253,0,Tidak ada,109,2009,0,Tahun,940025301090,940025301090,10.0


In [17]:
data_253 = data_253[['verval_val','vervar_label','tahun_label','content_label']].copy()
data_253["vervar_label"] = data_253["vervar_label"].str.replace(r"<.*?>", "", regex=True)

data_tb = data_253.rename(columns={
    'vervar_label'   : 'Asal',
    'tahun_label'    : 'Tahun',
    'content_label'  : 'Nilai'
})

api_order = data_253['vervar_label'].drop_duplicates().tolist()
data_tb['Asal'] = pd.Categorical(data_tb['Asal'], categories=api_order, ordered=True)

data_tb = data_tb.pivot_table(
    index=['Asal'],
    columns=['Tahun'],
    values='Nilai'
)

data_tb.columns = [
    '_'.join([str(c) for c in col if c]).strip() if isinstance(col, tuple) else str(col) 
    for col in data_tb.columns.values
]

data_tb = data_tb.reset_index()

data_tb = data_tb.applymap(
    lambda x: int(x) if isinstance(x, (int, float)) and x.is_integer() else x
)

data_tb


C:\Users\user\AppData\Local\Temp\ipykernel_19036\2250750897.py:13: FutureWarning: The default value of observed=False is deprecated and will change to observed=True in a future version of pandas. Specify observed=False to silence this warning and retain the current behavior
  data_tb = data_tb.pivot_table(
C:\Users\user\AppData\Local\Temp\ipykernel_19036\2250750897.py:26: FutureWarning: DataFrame.applymap has been deprecated. Use DataFrame.map instead.
  data_tb = data_tb.applymap(


,Asal,2009,2010,2011,2012,2013,2014,2015,2016,2017,2018
0,ACEH,75.0,67.0,12.0,104.0,NaN,2.0,192.0,108.0,124.0,73.0
1,SUMATERA UTARA,1146.0,2300.0,1488.0,281.0,209.0,298.0,472.0,571.0,2119.0,1498.0
2,JAMBI,95.0,NaN,29.0,51.0,12.0,137.0,232.0,234.0,164.0,39.0
3,SUMATERA SELATAN,265.0,134.0,715.0,560.0,253.0,358.0,290.0,3510.0,496.0,149.0
4,BENGKULU,57.0,42.0,245.0,121.0,305.0,301.0,116.0,100.0,216.0,50.0
5,LAMPUNG,157.0,34.0,210.0,653.0,243.0,408.0,569.0,384.0,421.0,492.0
6,KEP. RIAU,163.0,201.0,209.0,187.0,233.0,162.0,NaN,351.0,732.0,419.0
7,JAWA BARAT,525.0,1119.0,489.0,1061.0,1186.0,3008.0,4112.0,6354.0,2823.0,3393.0
8,JAWA TENGAH,696.0,751.0,1576.0,892.0,430.0,1726.0,1399.0,1326.0,829.0,162.0
9,JAWA TIMUR,988.0,2053.0,689.0,1790.0,716.0,482.0,865.0,1402.0,913.0,731.0


In [19]:
data_253.to_excel("data_253.xlsx", sheet_name= "Penumpang_Tiba_Pelabuhan", index=False)

Gini Rasio Provinsi Bali Menurut Kabupaten/Kota (41)

In [21]:
data_41 = getDataByVarId (domain='5100',varId='41')
data_41

,verval_val,vervar_label,var_val,turvar_val,turvar_label,tahun_val,tahun_label,turtahun_val,turtahun_label,id,content_val,content_label
0,1,Kab. Jembrana,41,0,Tidak ada,124,2024,0,Tahun,14101240,14101240,0.2884
1,2,Kab. Tabanan,41,0,Tidak ada,124,2024,0,Tahun,24101240,24101240,0.3007
2,3,Kab. Badung,41,0,Tidak ada,124,2024,0,Tahun,34101240,34101240,0.2808
3,4,Kab. Gianyar,41,0,Tidak ada,124,2024,0,Tahun,44101240,44101240,0.2860
4,5,Kab. Klungkung,41,0,Tidak ada,124,2024,0,Tahun,54101240,54101240,0.3071
...,...,...,...,...,...,...,...,...,...,...,...,...
5,6,Kab. Bangli,41,0,Tidak ada,115,2015,0,Tahun,64101150,64101150,0.3838
6,7,Kab. Karangasem,41,0,Tidak ada,115,2015,0,Tahun,74101150,74101150,0.3089
7,8,Kab. Buleleng,41,0,Tidak ada,115,2015,0,Tahun,84101150,84101150,0.3373
8,9,Kota Denpasar,41,0,Tidak ada,115,2015,0,Tahun,94101150,94101150,0.3589


In [23]:
data_41 = data_41[['verval_val','vervar_label','tahun_label','content_label']].copy()
data_41["vervar_label"] = data_41["vervar_label"].str.replace(r"<.*?>", "", regex=True)

data_tb = data_41.rename(columns={
    'vervar_label'   : 'Kabupaten/Kota',
    'tahun_label'    : 'Tahun',
    'content_label'  : 'Nilai'
})

api_order = data_41['vervar_label'].drop_duplicates().tolist()
data_tb['Kabupaten/Kota'] = pd.Categorical(data_tb['Kabupaten/Kota'], categories=api_order, ordered=True)

data_tb = data_tb.pivot_table(
    index=['Kabupaten/Kota'],
    columns=['Tahun'],
    values='Nilai'
)

data_tb.columns = [
    '_'.join([str(c) for c in col if c]).strip() if isinstance(col, tuple) else str(col) 
    for col in data_tb.columns.values
]

data_tb = data_tb.reset_index()

data_tb = data_tb.applymap(
    lambda x: int(x) if isinstance(x, (int, float)) and x.is_integer() else x
)

data_tb


C:\Users\user\AppData\Local\Temp\ipykernel_19036\1022704436.py:13: FutureWarning: The default value of observed=False is deprecated and will change to observed=True in a future version of pandas. Specify observed=False to silence this warning and retain the current behavior
  data_tb = data_tb.pivot_table(
C:\Users\user\AppData\Local\Temp\ipykernel_19036\1022704436.py:26: FutureWarning: DataFrame.applymap has been deprecated. Use DataFrame.map instead.
  data_tb = data_tb.applymap(


,Kabupaten/Kota,2015,2016,2017,2018,2019,2020,2021,2022,2023,2024
0,Kab. Jembrana,0.3079,0.3627,0.3170,0.3272,0.2903,0.3531,0.317,0.326,0.326,0.2884
1,Kab. Tabanan,0.3557,0.3444,0.3108,0.3249,0.3080,0.3239,0.311,0.299,0.347,0.3007
2,Kab. Badung,0.3147,0.3151,0.3190,0.3392,0.3244,0.3167,0.332,0.316,0.280,0.2808
3,Kab. Gianyar,0.3249,0.3049,0.2682,0.3050,0.2866,0.3171,0.333,0.307,0.317,0.2860
4,Kab. Klungkung,0.3695,0.3601,0.3714,0.3915,0.3847,0.3575,0.346,0.351,0.338,0.3071
5,Kab. Bangli,0.3838,0.3537,0.3005,0.3126,0.2744,0.2829,0.285,0.288,0.282,0.3021
6,Kab. Karangasem,0.3089,0.2930,0.3232,0.3393,0.3125,0.3268,0.318,0.297,0.337,0.2833
7,Kab. Buleleng,0.3373,0.3360,0.3086,0.3446,0.2847,0.2851,0.282,0.281,0.314,0.2785
8,Kota Denpasar,0.3589,0.3307,0.3414,0.3422,0.3473,0.3301,0.375,0.368,0.342,0.3413
9,Provinsi Bali,0.3768,0.3663,0.3840,0.3774,0.3662,0.3690,0.378,0.363,0.362,0.3611


In [24]:
data_41.to_excel("data_41.xlsx", sheet_name= "Gini_Rasio", index=False)

Indeks Kedalaman Kemiskinan Provinsi Bali Menurut Klasifikasi Daerah (66) ada slh

In [25]:
data_66 = getDataByVarId (domain='5100',varId='66')
data_66

,verval_val,vervar_label,var_val,turvar_val,turvar_label,tahun_val,tahun_label,turtahun_val,turtahun_label,id,content_val,content_label
0,1,Perkotaan,66,164,Maret,125,2025,0,Tahun,1661641250,1661641250,0.564
2,2,Perdesaan,66,164,Maret,125,2025,0,Tahun,2661641250,2661641250,0.551
4,3,Perkotaan + Perdesaan,66,164,Maret,125,2025,0,Tahun,3661641250,3661641250,0.560
0,1,Perkotaan,66,164,Maret,124,2024,0,Tahun,1661641240,1661641240,0.428
1,1,Perkotaan,66,165,September,124,2024,0,Tahun,1661651240,1661651240,0.399
...,...,...,...,...,...,...,...,...,...,...,...,...
1,1,Perkotaan,66,165,September,116,2016,0,Tahun,1661651160,1661651160,0.395
2,2,Perdesaan,66,164,Maret,116,2016,0,Tahun,2661641160,2661641160,0.616
3,2,Perdesaan,66,165,September,116,2016,0,Tahun,2661651160,2661651160,0.759
4,3,Perkotaan + Perdesaan,66,164,Maret,116,2016,0,Tahun,3661641160,3661641160,0.511


In [36]:
data_66 = data_66[['verval_val','vervar_label','turvar_label','tahun_label','content_label']].copy()
data_66["vervar_label"] = data_66["vervar_label"].str.replace(r"<.*?>", "", regex=True)

data_tb = data_66.rename(columns={
    'vervar_label'   : 'Indeks Kemiskinan',
    'turvar_label'   : 'Bulan',
    'tahun_label'    : 'Tahun',
    'content_label'  : 'Nilai'
})

api_order = data_66['vervar_label'].drop_duplicates().tolist()
data_tb['Indeks Kemiskinan'] = pd.Categorical(data_tb['Indeks Kemiskinan'], categories=api_order, ordered=True)

data_tb = data_tb.pivot_table(
    index=['Indeks Kemiskinan'],
    columns=['Tahun'],
    values='Nilai'
)

data_tb.columns = [
    '_'.join([str(c) for c in col if c]).strip() if isinstance(col, tuple) else str(col) 
    for col in data_tb.columns.values
]

data_tb = data_tb.reset_index()

data_tb = data_tb.applymap(
    lambda x: int(x) if isinstance(x, (int, float)) and x.is_integer() else x
)

data_tb


C:\Users\user\AppData\Local\Temp\ipykernel_19036\3604335966.py:14: FutureWarning: The default value of observed=False is deprecated and will change to observed=True in a future version of pandas. Specify observed=False to silence this warning and retain the current behavior
  data_tb = data_tb.pivot_table(
C:\Users\user\AppData\Local\Temp\ipykernel_19036\3604335966.py:27: FutureWarning: DataFrame.applymap has been deprecated. Use DataFrame.map instead.
  data_tb = data_tb.applymap(


,Indeks Kemiskinan,2016,2017,2018,2019,2020,2021,2022,2023,2024,2025
0,Perkotaan,0.4225,0.5320,0.474,0.5025,0.509,0.6815,0.5320,0.480,0.4135,0.564
1,Perdesaan,0.6875,0.7695,0.856,0.5565,0.700,0.8155,0.7495,0.740,0.6180,0.551
2,Perkotaan + Perdesaan,0.5205,0.6165,0.601,0.5195,0.567,0.7205,0.5940,0.553,0.4690,0.560


In [28]:
data_66.to_excel("data_66.xlsx", sheet_name= "Indeks_Kedalaman_Kemiskinan", index=False)

Indeks Keparahan Kemiskinan Provinsi Bali Menurut Kabupaten/Kota (260)

In [29]:
data_260 = getDataByVarId (domain='5100',varId='260')
data_260

,verval_val,vervar_label,var_val,turvar_val,turvar_label,tahun_val,tahun_label,turtahun_val,turtahun_label,id,content_val,content_label
0,1,Kab. Jembrana,260,0,Tidak ada,124,2024,0,Tahun,126001240,126001240,0.05
1,2,Kab. Tabanan,260,0,Tidak ada,124,2024,0,Tahun,226001240,226001240,0.19
2,3,Kab. Badung,260,0,Tidak ada,124,2024,0,Tahun,326001240,326001240,0.07
3,4,Kab. Gianyar,260,0,Tidak ada,124,2024,0,Tahun,426001240,426001240,0.09
4,5,Kab. Klungkung,260,0,Tidak ada,124,2024,0,Tahun,526001240,526001240,0.03
...,...,...,...,...,...,...,...,...,...,...,...,...
5,6,Kab. Bangli,260,0,Tidak ada,115,2015,0,Tahun,626001150,626001150,0.17
6,7,Kab. Karangasem,260,0,Tidak ada,115,2015,0,Tahun,726001150,726001150,0.20
7,8,Kab. Buleleng,260,0,Tidak ada,115,2015,0,Tahun,826001150,826001150,0.14
8,9,Kota Denpasar,260,0,Tidak ada,115,2015,0,Tahun,926001150,926001150,0.04


In [30]:
data_260 = data_260[['verval_val','vervar_label','tahun_label','content_label']].copy()
data_260["vervar_label"] = data_260["vervar_label"].str.replace(r"<.*?>", "", regex=True)

data_tb = data_260.rename(columns={
    'vervar_label'   : 'Kabupaten/Kota',
    'tahun_label'    : 'Tahun',
    'content_label'  : 'Nilai'
})

api_order = data_260['vervar_label'].drop_duplicates().tolist()
data_tb['Kabupaten/Kota'] = pd.Categorical(data_tb['Kabupaten/Kota'], categories=api_order, ordered=True)

data_tb = data_tb.pivot_table(
    index=['Kabupaten/Kota'],
    columns=['Tahun'],
    values='Nilai'
)

data_tb.columns = [
    '_'.join([str(c) for c in col if c]).strip() if isinstance(col, tuple) else str(col) 
    for col in data_tb.columns.values
]

data_tb = data_tb.reset_index()

data_tb = data_tb.applymap(
    lambda x: int(x) if isinstance(x, (int, float)) and x.is_integer() else x
)

data_tb


C:\Users\user\AppData\Local\Temp\ipykernel_19036\1704495062.py:13: FutureWarning: The default value of observed=False is deprecated and will change to observed=True in a future version of pandas. Specify observed=False to silence this warning and retain the current behavior
  data_tb = data_tb.pivot_table(
C:\Users\user\AppData\Local\Temp\ipykernel_19036\1704495062.py:26: FutureWarning: DataFrame.applymap has been deprecated. Use DataFrame.map instead.
  data_tb = data_tb.applymap(


,Kabupaten/Kota,2015,2016,2017,2018,2019,2020,2021,2022,2023,2024
0,Kab. Jembrana,0.19,0.08,0.19,0.09,0.11,0.06,0.12,0.15,0.04,0.05
1,Kab. Tabanan,0.09,0.12,0.13,0.10,0.03,0.10,0.22,0.13,0.09,0.19
2,Kab. Badung,0.02,0.05,0.05,0.07,0.03,0.07,0.10,0.05,0.01,0.07
3,Kab. Gianyar,0.10,0.07,0.11,0.08,0.03,0.11,0.09,0.21,0.06,0.09
4,Kab. Klungkung,0.22,0.13,0.04,0.21,0.13,0.17,0.25,0.08,0.09,0.03
5,Kab. Bangli,0.17,0.12,0.10,0.10,0.02,0.04,0.11,0.15,0.17,0.23
6,Kab. Karangasem,0.20,0.09,0.18,0.22,0.14,0.06,0.08,0.18,0.09,0.10
7,Kab. Buleleng,0.14,0.15,0.14,0.13,0.14,0.08,0.14,0.09,0.08,0.14
8,Kota Denpasar,0.04,0.03,0.12,0.07,0.06,0.05,0.06,0.05,0.10,0.08
9,Provinsi Bali,0.14,0.09,0.16,0.18,0.11,0.10,0.15,0.13,0.10,0.09


In [31]:
data_260.to_excel("data_260.xlsx", sheet_name= "Indeks_Keparahan_Kemiskinan", index=False)

Jumlah Penduduk Miskin Provinsi Bali Menurut Kabupaten/Kota (261)

In [32]:
data_261 = getDataByVarId (domain='5100',varId='261')
data_261

,verval_val,vervar_label,var_val,turvar_val,turvar_label,tahun_val,tahun_label,turtahun_val,turtahun_label,id,content_val,content_label
0,1,Kab. Jembrana,261,0,Tidak ada,124,2024,0,Tahun,126101240,126101240,12.90
1,2,Kab. Tabanan,261,0,Tidak ada,124,2024,0,Tahun,226101240,226101240,20.16
2,3,Kab. Badung,261,0,Tidak ada,124,2024,0,Tahun,326101240,326101240,16.87
3,4,Kab. Gianyar,261,0,Tidak ada,124,2024,0,Tahun,426101240,426101240,21.45
4,5,Kab. Klungkung,261,0,Tidak ada,124,2024,0,Tahun,526101240,526101240,9.68
...,...,...,...,...,...,...,...,...,...,...,...,...
5,6,Kab. Bangli,261,0,Tidak ada,115,2015,0,Tahun,626101150,626101150,12.74
6,7,Kab. Karangasem,261,0,Tidak ada,115,2015,0,Tahun,726101150,726101150,30.33
7,8,Kab. Buleleng,261,0,Tidak ada,115,2015,0,Tahun,826101150,826101150,43.43
8,9,Kota Denpasar,261,0,Tidak ada,115,2015,0,Tahun,926101150,926101150,20.94


In [33]:
data_261 = data_261[['verval_val','vervar_label','tahun_label','content_label']].copy()
data_261["vervar_label"] = data_261["vervar_label"].str.replace(r"<.*?>", "", regex=True)

data_tb = data_261.rename(columns={
    'vervar_label'   : 'Kabupaten/Kota',
    'tahun_label'    : 'Tahun',
    'content_label'  : 'Nilai'
})

api_order = data_261['vervar_label'].drop_duplicates().tolist()
data_tb['Kabupaten/Kota'] = pd.Categorical(data_tb['Kabupaten/Kota'], categories=api_order, ordered=True)

data_tb = data_tb.pivot_table(
    index=['Kabupaten/Kota'],
    columns=['Tahun'],
    values='Nilai'
)

data_tb.columns = [
    '_'.join([str(c) for c in col if c]).strip() if isinstance(col, tuple) else str(col) 
    for col in data_tb.columns.values
]

data_tb = data_tb.reset_index()

data_tb = data_tb.applymap(
    lambda x: int(x) if isinstance(x, (int, float)) and x.is_integer() else x
)

data_tb


C:\Users\user\AppData\Local\Temp\ipykernel_19036\42977362.py:13: FutureWarning: The default value of observed=False is deprecated and will change to observed=True in a future version of pandas. Specify observed=False to silence this warning and retain the current behavior
  data_tb = data_tb.pivot_table(
C:\Users\user\AppData\Local\Temp\ipykernel_19036\42977362.py:26: FutureWarning: DataFrame.applymap has been deprecated. Use DataFrame.map instead.
  data_tb = data_tb.applymap(


,Kabupaten/Kota,2015,2016,2017,2018,2019,2020,2021,2022,2023,2024
0,Kab. Jembrana,15.83,14.53,14.78,14.35,13.55,12.60,14.24,15.00,14.12,12.90
1,Kab. Tabanan,24.05,21.90,21.66,19.77,18.74,19.11,23.11,23.46,21.42,20.16
2,Kab. Badung,14.40,12.91,13.16,12.97,11.89,13.75,18.52,18.28,17.01,16.87
3,Kab. Gianyar,22.89,22.13,22.42,21.26,19.85,21.01,25.36,24.74,23.76,21.45
4,Kab. Klungkung,12.11,11.21,11.15,10.43,9.66,8.76,10.19,10.89,10.22,9.68
5,Kab. Bangli,12.74,11.66,11.76,11.05,10.08,9.56,11.68,12.17,12.24,11.79
6,Kab. Karangasem,30.33,27.12,27.02,26.02,25.99,24.69,28.52,29.45,27.83,27.76
7,Kab. Buleleng,43.43,37.55,37.48,35.20,34.26,35.25,40.92,41.68,39.52,36.55
8,Kota Denpasar,20.94,19.17,20.70,20.72,19.83,20.48,29.41,30.02,27.69,27.27
9,Provinsi Bali,196.71,178.18,180.13,171.76,163.85,165.19,201.97,205.68,193.78,184.43


In [34]:
data_261.to_excel("data_261.xlsx", sheet_name= "Jumlah_Penduduk_Miskin", index=False)

Indeks Keparahan Kemiskinan Provinsi Bali Menurut Klasifikasi Daerah (262)

In [35]:
data_262 = getDataByVarId (domain='5100',varId='262')
data_262

,verval_val,vervar_label,var_val,turvar_val,turvar_label,tahun_val,tahun_label,turtahun_val,turtahun_label,id,content_val,content_label
0,1,Perkotaan,262,164,Maret,125,2025,0,Tahun,12621641250,12621641250,0.128
2,2,Perdesaan,262,164,Maret,125,2025,0,Tahun,22621641250,22621641250,0.086
4,3,Perkotaan + Perdesaan,262,164,Maret,125,2025,0,Tahun,32621641250,32621641250,0.117
0,1,Perkotaan,262,164,Maret,124,2024,0,Tahun,12621641240,12621641240,0.074
1,1,Perkotaan,262,165,September,124,2024,0,Tahun,12621651240,12621651240,0.057
...,...,...,...,...,...,...,...,...,...,...,...,...
1,1,Perkotaan,262,165,September,116,2016,0,Tahun,12621651160,12621651160,0.064
2,2,Perdesaan,262,164,Maret,116,2016,0,Tahun,22621641160,22621641160,0.118
3,2,Perdesaan,262,165,September,116,2016,0,Tahun,22621651160,22621651160,0.178
4,3,Perkotaan + Perdesaan,262,164,Maret,116,2016,0,Tahun,32621641160,32621641160,0.093


In [40]:
data_262 = data_262[['verval_val','vervar_label','turvar_label','tahun_label','content_label']].copy()
data_262["vervar_label"] = data_262["vervar_label"].str.replace(r"<.*?>", "", regex=True)

data_tb = data_262.rename(columns={
    'vervar_label'   : 'Indeks Keparahan Kemiskinan',
    'turvar_label'   : 'Bulan',
    'tahun_label'    : 'Tahun',
    'content_label'  : 'Nilai'
})

api_order = data_262['vervar_label'].drop_duplicates().tolist()
data_tb['Indeks Keparahan Kemiskinan'] = pd.Categorical(data_tb['Indeks Keparahan Kemiskinan'], categories=api_order, ordered=True)

data_tb = data_tb.pivot_table(
    index=['Indeks Keparahan Kemiskinan'],
    columns=['Tahun', 'Bulan'],
    values='Nilai'
)

data_tb.columns = [
    '_'.join([str(c) for c in col if c]).strip() if isinstance(col, tuple) else str(col) 
    for col in data_tb.columns.values
]

data_tb = data_tb.reset_index()

data_tb = data_tb.applymap(
    lambda x: int(x) if isinstance(x, (int, float)) and x.is_integer() else x
)

data_tb


C:\Users\user\AppData\Local\Temp\ipykernel_19036\2701252377.py:14: FutureWarning: The default value of observed=False is deprecated and will change to observed=True in a future version of pandas. Specify observed=False to silence this warning and retain the current behavior
  data_tb = data_tb.pivot_table(
C:\Users\user\AppData\Local\Temp\ipykernel_19036\2701252377.py:27: FutureWarning: DataFrame.applymap has been deprecated. Use DataFrame.map instead.
  data_tb = data_tb.applymap(


,Indeks Keparahan Kemiskinan,2016_Maret,2016_September,2017_Maret,2017_September,2018_Maret,2018_September,2019_Maret,2019_September,2020_Maret,2020_September,2021_Maret,2021_September,2022_Maret,2022_September,2023_Maret,2024_Maret,2024_September,2025_Maret
0,Perkotaan,0.078,0.064,0.142,0.124,0.124,0.097,0.107,0.114,0.094,0.114,0.154,0.148,0.137,0.075,0.086,0.074,0.057,0.128
1,Perdesaan,0.118,0.178,0.190,0.112,0.283,0.153,0.130,0.079,0.123,0.143,0.156,0.226,0.109,0.170,0.142,0.135,0.085,0.086
2,Perkotaan + Perdesaan,0.093,0.106,0.160,0.120,0.178,0.115,0.114,0.103,0.103,0.123,0.154,0.171,0.129,0.102,0.102,0.090,0.065,0.117


In [44]:
data_262.to_excel("data_262.xlsx", sheet_name= "Indeks_Keparahan_Kemiskinan", index=False)

Garis Kemiskinan Makanan Provinsi Bali Menurut Klasifikasi Daerah (263)

In [7]:
data_263 = getDataByVarId (domain='5100',varId='263')
data_263

,verval_val,vervar_label,var_val,turvar_val,turvar_label,tahun_val,tahun_label,turtahun_val,turtahun_label,id,content_val,content_label
0,1,Perkotaan,263,164,Maret,125,2025,0,Tahun,12631641250,12631641250,434774.0
2,2,Perdesaan,263,164,Maret,125,2025,0,Tahun,22631641250,22631641250,376542.0
4,3,Perkotaan + Perdesaan,263,164,Maret,125,2025,0,Tahun,32631641250,32631641250,419319.0
0,1,Perkotaan,263,164,Maret,124,2024,0,Tahun,12631641240,12631641240,402771.0
1,1,Perkotaan,263,165,September,124,2024,0,Tahun,12631651240,12631651240,412279.0
...,...,...,...,...,...,...,...,...,...,...,...,...
1,1,Perkotaan,263,165,September,116,2016,0,Tahun,12631651160,12631651160,242429.0
2,2,Perdesaan,263,164,Maret,116,2016,0,Tahun,22631641160,22631641160,230108.0
3,2,Perdesaan,263,165,September,116,2016,0,Tahun,22631651160,22631651160,233243.0
4,3,Perkotaan + Perdesaan,263,164,Maret,116,2016,0,Tahun,32631641160,32631641160,234393.0


In [8]:
data_263 = data_263[['verval_val','vervar_label','turvar_label','tahun_label','content_label']].copy()
data_263["vervar_label"] = data_263["vervar_label"].str.replace(r"<.*?>", "", regex=True)

data_tb = data_263.rename(columns={
    'vervar_label'   : 'Klasifikasi Daerah',
    'turvar_label'   : 'Bulan',
    'tahun_label'    : 'Tahun',
    'content_label'  : 'Nilai'
})

api_order = data_263['vervar_label'].drop_duplicates().tolist()
data_tb['Klasifikasi Daerah'] = pd.Categorical(data_tb['Klasifikasi Daerah'], categories=api_order, ordered=True)

data_tb = data_tb.pivot_table(
    index=['Klasifikasi Daerah'],
    columns=['Tahun', 'Bulan'],
    values='Nilai'
)

data_tb.columns = [
    '_'.join([str(c) for c in col if c]).strip() if isinstance(col, tuple) else str(col) 
    for col in data_tb.columns.values
]

data_tb = data_tb.reset_index()

data_tb = data_tb.applymap(
    lambda x: int(x) if isinstance(x, (int, float)) and x.is_integer() else x
)

data_tb


C:\Users\user\AppData\Local\Temp\ipykernel_17372\4175666268.py:14: FutureWarning: The default value of observed=False is deprecated and will change to observed=True in a future version of pandas. Specify observed=False to silence this warning and retain the current behavior
  data_tb = data_tb.pivot_table(
C:\Users\user\AppData\Local\Temp\ipykernel_17372\4175666268.py:27: FutureWarning: DataFrame.applymap has been deprecated. Use DataFrame.map instead.
  data_tb = data_tb.applymap(


,Klasifikasi Daerah,2016_Maret,2016_September,2017_Maret,2017_September,2018_Maret,2018_September,2019_Maret,2019_September,2020_Maret,2020_September,2021_Maret,2021_September,2022_Maret,2022_September,2023_Maret,2024_Maret,2024_September,2025_Maret
0,Perkotaan,237835,242429,252294,252298,263613,268910,278769,292350,307459,309933,321245,325747,346903,367036,375253,402771,412279,434774
1,Perdesaan,230108,233243,245927,251307,264685,267059,269879,271679,280900,284541,291311,302550,318169,334827,340646,362933,364979,376542
2,Perkotaan + Perdesaan,234393,238822,249883,251921,263995,268275,275834,285920,298945,302154,312020,319095,338417,357640,365437,391618,399291,419319


In [12]:
data_263 = data_263.applymap(
    lambda x: f"{int(x):,}".replace(",", ".") if isinstance(x, (int, float)) and not pd.isna(x) else x
)

data_tb


C:\Users\user\AppData\Local\Temp\ipykernel_17372\210954472.py:1: FutureWarning: DataFrame.applymap has been deprecated. Use DataFrame.map instead.
  data_263 = data_263.applymap(


,verval_val,vervar_label,turvar_label,tahun_label,content_label
0,1,Perkotaan,Maret,2025,434.774
2,2,Perdesaan,Maret,2025,376.542
4,3,Perkotaan + Perdesaan,Maret,2025,419.319
0,1,Perkotaan,Maret,2024,402.771
1,1,Perkotaan,September,2024,412.279
...,...,...,...,...,...
1,1,Perkotaan,September,2016,242.429
2,2,Perdesaan,Maret,2016,230.108
3,2,Perdesaan,September,2016,233.243
4,3,Perkotaan + Perdesaan,Maret,2016,234.393


In [16]:
data_263.to_excel("data_263.xlsx", sheet_name= "Klasifikasi Daerah", index=False)

Garis Kemiskinan Non Makanan Provinsi Bali Menurut Klasifikasi Daerah (264)

In [17]:
data_264 = getDataByVarId (domain='5100',varId='264')
data_264

,verval_val,vervar_label,var_val,turvar_val,turvar_label,tahun_val,tahun_label,turtahun_val,turtahun_label,id,content_val,content_label
0,1,Perkotaan,264,164,Maret,125,2025,0,Tahun,12641641250,12641641250,196406.0
2,2,Perdesaan,264,164,Maret,125,2025,0,Tahun,22641641250,22641641250,166316.0
4,3,Perkotaan + Perdesaan,264,164,Maret,125,2025,0,Tahun,32641641250,32641641250,188528.0
0,1,Perkotaan,264,164,Maret,124,2024,0,Tahun,12641641240,12641641240,184174.0
1,1,Perkotaan,264,165,September,124,2024,0,Tahun,12641651240,12641651240,187981.0
...,...,...,...,...,...,...,...,...,...,...,...,...
1,1,Perkotaan,264,165,September,116,2016,0,Tahun,12641651160,12641651160,114998.0
2,2,Perdesaan,264,164,Maret,116,2016,0,Tahun,22641641160,22641641160,92552.0
3,2,Perdesaan,264,165,September,116,2016,0,Tahun,22641651160,22641651160,94790.0
4,3,Perkotaan + Perdesaan,264,164,Maret,116,2016,0,Tahun,32641641160,32641641160,104574.0


In [18]:
data_264 = data_264[['verval_val','vervar_label','turvar_label','tahun_label','content_label']].copy()
data_264["vervar_label"] = data_264["vervar_label"].str.replace(r"<.*?>", "", regex=True)

data_tb = data_264.rename(columns={
    'vervar_label'   : 'Klasifikasi Daerah',
    'turvar_label'   : 'Bulan',
    'tahun_label'    : 'Tahun',
    'content_label'  : 'Nilai'
})

api_order = data_264['vervar_label'].drop_duplicates().tolist()
data_tb['Klasifikasi Daerah'] = pd.Categorical(data_tb['Klasifikasi Daerah'], categories=api_order, ordered=True)

data_tb = data_tb.pivot_table(
    index=['Klasifikasi Daerah'],
    columns=['Tahun', 'Bulan'],
    values='Nilai'
)

data_tb.columns = [
    '_'.join([str(c) for c in col if c]).strip() if isinstance(col, tuple) else str(col) 
    for col in data_tb.columns.values
]

data_tb = data_tb.reset_index()

data_tb = data_tb.applymap(
    lambda x: int(x) if isinstance(x, (int, float)) and x.is_integer() else x
)

data_tb


C:\Users\user\AppData\Local\Temp\ipykernel_17372\882973450.py:14: FutureWarning: The default value of observed=False is deprecated and will change to observed=True in a future version of pandas. Specify observed=False to silence this warning and retain the current behavior
  data_tb = data_tb.pivot_table(
C:\Users\user\AppData\Local\Temp\ipykernel_17372\882973450.py:27: FutureWarning: DataFrame.applymap has been deprecated. Use DataFrame.map instead.
  data_tb = data_tb.applymap(


,Klasifikasi Daerah,2016_Maret,2016_September,2017_Maret,2017_September,2018_Maret,2018_September,2019_Maret,2019_September,2020_Maret,2020_September,2021_Maret,2021_September,2022_Maret,2022_September,2023_Maret,2024_Maret,2024_September,2025_Maret
0,Perkotaan,110736,114998,118320,118820,123953,125079,130026,131942,135611,141710,145944,148574,152669,163784,170443,184174,187981,196406
1,Perdesaan,92552,94790,99413,99519,108143,109674,113240,115867,120391,122774,126411,127328,131527,141203,147988,157305,161854,166316
2,Perkotaan + Perdesaan,104574,107576,111503,112143,118604,120176,124790,126986,130889,136013,140201,142437,146604,157398,164206,176892,181015,188528


In [22]:
data_264 = data_264.applymap(
    lambda x: f"{int(x):,}".replace(",", ".") 
              if isinstance(x, (int, float)) and not pd.isna(x) 
              else x
)

data_tb


C:\Users\user\AppData\Local\Temp\ipykernel_17372\2695651056.py:1: FutureWarning: DataFrame.applymap has been deprecated. Use DataFrame.map instead.
  data_264 = data_264.applymap(


,Klasifikasi Daerah,2016_Maret,2016_September,2017_Maret,2017_September,2018_Maret,2018_September,2019_Maret,2019_September,2020_Maret,2020_September,2021_Maret,2021_September,2022_Maret,2022_September,2023_Maret,2024_Maret,2024_September,2025_Maret
0,Perkotaan,110.736,114.998,118.320,118.820,123.953,125.079,130.026,131.942,135.611,141.710,145.944,148.574,152.669,163.784,170.443,184.174,187.981,196.406
1,Perdesaan,92.552,94.790,99.413,99.519,108.143,109.674,113.240,115.867,120.391,122.774,126.411,127.328,131.527,141.203,147.988,157.305,161.854,166.316
2,Perkotaan + Perdesaan,104.574,107.576,111.503,112.143,118.604,120.176,124.790,126.986,130.889,136.013,140.201,142.437,146.604,157.398,164.206,176.892,181.015,188.528


In [23]:
data_264.to_excel("data_264.xlsx", sheet_name= "Klasifikasi Daerah", index=False)

Garis Kemiskinan Provinsi Bali Menurut Klasifikasi Daerah (265)

In [24]:
data_265 = getDataByVarId (domain='5100',varId='265')
data_265

,verval_val,vervar_label,var_val,turvar_val,turvar_label,tahun_val,tahun_label,turtahun_val,turtahun_label,id,content_val,content_label
0,1,Perkotaan,265,164,Maret,125,2025,0,Tahun,12651641250,12651641250,631180.0
2,2,Perdesaan,265,164,Maret,125,2025,0,Tahun,22651641250,22651641250,542858.0
4,3,Perkotaan + Perdesaan,265,164,Maret,125,2025,0,Tahun,32651641250,32651641250,607847.0
0,1,Perkotaan,265,164,Maret,124,2024,0,Tahun,12651641240,12651641240,586945.0
1,1,Perkotaan,265,165,September,124,2024,0,Tahun,12651651240,12651651240,600260.0
...,...,...,...,...,...,...,...,...,...,...,...,...
1,1,Perkotaan,265,165,September,116,2016,0,Tahun,12651651160,12651651160,357427.0
2,2,Perdesaan,265,164,Maret,116,2016,0,Tahun,22651641160,22651641160,322660.0
3,2,Perdesaan,265,165,September,116,2016,0,Tahun,22651651160,22651651160,328033.0
4,3,Perkotaan + Perdesaan,265,164,Maret,116,2016,0,Tahun,32651641160,32651641160,338967.0


In [25]:
data_265 = data_265[['verval_val','vervar_label','turvar_label','tahun_label','content_label']].copy()
data_265["vervar_label"] = data_265["vervar_label"].str.replace(r"<.*?>", "", regex=True)

data_tb = data_265.rename(columns={
    'vervar_label'   : 'Klasifikasi Daerah',
    'turvar_label'   : 'Bulan',
    'tahun_label'    : 'Tahun',
    'content_label'  : 'Nilai'
})

api_order = data_265['vervar_label'].drop_duplicates().tolist()
data_tb['Klasifikasi Daerah'] = pd.Categorical(data_tb['Klasifikasi Daerah'], categories=api_order, ordered=True)

data_tb = data_tb.pivot_table(
    index=['Klasifikasi Daerah'],
    columns=['Tahun', 'Bulan'],
    values='Nilai'
)

data_tb.columns = [
    '_'.join([str(c) for c in col if c]).strip() if isinstance(col, tuple) else str(col) 
    for col in data_tb.columns.values
]

data_tb = data_tb.reset_index()

data_tb = data_tb.applymap(
    lambda x: int(x) if isinstance(x, (int, float)) and x.is_integer() else x
)

data_tb


C:\Users\user\AppData\Local\Temp\ipykernel_17372\2786631140.py:14: FutureWarning: The default value of observed=False is deprecated and will change to observed=True in a future version of pandas. Specify observed=False to silence this warning and retain the current behavior
  data_tb = data_tb.pivot_table(
C:\Users\user\AppData\Local\Temp\ipykernel_17372\2786631140.py:27: FutureWarning: DataFrame.applymap has been deprecated. Use DataFrame.map instead.
  data_tb = data_tb.applymap(


,Klasifikasi Daerah,2016_Maret,2016_September,2017_Maret,2017_September,2018_Maret,2018_September,2019_Maret,2019_September,2020_Maret,2020_September,2021_Maret,2021_September,2022_Maret,2022_September,2023_Maret,2024_Maret,2024_September,2025_Maret
0,Perkotaan,348571,357427,370615,371118,387566,393989,408795,424292,443070,451634,467189,474322,499572,530820,545700,586945,600260,631180
1,Perdesaan,322660,328033,345342,350826,372828,376733,383118,387546,401291,407316,417722,429877,449697,476031,488634,520238,526833,542858
2,Perkotaan + Perdesaan,338967,346398,361387,364064,382598,388451,400624,412906,429834,438167,452221,461532,485022,515037,529643,568510,580306,607847


In [ ]:


# pastikan kolom Nilai numeric
data_265= data_265.apply(pd.to_numeric, errors="ignore")

# format angka ribuan pakai titik
data_265 = data_265.applymap(
    lambda x: f"{int(x):,}".replace(",", ".") 
              if isinstance(x, (int, float)) and not pd.isna(x) 
              else x
)

data_tb


C:\Users\user\AppData\Local\Temp\ipykernel_17372\1412264409.py:2: FutureWarning: errors='ignore' is deprecated and will raise in a future version. Use to_numeric without passing `errors` and catch exceptions explicitly instead
  data_265= data_265.apply(pd.to_numeric, errors="ignore")
C:\Users\user\AppData\Local\Temp\ipykernel_17372\1412264409.py:5: FutureWarning: DataFrame.applymap has been deprecated. Use DataFrame.map instead.
  data_265 = data_265.applymap(


,Klasifikasi Daerah,2016_Maret,2016_September,2017_Maret,2017_September,2018_Maret,2018_September,2019_Maret,2019_September,2020_Maret,2020_September,2021_Maret,2021_September,2022_Maret,2022_September,2023_Maret,2024_Maret,2024_September,2025_Maret
0,Perkotaan,348.571,357.427,370.615,371.118,387.566,393.989,408.795,424.292,443.070,451.634,467.189,474.322,499.572,530.820,545.700,586.945,600.260,631.180
1,Perdesaan,322.660,328.033,345.342,350.826,372.828,376.733,383.118,387.546,401.291,407.316,417.722,429.877,449.697,476.031,488.634,520.238,526.833,542.858
2,Perkotaan + Perdesaan,338.967,346.398,361.387,364.064,382.598,388.451,400.624,412.906,429.834,438.167,452.221,461.532,485.022,515.037,529.643,568.510,580.306,607.847


In [30]:
data_265.to_excel("data_265.xlsx", sheet_name= "Klasifikasi Daerah", index=False)

Gini Rasio Provinsi Bali Menurut Klasifikasi Daerah (320)

In [31]:
data_320 = getDataByVarId (domain='5100',varId='320')
data_320

,verval_val,vervar_label,var_val,turvar_val,turvar_label,tahun_val,tahun_label,turtahun_val,turtahun_label,id,content_val,content_label
0,1,Perkotaan,320,164,Maret,125,2025,0,Tahun,13201641250,13201641250,0.352
2,2,Perdesaan,320,164,Maret,125,2025,0,Tahun,23201641250,23201641250,0.277
4,3,Perkotaan + Perdesaan,320,164,Maret,125,2025,0,Tahun,33201641250,33201641250,0.353
0,1,Perkotaan,320,164,Maret,124,2024,0,Tahun,13201641240,13201641240,0.364
1,1,Perkotaan,320,165,September,124,2024,0,Tahun,13201651240,13201651240,0.352
...,...,...,...,...,...,...,...,...,...,...,...,...
1,1,Perkotaan,320,165,September,116,2016,0,Tahun,13201651160,13201651160,0.378
2,2,Perdesaan,320,164,Maret,116,2016,0,Tahun,23201641160,23201641160,0.329
3,2,Perdesaan,320,165,September,116,2016,0,Tahun,23201651160,23201651160,0.335
4,3,Perkotaan + Perdesaan,320,164,Maret,116,2016,0,Tahun,33201641160,33201641160,0.366


In [33]:
data_320 = data_320[['verval_val','vervar_label','turvar_label','tahun_label','content_label']].copy()
data_320["vervar_label"] = data_320["vervar_label"].str.replace(r"<.*?>", "", regex=True)

data_tb = data_320.rename(columns={
    'vervar_label'   : 'Klasifikasi Daerah',
    'turvar_label'   : 'Bulan',
    'tahun_label'    : 'Tahun',
    'content_label'  : 'Nilai'
})

api_order = data_320['vervar_label'].drop_duplicates().tolist()
data_tb['Klasifikasi Daerah'] = pd.Categorical(data_tb['Klasifikasi Daerah'], categories=api_order, ordered=True)

data_tb = data_tb.pivot_table(
    index=['Klasifikasi Daerah'],
    columns=['Tahun', 'Bulan'],
    values='Nilai'
)

data_tb.columns = [
    '_'.join([str(c) for c in col if c]).strip() if isinstance(col, tuple) else str(col) 
    for col in data_tb.columns.values
]

data_tb = data_tb.reset_index()

data_tb = data_tb.applymap(
    lambda x: int(x) if isinstance(x, (int, float)) and x.is_integer() else x
)

data_tb


C:\Users\user\AppData\Local\Temp\ipykernel_17372\1770428214.py:14: FutureWarning: The default value of observed=False is deprecated and will change to observed=True in a future version of pandas. Specify observed=False to silence this warning and retain the current behavior
  data_tb = data_tb.pivot_table(
C:\Users\user\AppData\Local\Temp\ipykernel_17372\1770428214.py:27: FutureWarning: DataFrame.applymap has been deprecated. Use DataFrame.map instead.
  data_tb = data_tb.applymap(


,Klasifikasi Daerah,2016_Maret,2016_September,2017_Maret,2017_September,2018_Maret,2018_September,2019_Maret,2019_September,2020_Maret,2020_September,2021_Maret,2021_September,2022_Maret,2022_September,2023_Maret,2024_Maret,2024_September,2025_Maret
0,Perkotaan,0.369,0.378,0.382,0.385,0.381,0.363,0.370,0.365,0.372,0.378,0.387,0.379,0.371,0.371,0.367,0.364,0.352,0.352
1,Perdesaan,0.329,0.335,0.325,0.302,0.317,0.310,0.313,0.306,0.298,0.304,0.301,0.302,0.294,0.282,0.290,0.287,0.285,0.277
2,Perkotaan + Perdesaan,0.366,0.374,0.384,0.379,0.377,0.364,0.366,0.370,0.369,0.369,0.378,0.375,0.363,0.362,0.362,0.361,0.348,0.353


In [34]:
data_320.to_excel("data_320.xlsx", sheet_name= "Klasifikasi Daerah", index=False)

Produksi Daging Unggas Provinsi Bali Menurut Kabupaten/Kota (206)

In [35]:
data_206 = getDataByVarId (domain='5100',varId='206')
data_206

,verval_val,vervar_label,var_val,turvar_val,turvar_label,tahun_val,tahun_label,turtahun_val,turtahun_label,id,content_val,content_label
0,1,Kab. Jembrana,206,105,Ayam,122,2022,0,Tahun,12061051220,12061051220,1373.00
1,1,Kab. Jembrana,206,106,Itik,122,2022,0,Tahun,12061061220,12061061220,34.00
2,2,Kab. Tabanan,206,105,Ayam,122,2022,0,Tahun,22061051220,22061051220,7048.00
3,2,Kab. Tabanan,206,106,Itik,122,2022,0,Tahun,22061061220,22061061220,51.00
4,3,Kab. Badung,206,105,Ayam,122,2022,0,Tahun,32061051220,32061051220,2872.00
...,...,...,...,...,...,...,...,...,...,...,...,...
15,8,Kab. Buleleng,206,106,Itik,113,2013,0,Tahun,82061061130,82061061130,23.14
16,9,Kota Denpasar,206,105,Ayam,113,2013,0,Tahun,92061051130,92061051130,57.00
17,9,Kota Denpasar,206,106,Itik,113,2013,0,Tahun,92061061130,92061061130,2.55
18,10,<b>Provinsi Bali</b>,206,105,Ayam,113,2013,0,Tahun,102061051130,102061051130,31619.29


In [36]:
data_206 = data_206[['verval_val','vervar_label','turvar_label','tahun_label','content_label']].copy()
data_206["vervar_label"] = data_206["vervar_label"].str.replace(r"<.*?>", "", regex=True)

data_tb = data_206.rename(columns={
    'vervar_label'   : 'Kabupaten/Kota',
    'turvar_label'   : 'Unggas',
    'tahun_label'    : 'Tahun',
    'content_label'  : 'Nilai'
})

api_order = data_206['vervar_label'].drop_duplicates().tolist()
data_tb['Kabupaten/Kota'] = pd.Categorical(data_tb['Kabupaten/Kota'], categories=api_order, ordered=True)

data_tb = data_tb.pivot_table(
    index=['Kabupaten/Kota'],
    columns=['Tahun', 'Unggas'],
    values='Nilai'
)

data_tb.columns = [
    '_'.join([str(c) for c in col if c]).strip() if isinstance(col, tuple) else str(col) 
    for col in data_tb.columns.values
]

data_tb = data_tb.reset_index()

data_tb = data_tb.applymap(
    lambda x: int(x) if isinstance(x, (int, float)) and x.is_integer() else x
)

data_tb


C:\Users\user\AppData\Local\Temp\ipykernel_17372\3101084534.py:14: FutureWarning: The default value of observed=False is deprecated and will change to observed=True in a future version of pandas. Specify observed=False to silence this warning and retain the current behavior
  data_tb = data_tb.pivot_table(
C:\Users\user\AppData\Local\Temp\ipykernel_17372\3101084534.py:27: FutureWarning: DataFrame.applymap has been deprecated. Use DataFrame.map instead.
  data_tb = data_tb.applymap(


,Kabupaten/Kota,2013_Ayam,2013_Itik,2014_Ayam,2014_Itik,2015_Ayam,2015_Itik,2016_Ayam,2016_Itik,2017_Ayam,...,2018_Ayam,2018_Itik,2019_Ayam,2019_Itik,2020_Ayam,2020_Itik,2021_Ayam,2021_Itik,2022_Ayam,2022_Itik
0,Kab. Jembrana,3250.02,18.24,5194.20,36.54,6079.26,40.70,11056.50,40.01,791.29,...,887.48,26.27,1980.85,29.97,1442,18,3757,84,1373,34
1,Kab. Tabanan,9115.12,31.55,18225.67,56.69,19033.79,61.27,26305.83,45.54,3176.90,...,3266.97,36.86,3433.67,32.39,3391,118,23253,72,7048,51
2,Kab. Badung,2571.20,30.42,4208.49,53.53,4442.13,50.03,7761.87,48.46,665.93,...,774.36,31.48,1276.46,36.28,1449,68,2804,60,2872,41
3,Kab. Gianyar,2452.81,51.41,3405.25,88.71,3325.82,77.16,6439.86,75.10,835.61,...,4137.76,77.66,2289.64,78.07,1750,248,7127,209,2148,232
4,Kab. Klungkung,2025.80,40.80,3658.33,43.01,3620.10,44.57,6285.42,52.16,734.90,...,584.65,22.08,714.83,35.48,189,36,1056,0,244,55
5,Kab. Bangli,7568.78,14.32,12995.40,22.94,11905.74,25.95,16411.77,19.11,1605.24,...,1682.72,18.26,3237.96,17.51,2778,92,7882,3,625,2
6,Kab. Karangasem,3185.10,16.05,4268.07,26.54,12787.38,23.59,11772.99,27.19,582.75,...,921.46,19.67,4056.90,16.49,1018,17,3415,281,1548,300
7,Kab. Buleleng,1393.46,23.14,1886.39,35.36,1517.01,38.48,2716.38,35.14,280.29,...,427.82,37.95,1256.22,39.99,1443,33,7794,17,892,14
8,Kota Denpasar,57.00,2.55,11.55,4.69,9.90,2.48,39.24,2.50,0.00,...,19.80,2.07,95.81,2.50,8251,3,12980,1,8268,3
9,Provinsi Bali,31619.29,228.48,53853.35,368.01,62721.13,364.23,88789.86,345.21,8672.91,...,12703.02,272.30,18342.34,288.68,21711,633,70068,727,25018,733


In [ ]:
# pastikan numeric dulu
data_206 = data_206.apply(pd.to_numeric, errors="ignore")

# format angka gaya Indonesia: ribuan titik, desimal koma
data_206 = data_206.applymap(
    lambda x: f"{x:,.2f}".replace(",", "X").replace(".", ",").replace("X", ".") 
              if isinstance(x, (int, float)) and not pd.isna(x) else x
)

data_tb


C:\Users\user\AppData\Local\Temp\ipykernel_17372\551574989.py:2: FutureWarning: errors='ignore' is deprecated and will raise in a future version. Use to_numeric without passing `errors` and catch exceptions explicitly instead
  data_tb = data_tb.apply(pd.to_numeric, errors="ignore")
C:\Users\user\AppData\Local\Temp\ipykernel_17372\551574989.py:5: FutureWarning: DataFrame.applymap has been deprecated. Use DataFrame.map instead.
  data_tb = data_tb.applymap(


,Kabupaten/Kota,2013_Ayam,2013_Itik,2014_Ayam,2014_Itik,2015_Ayam,2015_Itik,2016_Ayam,2016_Itik,2017_Ayam,...,2018_Ayam,2018_Itik,2019_Ayam,2019_Itik,2020_Ayam,2020_Itik,2021_Ayam,2021_Itik,2022_Ayam,2022_Itik
0,Kab. Jembrana,"3,250","18,00","5,194","36,00","6,079","40,00","11,056","40,00",791,...,887,"26,00","1,980","29,00","1,442","18,00","3,757","84,00","1,373","34,00"
1,Kab. Tabanan,"9,115","31,00","18,225","56,00","19,033","61,00","26,305","45,00","3,176",...,"3,266","36,00","3,433","32,00","3,391","118,00","23,253","72,00","7,048","51,00"
2,Kab. Badung,"2,571","30,00","4,208","53,00","4,442","50,00","7,761","48,00",665,...,774,"31,00","1,276","36,00","1,449","68,00","2,804","60,00","2,872","41,00"
3,Kab. Gianyar,"2,452","51,00","3,405","88,00","3,325","77,00","6,439","75,00",835,...,"4,137","77,00","2,289","78,00","1,750","248,00","7,127","209,00","2,148","232,00"
4,Kab. Klungkung,"2,025","40,00","3,658","43,00","3,620","44,00","6,285","52,00",734,...,584,"22,00",714,"35,00",189,"36,00","1,056","0,00",244,"55,00"
5,Kab. Bangli,"7,568","14,00","12,995","22,00","11,905","25,00","16,411","19,00","1,605",...,"1,682","18,00","3,237","17,00","2,778","92,00","7,882","3,00",625,"2,00"
6,Kab. Karangasem,"3,185","16,00","4,268","26,00","12,787","23,00","11,772","27,00",582,...,921,"19,00","4,056","16,00","1,018","17,00","3,415","281,00","1,548","300,00"
7,Kab. Buleleng,"1,393","23,00","1,886","35,00","1,517","38,00","2,716","35,00",280,...,427,"37,00","1,256","39,00","1,443","33,00","7,794","17,00",892,"14,00"
8,Kota Denpasar,57,"2,00",11,"4,00",9,"2,00",39,"2,00",0,...,19,"2,00",95,"2,00","8,251","3,00","12,980","1,00","8,268","3,00"
9,Provinsi Bali,"31,619","228,00","53,853","368,00","62,721","364,00","88,789","345,00","8,672",...,"12,703","272,00","18,342","288,00","21,711","633,00","70,068","727,00","25,018","733,00"


In [40]:
data_206.to_excel("data_206.xlsx", sheet_name= "DagingUnggas", index=False)

Produksi Telur Unggas Provinsi Bali Menurut Kabupaten/Kota (208)

In [42]:
data_208 = getDataByVarId (domain='5100',varId='208')
data_208

,verval_val,vervar_label,var_val,turvar_val,turvar_label,tahun_val,tahun_label,turtahun_val,turtahun_label,id,content_val,content_label
0,1,Kab. Jembrana,208,107,Telur Ayam Ras,122,2022,0,Tahun,12081071220,12081071220,698.00
1,1,Kab. Jembrana,208,108,Telur Ayam Buras,122,2022,0,Tahun,12081081220,12081081220,98.00
2,1,Kab. Jembrana,208,109,Telur Itik,122,2022,0,Tahun,12081091220,12081091220,128.00
3,2,Kab. Tabanan,208,107,Telur Ayam Ras,122,2022,0,Tahun,22081071220,22081071220,7024.00
4,2,Kab. Tabanan,208,108,Telur Ayam Buras,122,2022,0,Tahun,22081081220,22081081220,188.00
...,...,...,...,...,...,...,...,...,...,...,...,...
25,9,Kota Denpasar,208,108,Telur Ayam Buras,113,2013,0,Tahun,92081081130,92081081130,56.30
26,9,Kota Denpasar,208,109,Telur Itik,113,2013,0,Tahun,92081091130,92081091130,48.23
27,10,<b>Provinsi Bali</b>,208,107,Telur Ayam Ras,113,2013,0,Tahun,102081071130,102081071130,36590.01
28,10,<b>Provinsi Bali</b>,208,108,Telur Ayam Buras,113,2013,0,Tahun,102081081130,102081081130,2973.51


In [43]:
data_208 = data_208[['verval_val','vervar_label','turvar_label','tahun_label','content_label']].copy()
data_208["vervar_label"] = data_208["vervar_label"].str.replace(r"<.*?>", "", regex=True)

data_tb = data_208.rename(columns={
    'vervar_label'   : 'Kabupaten/Kota',
    'turvar_label'   : 'Jenis Telur',
    'tahun_label'    : 'Tahun',
    'content_label'  : 'Nilai'
})

api_order = data_208['vervar_label'].drop_duplicates().tolist()
data_tb['Kabupaten/Kota'] = pd.Categorical(data_tb['Kabupaten/Kota'], categories=api_order, ordered=True)

data_tb = data_tb.pivot_table(
    index=['Kabupaten/Kota'],
    columns=['Tahun', 'Jenis Telur'],
    values='Nilai'
)

data_tb.columns = [
    '_'.join([str(c) for c in col if c]).strip() if isinstance(col, tuple) else str(col) 
    for col in data_tb.columns.values
]

data_tb = data_tb.reset_index()

data_tb = data_tb.applymap(
    lambda x: int(x) if isinstance(x, (int, float)) and x.is_integer() else x
)

data_tb


C:\Users\user\AppData\Local\Temp\ipykernel_17372\560827108.py:14: FutureWarning: The default value of observed=False is deprecated and will change to observed=True in a future version of pandas. Specify observed=False to silence this warning and retain the current behavior
  data_tb = data_tb.pivot_table(
C:\Users\user\AppData\Local\Temp\ipykernel_17372\560827108.py:27: FutureWarning: DataFrame.applymap has been deprecated. Use DataFrame.map instead.
  data_tb = data_tb.applymap(


,Kabupaten/Kota,2013_Telur Ayam Buras,2013_Telur Ayam Ras,2013_Telur Itik,2014_Telur Ayam Buras,2014_Telur Ayam Ras,2014_Telur Itik,2015_Telur Ayam Buras,2015_Telur Ayam Ras,2015_Telur Itik,...,2019_Telur Itik,2020_Telur Ayam Buras,2020_Telur Ayam Ras,2020_Telur Itik,2021_Telur Ayam Buras,2021_Telur Ayam Ras,2021_Telur Itik,2022_Telur Ayam Buras,2022_Telur Ayam Ras,2022_Telur Itik
0,Kab. Jembrana,486.10,147.00,344.98,470.59,151.20,414.66,460.39,302.40,461.88,...,177.90,110.51,353.37,201.68,111.26,474.56,192.90,98,698,128
1,Kab. Tabanan,353.87,15963.82,596.69,354.06,14887.53,643.31,370.20,15770.93,695.30,...,192.26,217.23,7187.56,372.26,205.45,7164.05,345.28,188,7024,289
2,Kab. Badung,316.96,634.20,575.31,336.14,793.80,607.51,316.43,819.00,567.80,...,215.36,172.53,1546.76,446.72,174.14,3145.62,435.38,109,729,246
3,Kab. Gianyar,354.92,233.02,972.42,357.11,233.02,1006.71,317.03,490.06,875.68,...,463.46,360.64,2445.12,915.52,371.46,2386.98,883.07,341,2297,854
4,Kab. Klungkung,132.74,0.00,771.76,120.03,41.16,488.14,121.37,42.00,505.84,...,210.64,93.82,449.00,409.00,95.22,347.00,406.40,95,347,520
5,Kab. Bangli,297.04,11033.40,270.85,290.69,11916.24,260.30,304.17,14534.52,294.48,...,103.93,214.10,21434.28,187.14,230.50,15075.48,187.14,235,10387,186
6,Kab. Karangasem,420.45,8419.00,303.52,420.45,8419.00,301.18,395.47,8838.41,267.66,...,97.92,283.10,5186.79,192.31,302.29,5310.28,175.41,262,4671,3
7,Kab. Buleleng,555.13,159.57,437.60,562.05,159.70,401.26,553.93,189.45,436.63,...,237.39,525.20,583.35,358.37,535.40,673.89,314.40,253,607,1
8,Kota Denpasar,56.30,0.00,48.23,58.98,0.00,53.25,57.72,0.00,28.14,...,14.24,58.02,26.04,33.79,33.95,9.58,25.51,33,0,22
9,Provinsi Bali,2973.51,36590.01,4321.36,2970.10,36601.65,4176.32,2896.71,40986.77,4133.41,...,1713.10,2035.15,39212.27,3116.79,2059.67,34587.44,2965.49,1613,26759,2247


In [44]:
# ambil kolom sesuai API
data_tb = data_208[['verval_val','vervar_label','turvar_label','tahun_label','content_label']].copy()
data_tb["vervar_label"] = data_tb["vervar_label"].str.replace(r"<.*?>", "", regex=True)

data_tb = data_tb.rename(columns={
    'vervar_label'   : 'Kabupaten/Kota',
    'turvar_label'   : 'Jenis Telur',
    'tahun_label'    : 'Tahun',
    'content_label'  : 'Nilai'
})

# urutan sesuai API
api_order = data_208['vervar_label'].drop_duplicates().tolist()
data_tb['Kabupaten/Kota'] = pd.Categorical(data_tb['Kabupaten/Kota'], categories=api_order, ordered=True)

# pivot tanpa ubah tipe data
data_tb = data_tb.pivot_table(
    index=['Kabupaten/Kota'],
    columns=['Tahun','Jenis Telur'],
    values='Nilai',
    aggfunc='first'   # ambil nilai asli
)

# gabungkan multiindex kolom
data_tb.columns = ['_'.join([str(c) for c in col if c]) for col in data_tb.columns]
data_tb = data_tb.reset_index()

# --- format angka sesuai API (Indonesia style) ---
data_tb = data_tb.applymap(
    lambda x: f"{float(x):,.2f}".replace(",", "X").replace(".", ",").replace("X", ".") 
              if isinstance(x, (int, float, str)) and str(x).replace('.', '', 1).isdigit()
              else x
)

data_tb


C:\Users\user\AppData\Local\Temp\ipykernel_17372\2735232385.py:17: FutureWarning: The default value of observed=False is deprecated and will change to observed=True in a future version of pandas. Specify observed=False to silence this warning and retain the current behavior
  data_tb = data_tb.pivot_table(
C:\Users\user\AppData\Local\Temp\ipykernel_17372\2735232385.py:29: FutureWarning: DataFrame.applymap has been deprecated. Use DataFrame.map instead.
  data_tb = data_tb.applymap(


,Kabupaten/Kota,2013_Telur Ayam Buras,2013_Telur Ayam Ras,2013_Telur Itik,2014_Telur Ayam Buras,2014_Telur Ayam Ras,2014_Telur Itik,2015_Telur Ayam Buras,2015_Telur Ayam Ras,2015_Telur Itik,...,2019_Telur Itik,2020_Telur Ayam Buras,2020_Telur Ayam Ras,2020_Telur Itik,2021_Telur Ayam Buras,2021_Telur Ayam Ras,2021_Telur Itik,2022_Telur Ayam Buras,2022_Telur Ayam Ras,2022_Telur Itik
0,Kab. Jembrana,"486,10","147,00","344,98","470,59","151,20","414,66","460,39","302,40","461,88",...,"177,90","110,51","353,37","201,68","111,26","474,56","192,90","98,00","698,00","128,00"
1,Kab. Tabanan,"353,87","15.963,82","596,69","354,06","14.887,53","643,31","370,20","15.770,93","695,30",...,"192,26","217,23","7.187,56","372,26","205,45","7.164,05","345,28","188,00","7.024,00","289,00"
2,Kab. Badung,"316,96","634,20","575,31","336,14","793,80","607,51","316,43","819,00","567,80",...,"215,36","172,53","1.546,76","446,72","174,14","3.145,62","435,38","109,00","729,00","246,00"
3,Kab. Gianyar,"354,92","233,02","972,42","357,11","233,02","1.006,71","317,03","490,06","875,68",...,"463,46","360,64","2.445,12","915,52","371,46","2.386,98","883,07","341,00","2.297,00","854,00"
4,Kab. Klungkung,"132,74","0,00","771,76","120,03","41,16","488,14","121,37","42,00","505,84",...,"210,64","93,82","449,00","409,00","95,22","347,00","406,40","95,00","347,00","520,00"
5,Kab. Bangli,"297,04","11.033,40","270,85","290,69","11.916,24","260,30","304,17","14.534,52","294,48",...,"103,93","214,10","21.434,28","187,14","230,50","15.075,48","187,14","235,00","10.387,00","186,00"
6,Kab. Karangasem,"420,45","8.419,00","303,52","420,45","8.419,00","301,18","395,47","8.838,41","267,66",...,"97,92","283,10","5.186,79","192,31","302,29","5.310,28","175,41","262,00","4.671,00","3,00"
7,Kab. Buleleng,"555,13","159,57","437,60","562,05","159,70","401,26","553,93","189,45","436,63",...,"237,39","525,20","583,35","358,37","535,40","673,89","314,40","253,00","607,00","1,00"
8,Kota Denpasar,"56,30","0,00","48,23","58,98","0,00","53,25","57,72","0,00","28,14",...,"14,24","58,02","26,04","33,79","33,95","9,58","25,51","33,00","0,00","22,00"
9,Provinsi Bali,"2.973,51","36.590,01","4.321,36","2.970,10","36.601,65","4.176,32","2.896,71","40.986,77","4.133,41",...,"1.713,10","2.035,15","39.212,27","3.116,79","2.059,67","34.587,44","2.965,49","1.613,00","26.759,00","2.247,00"


In [45]:
data_208.to_excel("data_208.xlsx", sheet_name= "Jenis Telur", index=False)

Indeks Pembangunan Manusia Provinsi Bali Menurut Kabupaten/Kota (30)

In [70]:
data_30 = getDataByVarId (domain='5100',varId='30')
data_30

,verval_val,vervar_label,var_val,turvar_val,turvar_label,tahun_val,tahun_label,turtahun_val,turtahun_label,id,content_val,content_label
0,1,Kab. Jembrana,30,0,Tidak ada,124,2024,0,Tahun,13001240,13001240,74.60
1,2,Kab. Tabanan,30,0,Tidak ada,124,2024,0,Tahun,23001240,23001240,78.42
2,3,Kab. Badung,30,0,Tidak ada,124,2024,0,Tahun,33001240,33001240,83.87
3,4,Kab. Gianyar,30,0,Tidak ada,124,2024,0,Tahun,43001240,43001240,79.82
4,5,Kab. Klungkung,30,0,Tidak ada,124,2024,0,Tahun,53001240,53001240,74.04
...,...,...,...,...,...,...,...,...,...,...,...,...
5,6,Kab. Bangli,30,0,Tidak ada,115,2015,0,Tahun,63001150,63001150,66.24
6,7,Kab. Karangasem,30,0,Tidak ada,115,2015,0,Tahun,73001150,73001150,64.68
7,8,Kab. Buleleng,30,0,Tidak ada,115,2015,0,Tahun,83001150,83001150,70.03
8,9,Kota Denpasar,30,0,Tidak ada,115,2015,0,Tahun,93001150,93001150,82.24


In [71]:
# --- ambil & bersihkan untuk urutan asli ---
data_30 = data_30[['vervar_label','tahun_label','content_label']].copy()
data_30['Kabupaten/Kota'] = (data_30['vervar_label'].astype(str)
                          .str.replace(r'<.*?>','', regex=True)
                          .str.replace(r'\s+',' ', regex=True)
                          .str.strip())
order = data_30['Kabupaten/Kota'].drop_duplicates().tolist()

# --- siapkan untuk pivot ---
df = data_30.rename(columns={'tahun_label':'tahun','content_label':'nilai'})[['Kabupaten/Kota','tahun','nilai']]
df['tahun'] = pd.to_numeric(df['tahun'], errors='coerce')
df = df.dropna(subset=['tahun']).copy()
df['tahun'] = df['tahun'].astype(int)

# --- PIVOT: kolom = tahun (MELEBAR) ---
wide = df.pivot_table(index='Kabupaten/Kota', columns='tahun', values='nilai', aggfunc='first', sort=False)

# urut kolom (tahun naik) + baris sesuai urutan awal
wide = wide.reindex(sorted(wide.columns), axis=1)
wide.index = pd.CategoricalIndex(wide.index, categories=[k for k in order if k in wide.index], ordered=True)
wide = wide.sort_index()

In [72]:
wide

tahun,2015,2016,2017,2018,2019,2020,2021,2022,2023,2024
Kabupaten/Kota,,,,,,,,,,
Kab. Jembrana,69.66,70.38,70.72,71.65,72.35,72.36,72.75,73.58,74.04,74.60
Kab. Tabanan,73.54,74.19,74.86,75.45,76.16,76.17,76.45,76.75,77.43,78.42
Kab. Badung,78.86,79.80,80.54,80.87,81.59,81.60,81.83,82.13,83.08,83.87
Kab. Gianyar,75.03,75.70,76.09,76.71,77.14,77.36,77.70,78.39,79.24,79.82
Kab. Klungkung,68.98,69.31,70.13,70.90,71.71,71.73,71.75,72.55,73.11,74.04
Kab. Bangli,66.24,67.03,68.24,68.96,69.35,69.36,69.37,70.26,70.79,71.34
Kab. Karangasem,64.68,65.23,65.57,66.49,67.34,67.35,67.36,68.28,68.91,69.74
Kab. Buleleng,70.03,70.65,71.11,71.70,72.30,72.55,72.56,73.45,73.97,74.52
Kota Denpasar,82.24,82.58,83.01,83.30,83.68,83.93,84.03,84.37,84.73,85.22


In [73]:
wide.reset_index().to_excel("data_30.xlsx", index=False, engine="openpyxl")

Umur Harapan Hidup Saat Lahir Provinsi Bali Menurut Kabupaten/Kota (358)

In [66]:
data_358 = getDataByVarId (domain='5100',varId='358')
data_358

,verval_val,vervar_label,var_val,turvar_val,turvar_label,tahun_val,tahun_label,turtahun_val,turtahun_label,id,content_val,content_label
0,1,Kab. Jembrana,358,0,Tidak ada,124,2024,0,Tahun,135801240,135801240,73.51
1,2,Kab. Tabanan,358,0,Tidak ada,124,2024,0,Tahun,235801240,235801240,74.80
2,3,Kab. Badung,358,0,Tidak ada,124,2024,0,Tahun,335801240,335801240,76.18
3,4,Kab. Gianyar,358,0,Tidak ada,124,2024,0,Tahun,435801240,435801240,74.86
4,5,Kab. Klungkung,358,0,Tidak ada,124,2024,0,Tahun,535801240,535801240,72.50
...,...,...,...,...,...,...,...,...,...,...,...,...
5,6,Kab. Bangli,358,0,Tidak ada,115,2015,0,Tahun,635801150,635801150,69.54
6,7,Kab. Karangasem,358,0,Tidak ada,115,2015,0,Tahun,735801150,735801150,69.48
7,8,Kab. Buleleng,358,0,Tidak ada,115,2015,0,Tahun,835801150,835801150,70.81
8,9,Kota Denpasar,358,0,Tidak ada,115,2015,0,Tahun,935801150,935801150,73.91


In [67]:
# --- ambil & bersihkan untuk urutan asli ---
data_358 = data_358[['vervar_label','tahun_label','content_label']].copy()
data_358['Kabupaten/Kota'] = (data_358['vervar_label'].astype(str)
                          .str.replace(r'<.*?>','', regex=True)
                          .str.replace(r'\s+',' ', regex=True)
                          .str.strip())
order = data_358['Kabupaten/Kota'].drop_duplicates().tolist()

# --- siapkan untuk pivot ---
df = data_358.rename(columns={'tahun_label':'tahun','content_label':'nilai'})[['Kabupaten/Kota','tahun','nilai']]
df['tahun'] = pd.to_numeric(df['tahun'], errors='coerce')
df = df.dropna(subset=['tahun']).copy()
df['tahun'] = df['tahun'].astype(int)

# --- PIVOT: kolom = tahun (MELEBAR) ---
wide = df.pivot_table(index='Kabupaten/Kota', columns='tahun', values='nilai', aggfunc='first', sort=False)

# urut kolom (tahun naik) + baris sesuai urutan awal
wide = wide.reindex(sorted(wide.columns), axis=1)
wide.index = pd.CategoricalIndex(wide.index, categories=[k for k in order if k in wide.index], ordered=True)
wide = wide.sort_index()

In [68]:
wide

tahun,2015,2016,2017,2018,2019,2020,2021,2022,2023,2024
Kabupaten/Kota,,,,,,,,,,
Kab. Jembrana,71.43,71.57,71.70,71.91,72.21,72.35,72.46,72.82,73.20,73.51
Kab. Tabanan,72.74,72.89,73.03,73.23,73.53,73.65,73.75,74.10,74.48,74.80
Kab. Badung,74.31,74.42,74.53,74.71,74.99,75.10,75.18,75.51,75.88,76.18
Kab. Gianyar,72.84,72.95,73.06,73.26,73.56,73.68,73.78,74.13,74.52,74.86
Kab. Klungkung,70.11,70.28,70.45,70.70,71.06,71.25,71.41,71.83,72.28,72.50
Kab. Bangli,69.54,69.69,69.83,70.05,70.37,70.52,70.62,70.97,71.33,71.63
Kab. Karangasem,69.48,69.66,69.85,70.05,70.35,70.47,70.56,70.89,71.25,71.53
Kab. Buleleng,70.81,70.97,71.14,71.36,71.68,71.83,71.95,72.32,72.70,72.92
Kota Denpasar,73.91,74.04,74.17,74.38,74.68,74.82,74.93,75.30,75.69,76.01


In [69]:
wide.reset_index().to_excel("data_358.xlsx", index=False, engine="openpyxl")

Harapan Lama Sekolah Provinsi Bali Menurut Kabupaten/Kota (359)

In [62]:
data_359 = getDataByVarId (domain='5100',varId='359')
data_359

,verval_val,vervar_label,var_val,turvar_val,turvar_label,tahun_val,tahun_label,turtahun_val,turtahun_label,id,content_val,content_label
0,1,Kab. Jembrana,359,0,Tidak ada,124,2024,0,Tahun,135901240,135901240,13.05
1,2,Kab. Tabanan,359,0,Tidak ada,124,2024,0,Tahun,235901240,235901240,13.29
2,3,Kab. Badung,359,0,Tidak ada,124,2024,0,Tahun,335901240,335901240,14.23
3,4,Kab. Gianyar,359,0,Tidak ada,124,2024,0,Tahun,435901240,435901240,14.11
4,5,Kab. Klungkung,359,0,Tidak ada,124,2024,0,Tahun,535901240,535901240,13.13
...,...,...,...,...,...,...,...,...,...,...,...,...
5,6,Kab. Bangli,359,0,Tidak ada,115,2015,0,Tahun,635901150,635901150,11.36
6,7,Kab. Karangasem,359,0,Tidak ada,115,2015,0,Tahun,735901150,735901150,12.11
7,8,Kab. Buleleng,359,0,Tidak ada,115,2015,0,Tahun,835901150,835901150,12.37
8,9,Kota Denpasar,359,0,Tidak ada,115,2015,0,Tahun,935901150,935901150,13.75


In [63]:
# --- ambil & bersihkan untuk urutan asli ---
data_359 = data_359[['vervar_label','tahun_label','content_label']].copy()
data_359['Kabupaten/Kota'] = (data_359['vervar_label'].astype(str)
                          .str.replace(r'<.*?>','', regex=True)
                          .str.replace(r'\s+',' ', regex=True)
                          .str.strip())
order = data_359['Kabupaten/Kota'].drop_duplicates().tolist()

# --- siapkan untuk pivot ---
df = data_359.rename(columns={'tahun_label':'tahun','content_label':'nilai'})[['Kabupaten/Kota','tahun','nilai']]
df['tahun'] = pd.to_numeric(df['tahun'], errors='coerce')
df = df.dropna(subset=['tahun']).copy()
df['tahun'] = df['tahun'].astype(int)

# --- PIVOT: kolom = tahun (MELEBAR) ---
wide = df.pivot_table(index='Kabupaten/Kota', columns='tahun', values='nilai', aggfunc='first', sort=False)

# urut kolom (tahun naik) + baris sesuai urutan awal
wide = wide.reindex(sorted(wide.columns), axis=1)
wide.index = pd.CategoricalIndex(wide.index, categories=[k for k in order if k in wide.index], ordered=True)
wide = wide.sort_index()

In [64]:
wide

tahun,2015,2016,2017,2018,2019,2020,2021,2022,2023,2024
Kabupaten/Kota,,,,,,,,,,
Kab. Jembrana,11.88,12.27,12.40,12.61,12.63,12.65,12.92,13.01,13.02,13.05
Kab. Tabanan,12.47,12.87,12.95,12.96,12.99,13.00,13.01,13.03,13.04,13.29
Kab. Badung,13.45,13.66,13.94,13.95,13.97,13.98,13.99,14.03,14.22,14.23
Kab. Gianyar,13.35,13.36,13.37,13.71,13.80,13.89,13.97,14.01,14.09,14.11
Kab. Klungkung,12.85,12.86,12.94,12.95,12.98,12.99,13.00,13.02,13.12,13.13
Kab. Bangli,11.36,11.82,12.30,12.31,12.33,12.34,12.35,12.49,12.52,12.53
Kab. Karangasem,12.11,12.33,12.38,12.39,12.40,12.41,12.42,12.62,12.63,12.91
Kab. Buleleng,12.37,12.61,12.62,12.89,12.91,13.07,13.08,13.26,13.27,13.30
Kota Denpasar,13.75,13.76,13.97,13.98,13.99,14.00,14.09,14.10,14.11,14.13


In [65]:
wide.reset_index().to_excel("data_359.xlsx", index=False, engine="openpyxl")

Rata-Rata Lama Sekolah Provinsi Bali Menurut Kabupaten/Kota (360)

In [58]:
data_360 = getDataByVarId (domain='5100',varId='360')
data_360

,verval_val,vervar_label,var_val,turvar_val,turvar_label,tahun_val,tahun_label,turtahun_val,turtahun_label,id,content_val,content_label
0,1,Kab. Jembrana,360,0,Tidak ada,124,2024,0,Tahun,136001240,136001240,8.70
1,2,Kab. Tabanan,360,0,Tidak ada,124,2024,0,Tahun,236001240,236001240,9.57
2,3,Kab. Badung,360,0,Tidak ada,124,2024,0,Tahun,336001240,336001240,11.14
3,4,Kab. Gianyar,360,0,Tidak ada,124,2024,0,Tahun,436001240,436001240,9.84
4,5,Kab. Klungkung,360,0,Tidak ada,124,2024,0,Tahun,536001240,536001240,8.74
...,...,...,...,...,...,...,...,...,...,...,...,...
5,6,Kab. Bangli,360,0,Tidak ada,115,2015,0,Tahun,636001150,636001150,6.41
6,7,Kab. Karangasem,360,0,Tidak ada,115,2015,0,Tahun,736001150,736001150,5.42
7,8,Kab. Buleleng,360,0,Tidak ada,115,2015,0,Tahun,836001150,836001150,6.77
8,9,Kota Denpasar,360,0,Tidak ada,115,2015,0,Tahun,936001150,936001150,11.02


In [ ]:
# --- ambil & bersihkan untuk urutan asli ---
data_360 = data_360[['vervar_label','tahun_label','content_label']].copy()
data_360['Kabupaten/Kota'] = (data_360['vervar_label'].astype(str)
                          .str.replace(r'<.*?>','', regex=True)
                          .str.replace(r'\s+',' ', regex=True)
                          .str.strip())
order = data_360['Kabupaten/Kota'].drop_duplicates().tolist()

# --- siapkan untuk pivot ---
df = data_360.rename(columns={'tahun_label':'tahun','content_label':'nilai'})[['Kabupaten/Kota','tahun','nilai']]
df['tahun'] = pd.to_numeric(df['tahun'], errors='coerce')
df = df.dropna(subset=['tahun']).copy()
df['tahun'] = df['tahun'].astype(int)

# --- PIVOT: kolom = tahun (MELEBAR) ---
wide = df.pivot_table(index='Kabupaten/Kota', columns='tahun', values='nilai', aggfunc='first', sort=False)

# urut kolom (tahun naik) + baris sesuai urutan awal
wide = wide.reindex(sorted(wide.columns), axis=1)
wide.index = pd.CategoricalIndex(wide.index, categories=[k for k in order if k in wide.index], ordered=True)
wide = wide.sort_index()

In [60]:
wide

tahun,2015,2016,2017,2018,2019,2020,2021,2022,2023,2024
Kabupaten/Kota,,,,,,,,,,
Kab. Jembrana,7.54,7.59,7.62,7.95,8.22,8.23,8.35,8.64,8.65,8.70
Kab. Tabanan,8.07,8.10,8.43,8.64,8.87,8.88,9.14,9.15,9.35,9.57
Kab. Badung,9.44,9.90,9.99,10.06,10.38,10.39,10.62,10.64,10.90,11.14
Kab. Gianyar,8.49,8.86,8.87,8.92,8.94,9.04,9.29,9.55,9.80,9.84
Kab. Klungkung,6.98,7.06,7.46,7.75,8.12,8.13,8.14,8.46,8.48,8.74
Kab. Bangli,6.41,6.44,6.80,7.13,7.16,7.17,7.18,7.47,7.57,7.58
Kab. Karangasem,5.42,5.48,5.52,5.97,6.31,6.32,6.33,6.67,6.68,6.70
Kab. Buleleng,6.77,6.85,7.03,7.04,7.08,7.24,7.25,7.56,7.57,7.69
Kota Denpasar,11.02,11.14,11.15,11.16,11.23,11.47,11.48,11.50,11.52,11.53


In [61]:
wide.reset_index().to_excel("data_360.xlsx", index=False, engine="openpyxl")

Rata-rata Pengeluaran Per Kapita Disesuaikan Provinsi Bali Menurut Kabupaten/Kota (361)

In [54]:
data_361 = getDataByVarId (domain='5100',varId='361')
data_361

,verval_val,vervar_label,var_val,turvar_val,turvar_label,tahun_val,tahun_label,turtahun_val,turtahun_label,id,content_val,content_label
0,1,Kab. Jembrana,361,0,Tidak ada,124,2024,0,Tahun,136101240,136101240,12638
1,2,Kab. Tabanan,361,0,Tidak ada,124,2024,0,Tahun,236101240,236101240,15297
2,3,Kab. Badung,361,0,Tidak ada,124,2024,0,Tahun,336101240,336101240,18556
3,4,Kab. Gianyar,361,0,Tidak ada,124,2024,0,Tahun,436101240,436101240,15577
4,5,Kab. Klungkung,361,0,Tidak ada,124,2024,0,Tahun,536101240,536101240,12358
...,...,...,...,...,...,...,...,...,...,...,...,...
5,6,Kab. Bangli,361,0,Tidak ada,115,2015,0,Tahun,636101150,636101150,10649
6,7,Kab. Karangasem,361,0,Tidak ada,115,2015,0,Tahun,736101150,736101150,9556
7,8,Kab. Buleleng,361,0,Tidak ada,115,2015,0,Tahun,836101150,836101150,12587
8,9,Kota Denpasar,361,0,Tidak ada,115,2015,0,Tahun,936101150,936101150,18849


In [55]:
# --- ambil & bersihkan untuk urutan asli ---
data_361 = data_361[['vervar_label','tahun_label','content_label']].copy()
data_361['Kabupaten/Kota'] = (data_361['vervar_label'].astype(str)
                          .str.replace(r'<.*?>','', regex=True)
                          .str.replace(r'\s+',' ', regex=True)
                          .str.strip())
order = data_361['Kabupaten/Kota'].drop_duplicates().tolist()

# --- siapkan untuk pivot ---
df = data_361.rename(columns={'tahun_label':'tahun','content_label':'nilai'})[['Kabupaten/Kota','tahun','nilai']]
df['tahun'] = pd.to_numeric(df['tahun'], errors='coerce')
df = df.dropna(subset=['tahun']).copy()
df['tahun'] = df['tahun'].astype(int)

# --- PIVOT: kolom = tahun (MELEBAR) ---
wide = df.pivot_table(index='Kabupaten/Kota', columns='tahun', values='nilai', aggfunc='first', sort=False)

# urut kolom (tahun naik) + baris sesuai urutan awal
wide = wide.reindex(sorted(wide.columns), axis=1)
wide.index = pd.CategoricalIndex(wide.index, categories=[k for k in order if k in wide.index], ordered=True)
wide = wide.sort_index()

In [56]:
wide

tahun,2015,2016,2017,2018,2019,2020,2021,2022,2023,2024
Kabupaten/Kota,,,,,,,,,,
Kab. Jembrana,11168,11343,11468,11666,11902,11790,11675,11915,12236,12638
Kab. Tabanan,13665,13800,13923,14245,14608,14494,14326,14475,14832,15297
Kab. Badung,16409,16567,17063,17325,17628,17503,17327,17445,17915,18556
Kab. Gianyar,13578,13766,14222,14376,14623,14544,14391,14630,15047,15577
Kab. Klungkung,10711,10852,11005,11318,11484,11376,11287,11500,11760,12358
Kab. Bangli,10649,10819,10956,11160,11369,11268,11201,11424,11670,12148
Kab. Karangasem,9556,9690,9833,10050,10302,10237,10175,10278,10753,11176
Kab. Buleleng,12587,12814,12995,13235,13780,13463,13362,13529,13987,14377
Kota Denpasar,18849,19084,19364,19698,19992,19723,19598,19850,20128,20763


In [ ]:
wide.reset_index().to_excel("data_361.xlsx", index=False, engine="openpyxl")

[Metode Baru] Umur Harapan Hidup Saat Lahir (UHH) Menurut Kabupaten/Kota di Provinsi Bali Hasil Long Form SP2020 (400)

In [30]:
data_400 = getDataByVarId (domain='5100',varId='400')
data_400

,verval_val,vervar_label,var_val,turvar_val,turvar_label,tahun_val,tahun_label,turtahun_val,turtahun_label,id,content_val,content_label
0,1,Kab. Jembrana,400,0,Tidak ada,124,2024,0,Tahun,140001240,140001240,75.07
1,2,Kab. Tabanan,400,0,Tidak ada,124,2024,0,Tahun,240001240,240001240,75.67
2,3,Kab. Badung,400,0,Tidak ada,124,2024,0,Tahun,340001240,340001240,75.90
3,4,Kab. Gianyar,400,0,Tidak ada,124,2024,0,Tahun,440001240,440001240,75.72
4,5,Kab. Klungkung,400,0,Tidak ada,124,2024,0,Tahun,540001240,540001240,74.90
5,6,Kab. Bangli,400,0,Tidak ada,124,2024,0,Tahun,640001240,640001240,74.23
6,7,Kab. Karangasem,400,0,Tidak ada,124,2024,0,Tahun,740001240,740001240,74.17
7,8,Kab. Buleleng,400,0,Tidak ada,124,2024,0,Tahun,840001240,840001240,74.71
8,9,Kota Denpasar,400,0,Tidak ada,124,2024,0,Tahun,940001240,940001240,75.80
9,10,<b>Provinsi Bali</b>,400,0,Tidak ada,124,2024,0,Tahun,1040001240,1040001240,75.10


In [32]:
# --- ambil & bersihkan untuk urutan asli ---
data_400 = data_400[['vervar_label','tahun_label','content_label']].copy()
data_400['Kabupaten/Kota'] = (data_400['vervar_label'].astype(str)
                          .str.replace(r'<.*?>','', regex=True)
                          .str.replace(r'\s+',' ', regex=True)
                          .str.strip())
order = data_400['Kabupaten/Kota'].drop_duplicates().tolist()

# --- siapkan untuk pivot ---
df = data_400.rename(columns={'tahun_label':'tahun','content_label':'nilai'})[['Kabupaten/Kota','tahun','nilai']]
df['tahun'] = pd.to_numeric(df['tahun'], errors='coerce')
df = df.dropna(subset=['tahun']).copy()
df['tahun'] = df['tahun'].astype(int)

# --- PIVOT: kolom = tahun (MELEBAR) ---
wide = df.pivot_table(index='Kabupaten/Kota', columns='tahun', values='nilai', aggfunc='first', sort=False)

# urut kolom (tahun naik) + baris sesuai urutan awal
wide = wide.reindex(sorted(wide.columns), axis=1)
wide.index = pd.CategoricalIndex(wide.index, categories=[k for k in order if k in wide.index], ordered=True)
wide = wide.sort_index()

In [33]:
wide

tahun,2020,2021,2022,2023,2024
Kabupaten/Kota,,,,,
Kab. Jembrana,74.16,74.25,74.56,74.84,75.07
Kab. Tabanan,74.72,74.81,75.10,75.41,75.67
Kab. Badung,75.11,75.19,75.52,75.73,75.90
Kab. Gianyar,74.76,74.84,75.13,75.45,75.72
Kab. Klungkung,73.97,74.11,74.48,74.61,74.90
Kab. Bangli,73.28,73.36,73.67,73.98,74.23
Kab. Karangasem,73.10,73.35,73.63,73.93,74.17
Kab. Buleleng,74.12,74.22,74.55,74.65,74.71
Kota Denpasar,74.87,74.96,75.33,75.59,75.80


In [34]:
wide.reset_index().to_excel("data_400.xlsx", index=False, engine="openpyxl")

 [Metode Baru] Indeks Pembangunan Manusia menurut Menurut Kabupaten/Kota di Provinsi Bali (Umur Harapan Hidup Hasil Long Form SP2020) (401)

In [26]:
data_401 = getDataByVarId (domain='5100',varId='401')
data_401

,verval_val,vervar_label,var_val,turvar_val,turvar_label,tahun_val,tahun_label,turtahun_val,turtahun_label,id,content_val,content_label
0,1,Kab. Jembrana,401,0,Tidak ada,124,2024,0,Tahun,140101240,140101240,75.32
1,2,Kab. Tabanan,401,0,Tidak ada,124,2024,0,Tahun,240101240,240101240,78.83
2,3,Kab. Badung,401,0,Tidak ada,124,2024,0,Tahun,340101240,340101240,83.73
3,4,Kab. Gianyar,401,0,Tidak ada,124,2024,0,Tahun,440101240,440101240,80.23
4,5,Kab. Klungkung,401,0,Tidak ada,124,2024,0,Tahun,540101240,540101240,75.15
5,6,Kab. Bangli,401,0,Tidak ada,124,2024,0,Tahun,640101240,640101240,72.51
6,7,Kab. Karangasem,401,0,Tidak ada,124,2024,0,Tahun,740101240,740101240,70.91
7,8,Kab. Buleleng,401,0,Tidak ada,124,2024,0,Tahun,840101240,840101240,75.35
8,9,Kota Denpasar,401,0,Tidak ada,124,2024,0,Tahun,940101240,940101240,85.11
9,10,<b>Provinsi Bali</b>,401,0,Tidak ada,124,2024,0,Tahun,1040101240,1040101240,78.63


In [31]:
# --- ambil & bersihkan untuk urutan asli ---
data_401 = data_401[['vervar_label','tahun_label','content_label']].copy()
data_401['Kabupaten/Kota'] = (data_401['vervar_label'].astype(str)
                          .str.replace(r'<.*?>','', regex=True)
                          .str.replace(r'\s+',' ', regex=True)
                          .str.strip())
order = data_401['Kabupaten/Kota'].drop_duplicates().tolist()

# --- siapkan untuk pivot ---
df = data_401.rename(columns={'tahun_label':'tahun','content_label':'nilai'})[['Kabupaten/Kota','tahun','nilai']]
df['tahun'] = pd.to_numeric(df['tahun'], errors='coerce')
df = df.dropna(subset=['tahun']).copy()
df['tahun'] = df['tahun'].astype(int)

# --- PIVOT: kolom = tahun (MELEBAR) ---
wide = df.pivot_table(index='Kabupaten/Kota', columns='tahun', values='nilai', aggfunc='first', sort=False)

# urut kolom (tahun naik) + baris sesuai urutan awal
wide = wide.reindex(sorted(wide.columns), axis=1)
wide.index = pd.CategoricalIndex(wide.index, categories=[k for k in order if k in wide.index], ordered=True)
wide = wide.sort_index()

In [28]:
wide

tahun,2020,2021,2022,2023,2024
Kabupaten/Kota,,,,,
Kab. Jembrana,73.18,73.57,74.38,74.80,75.32
Kab. Tabanan,76.67,76.95,77.22,77.87,78.83
Kab. Badung,81.60,81.84,82.13,83.00,83.73
Kab. Gianyar,77.88,78.21,78.87,79.69,80.23
Kab. Klungkung,72.97,72.98,73.77,74.18,75.15
Kab. Bangli,70.60,70.60,71.47,71.99,72.51
Kab. Karangasem,68.50,68.58,69.48,70.09,70.91
Kab. Buleleng,73.60,73.60,74.48,74.87,75.35
Kota Denpasar,83.95,84.04,84.39,84.68,85.11


In [29]:
wide.reset_index().to_excel("data_401.xlsx", index=False, engine="openpyxl")

Indeks Demokrasi Provinsi Bali Menurut Aspek (377)

In [50]:
data_377 = getDataByVarId (domain='5100',varId='377')
data_377

,verval_val,vervar_label,var_val,turvar_val,turvar_label,tahun_val,tahun_label,turtahun_val,turtahun_label,id,content_val,content_label
0,1,Aspek Kebebasan Sipil,377,0,Tidak ada,120,2020,0,Tahun,137701200,137701200,96.86
1,2,Hak-hak Politik,377,0,Tidak ada,120,2020,0,Tahun,237701200,237701200,69.07
2,3,Lembaga Demokrasi,377,0,Tidak ada,120,2020,0,Tahun,337701200,337701200,66.90
3,4,Skor Indeks Demokrasi,377,0,Tidak ada,120,2020,0,Tahun,437701200,437701200,77.59
0,1,Aspek Kebebasan Sipil,377,0,Tidak ada,119,2019,0,Tahun,137701190,137701190,90.60
...,...,...,...,...,...,...,...,...,...,...,...,...
3,4,Skor Indeks Demokrasi,377,0,Tidak ada,112,2012,0,Tahun,437701120,437701120,71.75
0,1,Aspek Kebebasan Sipil,377,0,Tidak ada,111,2011,0,Tahun,137701110,137701110,95.44
1,2,Hak-hak Politik,377,0,Tidak ada,111,2011,0,Tahun,237701110,237701110,50.08
2,3,Lembaga Demokrasi,377,0,Tidak ada,111,2011,0,Tahun,337701110,337701110,85.79


In [51]:
# --- ambil & bersihkan untuk urutan asli ---
data_377 = data_377[['vervar_label','tahun_label','content_label']].copy()
data_377['Aspek Kebebasan Sipil'] = (data_377['vervar_label'].astype(str)
                          .str.replace(r'<.*?>','', regex=True)
                          .str.replace(r'\s+',' ', regex=True)
                          .str.strip())
order = data_377['Aspek Kebebasan Sipil'].drop_duplicates().tolist()

# --- siapkan untuk pivot ---
df = data_377.rename(columns={'tahun_label':'tahun','content_label':'nilai'})[['Aspek Kebebasan Sipil','tahun','nilai']]
df['tahun'] = pd.to_numeric(df['tahun'], errors='coerce')
df = df.dropna(subset=['tahun']).copy()
df['tahun'] = df['tahun'].astype(int)

# --- PIVOT: kolom = tahun (MELEBAR) ---
wide = df.pivot_table(index='Aspek Kebebasan Sipil', columns='tahun', values='nilai', aggfunc='first', sort=False)

# urut kolom (tahun naik) + baris sesuai urutan awal
wide = wide.reindex(sorted(wide.columns), axis=1)
wide.index = pd.CategoricalIndex(wide.index, categories=[k for k in order if k in wide.index], ordered=True)
wide = wide.sort_index()

In [52]:
wide

tahun,2011,2012,2013,2014,2015,2016,2017,2018,2019,2020
Aspek Kebebasan Sipil,,,,,,,,,,
Aspek Kebebasan Sipil,95.44,94.18,92.55,92.16,94.42,96.94,95.51,94.72,90.60,96.86
Hak-hak Politik,50.08,45.00,52.87,61.27,77.42,69.60,66.80,68.08,68.38,69.07
Lembaga Demokrasi,85.79,86.01,77.38,79.56,65.31,71.18,76.85,89.55,90.42,66.90
Skor Indeks Demokrasi,74.20,71.75,72.22,76.13,79.83,78.95,78.80,82.37,81.38,77.59


In [53]:
wide.reset_index().to_excel("data_377.xlsx", index=False, engine="openpyxl")

Indeks Demokrasi Provinsi Bali Menurut Variabel (378)

In [46]:
data_378 = getDataByVarId (domain='5100',varId='378')
data_378

,verval_val,vervar_label,var_val,turvar_val,turvar_label,tahun_val,tahun_label,turtahun_val,turtahun_label,id,content_val,content_label
0,1,Kebebasan berkumpul dan berserikat,378,0,Tidak ada,120,2020,0,Tahun,137801200,137801200,100.00
1,2,Kebebasan berpendapat,378,0,Tidak ada,120,2020,0,Tahun,237801200,237801200,100.00
2,3,Kebebasan berkeyakinan,378,0,Tidak ada,120,2020,0,Tahun,337801200,337801200,94.63
3,4,Kebebasan dari diskriminasi,378,0,Tidak ada,120,2020,0,Tahun,437801200,437801200,100.00
4,5,Hak memilih dan dipilih,378,0,Tidak ada,120,2020,0,Tahun,537801200,537801200,83.27
...,...,...,...,...,...,...,...,...,...,...,...,...
8,9,Peran partai politik,378,0,Tidak ada,112,2012,0,Tahun,937801120,937801120,99.47
9,10,Peran birokrasi pemerintah daerah,378,0,Tidak ada,112,2012,0,Tahun,1037801120,1037801120,98.48
10,11,Peran peradilan yang independen,378,0,Tidak ada,112,2012,0,Tahun,1137801120,1137801120,90.00
11,12,Skor Indeks Demokrasi,378,0,Tidak ada,112,2012,0,Tahun,1237801120,1237801120,71.75


In [47]:
# --- ambil & bersihkan untuk urutan asli ---
data_378 = data_378[['vervar_label','tahun_label','content_label']].copy()
data_378['Variabel Indeks Demokrasi'] = (data_378['vervar_label'].astype(str)
                          .str.replace(r'<.*?>','', regex=True)
                          .str.replace(r'\s+',' ', regex=True)
                          .str.strip())
order = data_378['Variabel Indeks Demokrasi'].drop_duplicates().tolist()

# --- siapkan untuk pivot ---
df = data_378.rename(columns={'tahun_label':'tahun','content_label':'nilai'})[['Variabel Indeks Demokrasi','tahun','nilai']]
df['tahun'] = pd.to_numeric(df['tahun'], errors='coerce')
df = df.dropna(subset=['tahun']).copy()
df['tahun'] = df['tahun'].astype(int)

# --- PIVOT: kolom = tahun (MELEBAR) ---
wide = df.pivot_table(index='Variabel Indeks Demokrasi', columns='tahun', values='nilai', aggfunc='first', sort=False)

# urut kolom (tahun naik) + baris sesuai urutan awal
wide = wide.reindex(sorted(wide.columns), axis=1)
wide.index = pd.CategoricalIndex(wide.index, categories=[k for k in order if k in wide.index], ordered=True)
wide = wide.sort_index()

In [48]:
wide

tahun,2011,2012,2013,2014,2015,2016,2017,2018,2019,2020
Variabel Indeks Demokrasi,,,,,,,,,,
Kebebasan berkumpul dan berserikat,NaN,95.63,100.00,55.00,96.88,100.00,100.00,67.19,67.19,100.00
Kebebasan berpendapat,NaN,61.09,74.97,76.10,56.94,89.59,72.89,72.89,52.06,100.00
Kebebasan berkeyakinan,NaN,98.21,97.08,98.21,97.79,96.42,100.00,100.00,96.26,94.63
Kebebasan dari diskriminasi,NaN,96.70,85.70,96.70,99.90,100.00,91.75,100.00,100.00,100.00
Hak memilih dan dipilih,NaN,50.55,51.13,78.92,79.01,78.92,78.92,79.50,83.27,83.27
Partisipasi politik dalam pengambilan keputusan dan pengawasan,NaN,39.46,54.61,43.62,75.82,60.28,54.68,56.66,53.50,54.88
Pemilu yang bebas dan adil,NaN,98.10,98.10,93.67,93.67,93.67,93.67,93.67,89.64,89.64
Peran DPRD,NaN,44.25,43.68,45.46,40.94,44.40,58.58,58.74,63.52,47.22
Peran partai politik,NaN,99.47,44.71,61.43,20.85,48.57,35.71,100.00,100.00,100.00


In [49]:
wide.reset_index().to_excel("data_378.xlsx", index=False, engine="openpyxl")

Indeks Demokrasi Provinsi Bali Menurut Aspek (Metode Baru) (381)

In [39]:
data_381 = getDataByVarId (domain='5100',varId='381')
data_381

,verval_val,vervar_label,var_val,turvar_val,turvar_label,tahun_val,tahun_label,turtahun_val,turtahun_label,id,content_val,content_label
0,1,Kebebasan,381,0,Tidak ada,123,2023,0,Tahun,138101230,138101230,85.23
1,2,Kesetaraan,381,0,Tidak ada,123,2023,0,Tahun,238101230,238101230,85.75
2,3,Kapasitas Lembaga Demokrasi,381,0,Tidak ada,123,2023,0,Tahun,338101230,338101230,84.28
3,4,<b>Skor Indeks Demokrasi</b>,381,0,Tidak ada,123,2023,0,Tahun,438101230,438101230,85.13
0,1,Kebebasan,381,0,Tidak ada,122,2022,0,Tahun,138101220,138101220,85.78
1,2,Kesetaraan,381,0,Tidak ada,122,2022,0,Tahun,238101220,238101220,83.90
2,3,Kapasitas Lembaga Demokrasi,381,0,Tidak ada,122,2022,0,Tahun,338101220,338101220,79.83
3,4,<b>Skor Indeks Demokrasi</b>,381,0,Tidak ada,122,2022,0,Tahun,438101220,438101220,83.21
0,1,Kebebasan,381,0,Tidak ada,121,2021,0,Tahun,138101210,138101210,84.62
1,2,Kesetaraan,381,0,Tidak ada,121,2021,0,Tahun,238101210,238101210,77.25


In [42]:
# --- ambil & bersihkan untuk urutan asli ---
data_381 = data_381[['vervar_label','tahun_label','content_label']].copy()
data_381['Aspek Indeks Demokrasi'] = (data_381['vervar_label'].astype(str)
                          .str.replace(r'<.*?>','', regex=True)
                          .str.replace(r'\s+',' ', regex=True)
                          .str.strip())
order = data_381['Aspek Indeks Demokrasi'].drop_duplicates().tolist()

# --- siapkan untuk pivot ---
df = data_381.rename(columns={'tahun_label':'tahun','content_label':'nilai'})[['Aspek Indeks Demokrasi','tahun','nilai']]
df['tahun'] = pd.to_numeric(df['tahun'], errors='coerce')
df = df.dropna(subset=['tahun']).copy()
df['tahun'] = df['tahun'].astype(int)

# --- PIVOT: kolom = tahun (MELEBAR) ---
wide = df.pivot_table(index='Aspek Indeks Demokrasi', columns='tahun', values='nilai', aggfunc='first', sort=False)

# urut kolom (tahun naik) + baris sesuai urutan awal
wide = wide.reindex(sorted(wide.columns), axis=1)
wide.index = pd.CategoricalIndex(wide.index, categories=[k for k in order if k in wide.index], ordered=True)
wide = wide.sort_index()

In [43]:
wide

tahun,2021,2022,2023
Aspek Indeks Demokrasi,,,
Kebebasan,84.62,85.78,85.23
Kesetaraan,77.25,83.90,85.75
Kapasitas Lembaga Demokrasi,63.92,79.83,84.28
Skor Indeks Demokrasi,75.35,83.21,85.13


In [45]:
wide.reset_index().to_excel("data_381.xlsx", index=False, engine="openpyxl")

Banyaknya Produk DPRD Provinsi Bali (393)

In [22]:
data_393 = getDataByVarId (domain='5100',varId='393')
data_393

,verval_val,vervar_label,var_val,turvar_val,turvar_label,tahun_val,tahun_label,turtahun_val,turtahun_label,id,content_val,content_label
0,1,Peraturan Daerah,393,0,Tidak ada,124,2024,0,Tahun,139301240,139301240,7
1,2,Keputusan Dewan,393,0,Tidak ada,124,2024,0,Tahun,239301240,239301240,30
2,3,Keputusan Pimpinan Dewan,393,0,Tidak ada,124,2024,0,Tahun,339301240,339301240,7
3,4,Jumlah,393,0,Tidak ada,124,2024,0,Tahun,439301240,439301240,44
0,1,Peraturan Daerah,393,0,Tidak ada,123,2023,0,Tahun,139301230,139301230,15
...,...,...,...,...,...,...,...,...,...,...,...,...
3,4,Jumlah,393,0,Tidak ada,116,2016,0,Tahun,439301160,439301160,48
0,1,Peraturan Daerah,393,0,Tidak ada,115,2015,0,Tahun,139301150,139301150,11
1,2,Keputusan Dewan,393,0,Tidak ada,115,2015,0,Tahun,239301150,239301150,33
2,3,Keputusan Pimpinan Dewan,393,0,Tidak ada,115,2015,0,Tahun,339301150,339301150,7


In [23]:
# --- ambil & bersihkan untuk urutan asli ---
data_393 = data_393[['vervar_label','tahun_label','content_label']].copy()
data_393['Kabupaten/Kota'] = (data_393['vervar_label'].astype(str)
                          .str.replace(r'<.*?>','', regex=True)
                          .str.replace(r'\s+',' ', regex=True)
                          .str.strip())
order = data_393['Kabupaten/Kota'].drop_duplicates().tolist()

# --- siapkan untuk pivot ---
df = data_393.rename(columns={'tahun_label':'tahun','content_label':'nilai'})[['Kabupaten/Kota','tahun','nilai']]
df['tahun'] = pd.to_numeric(df['tahun'], errors='coerce')
df = df.dropna(subset=['tahun']).copy()
df['tahun'] = df['tahun'].astype(int)

# --- PIVOT: kolom = tahun (MELEBAR) ---
wide = df.pivot_table(index='Kabupaten/Kota', columns='tahun', values='nilai', aggfunc='first', sort=False)

# urut kolom (tahun naik) + baris sesuai urutan awal
wide = wide.reindex(sorted(wide.columns), axis=1)
wide.index = pd.CategoricalIndex(wide.index, categories=[k for k in order if k in wide.index], ordered=True)
wide = wide.sort_index()

In [24]:
wide

tahun,2015,2016,2017,2018,2019,2020,2021,2022,2023,2024
Kabupaten/Kota,,,,,,,,,,
Peraturan Daerah,11,10,10,13,12,11,11,14,15,7
Keputusan Dewan,33,32,38,40,39,30,38,27,40,30
Keputusan Pimpinan Dewan,7,6,3,10,1,6,6,4,4,7
Jumlah,51,48,51,48,51,47,55,45,59,44


In [25]:
wide.reset_index().to_excel("data_393.xlsx", index=False, engine="openpyxl")

Banyaknya Tindak Pidana Menonjol Menurut Jenisnya di Provinsi Bali (394)

In [12]:
data_394 = getDataByVarId (domain='5100',varId='394')
data_394

,verval_val,vervar_label,var_val,turvar_val,turvar_label,tahun_val,tahun_label,turtahun_val,turtahun_label,id,content_val,content_label
0,1,Pencurian dengan Pemberatan,394,0,Tidak ada,124,2024,0,Tahun,139401240,139401240,448
1,2,Penipuan,394,0,Tidak ada,124,2024,0,Tahun,239401240,239401240,378
2,3,Pencurian Biasa,394,0,Tidak ada,124,2024,0,Tahun,339401240,339401240,946
3,4,Penggelapan,394,0,Tidak ada,124,2024,0,Tahun,439401240,439401240,458
4,5,Penganiayaan Ringan,394,0,Tidak ada,124,2024,0,Tahun,539401240,539401240,590
...,...,...,...,...,...,...,...,...,...,...,...,...
6,7,Narkotika,394,0,Tidak ada,115,2015,0,Tahun,739401150,739401150,396
7,8,Perjudian,394,0,Tidak ada,115,2015,0,Tahun,839401150,839401150,390
8,9,Pencurian dengan Kekerasan,394,0,Tidak ada,115,2015,0,Tahun,939401150,939401150,66
9,10,Lain-lain,394,0,Tidak ada,115,2015,0,Tahun,1039401150,1039401150,4725


In [17]:
# --- ambil & bersihkan untuk urutan asli ---
data_394 = data_394[['vervar_label','tahun_label','content_label']].copy()
data_394['Kabupaten/Kota'] = (data_394['vervar_label'].astype(str)
                          .str.replace(r'<.*?>','', regex=True)
                          .str.replace(r'\s+',' ', regex=True)
                          .str.strip())
order = data_394['Kabupaten/Kota'].drop_duplicates().tolist()

# --- siapkan untuk pivot ---
df = data_394.rename(columns={'tahun_label':'tahun','content_label':'nilai'})[['Kabupaten/Kota','tahun','nilai']]
df['tahun'] = pd.to_numeric(df['tahun'], errors='coerce')
df = df.dropna(subset=['tahun']).copy()
df['tahun'] = df['tahun'].astype(int)

# --- PIVOT: kolom = tahun (MELEBAR) ---
wide = df.pivot_table(index='Kabupaten/Kota', columns='tahun', values='nilai', aggfunc='first', sort=False)

# urut kolom (tahun naik) + baris sesuai urutan awal
wide = wide.reindex(sorted(wide.columns), axis=1)
wide.index = pd.CategoricalIndex(wide.index, categories=[k for k in order if k in wide.index], ordered=True)
wide = wide.sort_index()

In [18]:
wide

tahun,2015,2016,2017,2018,2019,2020,2021,2022,2023,2024
Kabupaten/Kota,,,,,,,,,,
Pencurian dengan Pemberatan,534,425,413,420,361,265,240,344,371,448
Penipuan,253,239,185,115,107,83,88,220,381,378
Pencurian Biasa,460,378,396,466,474,289,355,577,739,946
Penggelapan,388,318,225,218,202,131,142,244,375,458
Penganiayaan Ringan,129,225,101,89,5,275,255,368,503,590
Penganiayaan Berat,15,2,6,4,7,12,7,5,9,47
Narkotika,396,546,685,818,830,770,718,718,811,997
Perjudian,390,339,190,129,68,55,75,80,39,39
Pencurian dengan Kekerasan,66,51,66,59,58,53,27,68,31,34


In [20]:
wide.reset_index().to_excel("data_394.xlsx", index=False, engine="openpyxl")

Indeks Pemberdayaan Gender (IDG) Provinsi Bali Menurut Kabupaten/Kota (134)

In [100]:
data_134 = getDataByVarId (domain='5100',varId='134')
data_134

,verval_val,vervar_label,var_val,turvar_val,turvar_label,tahun_val,tahun_label,turtahun_val,turtahun_label,id,content_val,content_label
0,1,Kab. Jembrana,134,0,Tidak ada,124,2024,0,Tahun,113401240,113401240,72.94
1,2,Kab. Tabanan,134,0,Tidak ada,124,2024,0,Tahun,213401240,213401240,75.48
2,3,Kab. Badung,134,0,Tidak ada,124,2024,0,Tahun,313401240,313401240,64.20
3,4,Kab. Gianyar,134,0,Tidak ada,124,2024,0,Tahun,413401240,413401240,73.38
4,5,Kab. Klungkung,134,0,Tidak ada,124,2024,0,Tahun,513401240,513401240,69.98
...,...,...,...,...,...,...,...,...,...,...,...,...
5,6,Kab. Bangli,134,0,Tidak ada,115,2015,0,Tahun,613401150,613401150,61.12
6,7,Kab. Karangasem,134,0,Tidak ada,115,2015,0,Tahun,713401150,713401150,60.24
7,8,Kab. Buleleng,134,0,Tidak ada,115,2015,0,Tahun,813401150,813401150,65.15
8,9,Kota Denpasar,134,0,Tidak ada,115,2015,0,Tahun,913401150,913401150,58.50


In [102]:
data_134 = data_134[['verval_val','vervar_label','tahun_label','content_label']].copy()
data_134["vervar_label"] = data_134["vervar_label"].str.replace(r"<.*?>", "", regex=True)

data_tb = data_134.rename(columns={
    'vervar_label'   : 'Kabupaten/Kota',
    'tahun_label'    : 'Tahun',
    'content_label'  : 'Nilai'
})

api_order = data_134['vervar_label'].drop_duplicates().tolist()
data_tb['Kabupaten/Kota'] = pd.Categorical(data_tb['Kabupaten/Kota'], categories=api_order, ordered=True)

data_tb = data_tb.pivot_table(
    index=['Kabupaten/Kota'],
    columns=['Tahun'],
    values='Nilai'
)

data_tb.columns = [
    '_'.join([str(c) for c in col if c]).strip() if isinstance(col, tuple) else str(col) 
    for col in data_tb.columns.values
]

data_tb = data_tb.reset_index()

data_tb = data_tb.applymap(
    lambda x: int(x) if isinstance(x, (int, float)) and x.is_integer() else x
)

data_tb


C:\Users\user\AppData\Local\Temp\ipykernel_17372\3111838973.py:13: FutureWarning: The default value of observed=False is deprecated and will change to observed=True in a future version of pandas. Specify observed=False to silence this warning and retain the current behavior
  data_tb = data_tb.pivot_table(
C:\Users\user\AppData\Local\Temp\ipykernel_17372\3111838973.py:26: FutureWarning: DataFrame.applymap has been deprecated. Use DataFrame.map instead.
  data_tb = data_tb.applymap(


,Kabupaten/Kota,2015,2016,2017,2018,2019,2020,2021,2022,2023,2024
0,Kab. Jembrana,65.07,NaN,66.23,66.23,74.60,74.39,74.55,72.40,72.44,72.94
1,Kab. Tabanan,59.56,NaN,61.06,61.62,78.14,77.95,78.27,78.51,78.07,75.48
2,Kab. Badung,58.80,NaN,61.48,62.03,75.23,75.49,76.99,76.33,76.08,64.20
3,Kab. Gianyar,61.45,NaN,62.35,62.59,66.22,66.14,65.64,68.58,68.82,73.38
4,Kab. Klungkung,74.89,NaN,72.60,71.77,78.35,78.36,75.50,75.75,76.07,69.98
5,Kab. Bangli,61.12,NaN,59.57,61.07,61.81,65.16,65.36,65.69,65.57,69.06
6,Kab. Karangasem,60.24,NaN,59.30,60.20,60.77,63.61,65.09,64.59,65.79,72.31
7,Kab. Buleleng,65.15,NaN,67.68,65.58,73.13,72.89,72.91,73.73,72.57,66.71
8,Kota Denpasar,58.50,NaN,58.80,59.09,62.16,67.26,67.02,67.21,69.22,67.40
9,Provinsi Bali,62.99,63.97,63.76,64.18,72.27,72.16,72.17,72.29,73.77,74.38


In [103]:
data_tb.to_excel("data_134.xlsx", sheet_name= "IDG", index=False)

70  Persentase Penduduk Miskin Provinsi Bali Menurut Klasifikasi Daerah

In [82]:
data_70 = getDataByVarId (domain='5100',varId='70')
data_70

,verval_val,vervar_label,var_val,turvar_val,turvar_label,tahun_val,tahun_label,turtahun_val,turtahun_label,id,content_val,content_label
0,1,Perkotaan,70,164,Maret,125,2025,0,Tahun,1701641250,1701641250,3.27
2,2,Perdesaan,70,164,Maret,125,2025,0,Tahun,2701641250,2701641250,4.97
4,3,Perkotaan + Perdesaan,70,164,Maret,125,2025,0,Tahun,3701641250,3701641250,3.72
0,1,Perkotaan,70,164,Maret,124,2024,0,Tahun,1701641240,1701641240,3.55
1,1,Perkotaan,70,165,September,124,2024,0,Tahun,1701651240,1701651240,3.32
...,...,...,...,...,...,...,...,...,...,...,...,...
1,1,Perkotaan,70,165,September,116,2016,0,Tahun,1701651160,1701651160,3.53
2,2,Perdesaan,70,164,Maret,116,2016,0,Tahun,2701641160,2701641160,5.23
3,2,Perdesaan,70,165,September,116,2016,0,Tahun,2701651160,2701651160,5.21
4,3,Perkotaan + Perdesaan,70,164,Maret,116,2016,0,Tahun,3701641160,3701641160,4.25


In [83]:
# --- ambil & bersihkan untuk urutan asli ---
data_70 = data_70[['vervar_label','tahun_label','content_label']].copy()
data_70['Daerah'] = (data_70['vervar_label'].astype(str)
                          .str.replace(r'<.*?>','', regex=True)
                          .str.replace(r'\s+',' ', regex=True)
                          .str.strip())
order = data_70['Daerah'].drop_duplicates().tolist()

# --- siapkan untuk pivot ---
df = data_70.rename(columns={'tahun_label':'tahun','content_label':'nilai'})[['Daerah','tahun','nilai']]
df['tahun'] = pd.to_numeric(df['tahun'], errors='coerce')
df = df.dropna(subset=['tahun']).copy()
df['tahun'] = df['tahun'].astype(int)

# --- PIVOT: kolom = tahun (MELEBAR) ---
wide = df.pivot_table(index='Daerah', columns='tahun', values='nilai', aggfunc='first', sort=False)

# urut kolom (tahun naik) + baris sesuai urutan awal
wide = wide.reindex(sorted(wide.columns), axis=1)
wide.index = pd.CategoricalIndex(wide.index, categories=[k for k in order if k in wide.index], ordered=True)
wide = wide.sort_index()

In [ ]:
wide